In [ ]:
import torch
print(torch.cuda.is_available())  # Should print True if GPU is available

True


In [ ]:
import torch
print(torch.cuda.is_available())  # Should print True if GPU is available

True


In [ ]:
!pip install llama-cpp-python diffusers torch torchvision transformers accelerate numpy pillow opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 12.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install llama-cpp-python torch diffusers transformers accelerate safetensors matplotlib Pillow gradio

In [ ]:
import os
import torch
import matplotlib.pyplot as plt
from llama_cpp import Llama
from diffusers import StableDiffusionXLPipeline
from PIL import Image
import gradio as gr

# Load the Llama Model
def load_llama_model(model_path=None):
    # If no model path provided, use default path
    if not model_path:
        model_path = "/content/drive/MyDrive/llama-2-7b.Q3_K_S.gguf"

    # Check if model exists at the specified path
    if not os.path.exists(model_path):
        print(f"Warning: Model not found at {model_path}")
        print("You may need to adjust the model path or download the model first.")
        raise FileNotFoundError(f"Model not found at {model_path}")

    model = Llama(
        model_path=model_path,
        n_ctx=2048,
        verbose=False  # Reduce the logging output
    )
    return model

# Enhanced story generation function
def generate_story(model, prompt, max_tokens=500):
    # Create a more specific prompt that encourages creative storytelling
    story_prompt = f"""<s>[INST] Write an original, engaging short story based on the following prompt:

    "{prompt}"

    Make it creative with a clear beginning, middle, and end. Include interesting characters,
    dialogue, and descriptive language. The story should be approximately 200-300 words. [/INST]"""

    try:
        # Increase temperature for more creative outputs
        response = model(
            story_prompt,
            max_tokens=max_tokens,
            temperature=0.85,    # Slightly higher temperature for creativity
            top_p=0.92,          # Slightly higher top_p for more varied outputs
            frequency_penalty=0.3,  # Reduce repetition
            presence_penalty=0.3,   # Encourage novel wording
            echo=False,
            stop=["</s>", "[INST]"]  # Stop at proper ending tokens
        )

        generated_text = response['choices'][0]['text'].strip()

        # Only fall back if the response is truly problematic
        if len(generated_text) < 50 or "[INST]" in generated_text:
            print("Warning: Low quality story generated. Retrying with different parameters...")

            # Second attempt with different parameters
            retry_response = model(
                story_prompt,
                max_tokens=max_tokens,
                temperature=0.75,  # More moderate temperature
                top_p=0.85,        # More focused sampling
                frequency_penalty=0.2,
                presence_penalty=0.2,
                echo=False,
                stop=["</s>", "[INST]"]
            )

            retry_text = retry_response['choices'][0]['text'].strip()

            if len(retry_text) >= 50 and "[INST]" not in retry_text:
                return retry_text
            else:
                # Only use fallback as a last resort
                print("Warning: Retry failed. Creating a dynamic fallback story.")
                # Create a more dynamic fallback story based on the prompt
                return create_dynamic_fallback_story(prompt)

        return generated_text
    except Exception as e:
        print(f"Error in story generation: {e}")
        return create_dynamic_fallback_story(prompt)

# Function to create a more varied fallback story when needed
def create_dynamic_fallback_story(prompt):
    # Extract potential characters or themes from the prompt
    words = prompt.lower().split()

    # Look for keywords that might be character types or settings
    character_keywords = ["about", "with", "featuring", "starring", "where"]
    setting_keywords = ["in", "at", "during", "through"]

    characters = "characters"
    setting = "a mysterious place"

    # Extract potential character information
    for keyword in character_keywords:
        if keyword in words:
            idx = words.index(keyword)
            if idx + 1 < len(words):
                characters = " ".join(words[idx+1:min(idx+6, len(words))])
                break

    # Extract potential setting information
    for keyword in setting_keywords:
        if keyword in words:
            idx = words.index(keyword)
            if idx + 1 < len(words):
                setting = " ".join(words[idx+1:min(idx+4, len(words))])
                break

    # Create one of several story templates using the extracted information
    import random
    templates = [
        f"The sun cast long shadows across {setting} as {characters} embarked on an unexpected journey. "
        f"What began as an ordinary day quickly transformed when they discovered a hidden secret that would change everything. "
        f"Through challenges and revelations, they found strength in each other they never knew existed. "
        f"By the journey's end, nothing would ever be the same for them again.",

        f"A chance encounter in {setting} brought {characters} together at precisely the right moment. "
        f"Neither could have anticipated how their lives would intertwine, nor the adventure that awaited them. "
        f"Facing impossible odds, they were forced to trust each other despite their differences. "
        f"Their story became one that would be told for generations to come.",

        f"Deep within {setting}, {characters} discovered something that should have remained hidden. "
        f"As they unraveled ancient mysteries, tension grew between them, testing the limits of their relationship. "
        f"A moment of truth revealed their true character when everything was at stake. "
        f"In the end, they emerged transformed, carrying both scars and wisdom from their extraordinary experience."
    ]

    # Return a randomly selected template
    return random.choice(templates)

# Function to split the story into panels
def split_story_into_panels(story, num_panels=4):
    # First check if we have enough content to split
    if len(story) < 100:
        print("Warning: Story is very short. Panel division may not be optimal.")

    # Try to split by paragraphs first for more natural divisions
    paragraphs = story.split('\n')
    paragraphs = [p for p in paragraphs if p.strip()]  # Remove empty paragraphs

    if len(paragraphs) >= num_panels:
        # We have enough paragraphs, so use them as natural divisions
        if len(paragraphs) > num_panels:
            # Combine paragraphs to match the desired number of panels
            step = len(paragraphs) / num_panels
            panels = []
            for i in range(num_panels):
                start_idx = int(i * step)
                end_idx = int((i + 1) * step) if i < num_panels - 1 else len(paragraphs)
                panel_text = " ".join(paragraphs[start_idx:end_idx])
                panels.append(panel_text)
            return panels
        else:
            # Perfect match or fewer paragraphs than panels
            return paragraphs[:num_panels]

    # If we don't have enough paragraphs, fall back to sentence splitting
    sentences = []
    # Split by common sentence endings
    for chunk in story.split('. '):
        sentences.extend([s.strip() + '.' for s in chunk.split('! ')])
    sentences = [s for s in sentences if len(s) > 1]  # Remove empty or too-short sentences

    # Handle edge case where there are fewer sentences than panels
    if len(sentences) < num_panels:
        # Create more sentences by splitting the longest ones
        while len(sentences) < num_panels:
            longest_idx = max(range(len(sentences)), key=lambda i: len(sentences[i]))
            longest = sentences[longest_idx]

            # Split at a comma or other natural break point if possible
            if ', ' in longest:
                parts = longest.split(', ', 1)
                sentences[longest_idx] = parts[0] + '.'
                sentences.insert(longest_idx + 1, parts[1])
            else:
                # If no good split point, duplicate the sentence with a variation
                sentences.append(f"Meanwhile, {longest.lower()}")

    # Calculate how many sentences per panel for a balanced distribution
    step = max(1, len(sentences) // num_panels)

    panels = []
    for i in range(num_panels):
        start_idx = i * step
        # For the last panel, include any remaining sentences
        end_idx = (i + 1) * step if i < num_panels - 1 else len(sentences)
        panel_text = " ".join(sentences[start_idx:end_idx])
        panels.append(panel_text)

    return panels

# Load the Stable Diffusion XL model
def load_sdxl_model():
    print("Loading SDXL model - this may take a while...")
    pipe = StableDiffusionXLPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True
    )

    # Check if CUDA is available, otherwise use CPU
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device == "cpu":
        print("Warning: CUDA not available, using CPU. This will be very slow.")
    pipe = pipe.to(device)

    return pipe

# Function to generate comic panels from any story prompt
def generate_comic(story_prompt, num_panels=4, model_path=None, art_style="cartoon style"):
    # Make sure the story prompt is compelling
    if len(story_prompt) < 10:
        story_prompt = f"Create an interesting story about {story_prompt}"

    try:
        # Load the Llama model
        print("Loading Llama model...")
        llama_model = load_llama_model(model_path)

        # Generate story from prompt with multiple attempts if needed
        print(f"Generating story from prompt: '{story_prompt}'")
        story_text = generate_story(llama_model, story_prompt)
        print("Story generated:")
        print(story_text)

        # Check if story generated properly
        if not story_text or len(story_text) < 50:
            print("Story generation produced insufficient content. Trying one more time with different approach.")
            # Try a more direct approach with the model
            direct_prompt = f"""<s>[INST] Write a creative short story for children about {story_prompt}. Make it exciting with interesting characters and events. [/INST]"""

            retry_response = llama_model(
                direct_prompt,
                max_tokens=600,
                temperature=0.8,
                top_p=0.9,
                echo=False,
                stop=["</s>", "[INST]"]
            )

            story_text = retry_response['choices'][0]['text'].strip()

            if len(story_text) < 50:
                print("Still unable to generate a good story. Using the last fallback option.")
                story_text = (
                    f"In a world where anything was possible, {story_prompt} became reality. "
                    f"The adventure began unexpectedly one morning when strange events unfolded. "
                    f"Characters found themselves facing mysterious challenges that required courage and wit. "
                    f"After many twists and turns, they discovered a solution that brought everything to a satisfying conclusion."
                )
    except Exception as e:
        print(f"Error generating story: {e}")
        # Provide a fallback story
        story_text = (
            f"In a world where anything was possible, {story_prompt} became reality. "
            f"The adventure began unexpectedly one morning when strange events unfolded. "
            f"Characters found themselves facing mysterious challenges that required courage and wit. "
            f"After many twists and turns, they discovered a solution that brought everything to a satisfying conclusion."
        )

    # Split story into panels
    print("\nSplitting story into panels...")
    story_panels = split_story_into_panels(story_text, num_panels)
    for i, panel in enumerate(story_panels):
        print(f"Panel {i+1}: {panel}")

    try:
        # Load SDXL model
        pipe = load_sdxl_model()

        print("\nGenerating panel images...")
        panel_images = []
        for i, panel_text in enumerate(story_panels):
            print(f"Generating image for panel {i+1}...")
            # Create a prompt that helps SDXL understand it's a comic panel
            # Use the art style specified by the user
            prompt = f"A detailed {art_style} illustration for a comic panel showing: {panel_text} cinematic lighting, expressive characters, vibrant colors"

            # Generate the image
            image = pipe(
                prompt=prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                num_inference_steps=30
            ).images[0]
            panel_images.append(image)

            # Save the image
            image_path = f"panel_{i+1}.png"
            image.save(image_path)

        print("Comic generation complete! All panels saved as PNG files.")
        return story_text, story_panels, panel_images

    except Exception as e:
        print(f"Error in image generation: {e}")
        print("Unable to generate images. Please check if you have enough GPU memory and proper installations.")
        return story_text, story_panels, None

# Web UI using Gradio
def create_ui():
    with gr.Blocks() as app:
        gr.Markdown("# Story to Comic Generator")
        gr.Markdown("Enter a prompt to generate a unique story and convert it into comic panels")

        with gr.Row():
            with gr.Column():
                story_prompt = gr.Textbox(
                    label="Story Prompt",
                    placeholder="Write a story about a brave cat who learns to fly",
                    lines=2
                )
                num_panels = gr.Slider(minimum=2, maximum=8, value=4, step=1, label="Number of Panels")
                model_path = gr.Textbox(label="Optional: Custom Llama Model Path", placeholder="Leave empty for default")
                art_style = gr.Dropdown(
                    choices=[
                        "cartoon style",
                        "manga style",
                        "comic book style",
                        "watercolor style",
                        "3D rendered",
                        "pixel art style",
                        "oil painting style",
                        "storybook illustration style"
                    ],
                    value="cartoon style",
                    label="Art Style"
                )
                generate_button = gr.Button("Generate Comic")

            with gr.Column():
                story_output = gr.Textbox(label="Generated Story", lines=8)

        # Fixed Gallery component
        panel_gallery = gr.Gallery(label="Comic Panels", columns=2, rows=2, height=600)

        # Status output for error messages
        status_output = gr.Textbox(label="Status", lines=2)

        def run_generation(prompt, panels, path, style):
            if not prompt:
                return "Please enter a story prompt", [], "Error: No prompt provided"

            model_path_to_use = path if path.strip() else None

            try:
                story_text, story_panels, images = generate_comic(prompt, int(panels), model_path_to_use, style)

                if story_text and images:
                    status = "Success: Comic generated successfully!"
                elif story_text:
                    status = "Warning: Story generated but image creation failed"
                else:
                    status = "Error: Story generation failed"

                return story_text, images if images else [], status
            except Exception as e:
                return f"An error occurred: {str(e)}", [], f"Error: {str(e)}"

        generate_button.click(
            run_generation,
            inputs=[story_prompt, num_panels, model_path, art_style],
            outputs=[story_output, panel_gallery, status_output]
        )

    return app

# Main function to either run the generator directly or launch the web UI
def main(use_web_ui=True):
    if use_web_ui:
        app = create_ui()
        app.launch(share=True)  # Enable sharing for remote access
    else:
        # Example without web UI
        story_prompt = input("Enter a story prompt: ")
        num_panels = int(input("Enter number of panels (2-8): "))
        art_style = input("Enter art style (e.g., cartoon style, manga style): ")

        story, panels, images = generate_comic(story_prompt, num_panels, None, art_style)

        if images:
            for i, img in enumerate(images):
                plt.figure(figsize=(8, 8))
                plt.imshow(img)
                plt.title(f"Panel {i+1}")
                plt.axis('off')
                plt.show()

if __name__ == "__main__":
    main(use_web_ui=True)  # Set to False to use command line instead of web UI

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


2025/04/23 07:03:22 [W] [service.go:132] login to server failed: dial tcp 44.237.78.176:7000: i/o timeout


<IPython.core.display.Javascript object>

In [ ]:
import os
import torch
import matplotlib.pyplot as plt
from llama_cpp import Llama
from diffusers import StableDiffusionXLPipeline
from PIL import Image
import gradio as gr

# Load the Llama Model
def load_llama_model(model_path=None):
    # If no model path provided, use default path
    if not model_path:
        model_path = "/content/drive/MyDrive/llama-2-7b.Q3_K_S.gguf"

    # Check if model exists at the specified path
    if not os.path.exists(model_path):
        print(f"Warning: Model not found at {model_path}")
        print("You may need to adjust the model path or download the model first.")
        raise FileNotFoundError(f"Model not found at {model_path}")

    model = Llama(
        model_path=model_path,
        n_ctx=2048,
        verbose=False  # Reduce the logging output
    )
    return model

# Enhanced story generation function
def generate_story(model, prompt, max_tokens=500):
    # Create a more specific prompt that encourages creative storytelling
    story_prompt = f"""<s>[INST] Write an original, engaging short story based on the following prompt:

    "{prompt}"

    Make it creative with a clear beginning, middle, and end. Include interesting characters,
    dialogue, and descriptive language. The story should be approximately 200-300 words. [/INST]"""

    try:
        # Increase temperature for more creative outputs
        response = model(
            story_prompt,
            max_tokens=max_tokens,
            temperature=0.85,    # Slightly higher temperature for creativity
            top_p=0.92,          # Slightly higher top_p for more varied outputs
            frequency_penalty=0.3,  # Reduce repetition
            presence_penalty=0.3,   # Encourage novel wording
            echo=False,
            stop=["</s>", "[INST]"]  # Stop at proper ending tokens
        )

        generated_text = response['choices'][0]['text'].strip()

        # Only fall back if the response is truly problematic
        if len(generated_text) < 50 or "[INST]" in generated_text:
            print("Warning: Low quality story generated. Retrying with different parameters...")

            # Second attempt with different parameters
            retry_response = model(
                story_prompt,
                max_tokens=max_tokens,
                temperature=0.75,  # More moderate temperature
                top_p=0.85,        # More focused sampling
                frequency_penalty=0.2,
                presence_penalty=0.2,
                echo=False,
                stop=["</s>", "[INST]"]
            )

            retry_text = retry_response['choices'][0]['text'].strip()

            if len(retry_text) >= 50 and "[INST]" not in retry_text:
                return retry_text
            else:
                # Only use fallback as a last resort
                print("Warning: Retry failed. Creating a dynamic fallback story.")
                # Create a more dynamic fallback story based on the prompt
                return create_dynamic_fallback_story(prompt)

        return generated_text
    except Exception as e:
        print(f"Error in story generation: {e}")
        return create_dynamic_fallback_story(prompt)

# Function to create a more varied fallback story when needed
def create_dynamic_fallback_story(prompt):
    # Extract potential characters or themes from the prompt
    words = prompt.lower().split()

    # Look for keywords that might be character types or settings
    character_keywords = ["about", "with", "featuring", "starring", "where"]
    setting_keywords = ["in", "at", "during", "through"]

    characters = "characters"
    setting = "a mysterious place"

    # Extract potential character information
    for keyword in character_keywords:
        if keyword in words:
            idx = words.index(keyword)
            if idx + 1 < len(words):
                characters = " ".join(words[idx+1:min(idx+6, len(words))])
                break

    # Extract potential setting information
    for keyword in setting_keywords:
        if keyword in words:
            idx = words.index(keyword)
            if idx + 1 < len(words):
                setting = " ".join(words[idx+1:min(idx+4, len(words))])
                break

    # Create one of several story templates using the extracted information
    import random
    templates = [
        f"The sun cast long shadows across {setting} as {characters} embarked on an unexpected journey. "
        f"What began as an ordinary day quickly transformed when they discovered a hidden secret that would change everything. "
        f"Through challenges and revelations, they found strength in each other they never knew existed. "
        f"By the journey's end, nothing would ever be the same for them again.",

        f"A chance encounter in {setting} brought {characters} together at precisely the right moment. "
        f"Neither could have anticipated how their lives would intertwine, nor the adventure that awaited them. "
        f"Facing impossible odds, they were forced to trust each other despite their differences. "
        f"Their story became one that would be told for generations to come.",

        f"Deep within {setting}, {characters} discovered something that should have remained hidden. "
        f"As they unraveled ancient mysteries, tension grew between them, testing the limits of their relationship. "
        f"A moment of truth revealed their true character when everything was at stake. "
        f"In the end, they emerged transformed, carrying both scars and wisdom from their extraordinary experience."
    ]

    # Return a randomly selected template
    return random.choice(templates)

# Function to split the story into panels
def split_story_into_panels(story, num_panels=4):
    # First check if we have enough content to split
    if len(story) < 100:
        print("Warning: Story is very short. Panel division may not be optimal.")

    # Try to split by paragraphs first for more natural divisions
    paragraphs = story.split('\n')
    paragraphs = [p for p in paragraphs if p.strip()]  # Remove empty paragraphs

    if len(paragraphs) >= num_panels:
        # We have enough paragraphs, so use them as natural divisions
        if len(paragraphs) > num_panels:
            # Combine paragraphs to match the desired number of panels
            step = len(paragraphs) / num_panels
            panels = []
            for i in range(num_panels):
                start_idx = int(i * step)
                end_idx = int((i + 1) * step) if i < num_panels - 1 else len(paragraphs)
                panel_text = " ".join(paragraphs[start_idx:end_idx])
                panels.append(panel_text)
            return panels
        else:
            # Perfect match or fewer paragraphs than panels
            return paragraphs[:num_panels]

    # If we don't have enough paragraphs, fall back to sentence splitting
    sentences = []
    # Split by common sentence endings
    for chunk in story.split('. '):
        sentences.extend([s.strip() + '.' for s in chunk.split('! ')])
    sentences = [s for s in sentences if len(s) > 1]  # Remove empty or too-short sentences

    # Handle edge case where there are fewer sentences than panels
    if len(sentences) < num_panels:
        # Create more sentences by splitting the longest ones
        while len(sentences) < num_panels:
            longest_idx = max(range(len(sentences)), key=lambda i: len(sentences[i]))
            longest = sentences[longest_idx]

            # Split at a comma or other natural break point if possible
            if ', ' in longest:
                parts = longest.split(', ', 1)
                sentences[longest_idx] = parts[0] + '.'
                sentences.insert(longest_idx + 1, parts[1])
            else:
                # If no good split point, duplicate the sentence with a variation
                sentences.append(f"Meanwhile, {longest.lower()}")

    # Calculate how many sentences per panel for a balanced distribution
    step = max(1, len(sentences) // num_panels)

    panels = []
    for i in range(num_panels):
        start_idx = i * step
        # For the last panel, include any remaining sentences
        end_idx = (i + 1) * step if i < num_panels - 1 else len(sentences)
        panel_text = " ".join(sentences[start_idx:end_idx])
        panels.append(panel_text)

    return panels

# Load the Stable Diffusion XL model
def load_sdxl_model():
    print("Loading SDXL model - this may take a while...")
    pipe = StableDiffusionXLPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True
    )

    # Check if CUDA is available, otherwise use CPU
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device == "cpu":
        print("Warning: CUDA not available, using CPU. This will be very slow.")
    pipe = pipe.to(device)

    return pipe

# Function to generate comic panels from any story prompt
def generate_comic(story_prompt, num_panels=4, model_path=None, art_style="cartoon style"):
    # Make sure the story prompt is compelling
    if len(story_prompt) < 10:
        story_prompt = f"Create an interesting story about {story_prompt}"

    try:
        # Load the Llama model
        print("Loading Llama model...")
        llama_model = load_llama_model(model_path)

        # Generate story from prompt with multiple attempts if needed
        print(f"Generating story from prompt: '{story_prompt}'")
        story_text = generate_story(llama_model, story_prompt)
        print("Story generated:")
        print(story_text)

        # Check if story generated properly
        if not story_text or len(story_text) < 50:
            print("Story generation produced insufficient content. Trying one more time with different approach.")
            # Try a more direct approach with the model
            direct_prompt = f"""<s>[INST] Write a creative short story for children about {story_prompt}. Make it exciting with interesting characters and events. [/INST]"""

            retry_response = llama_model(
                direct_prompt,
                max_tokens=600,
                temperature=0.8,
                top_p=0.9,
                echo=False,
                stop=["</s>", "[INST]"]
            )

            story_text = retry_response['choices'][0]['text'].strip()

            if len(story_text) < 50:
                print("Still unable to generate a good story. Using the last fallback option.")
                story_text = (
                    f"In a world where anything was possible, {story_prompt} became reality. "
                    f"The adventure began unexpectedly one morning when strange events unfolded. "
                    f"Characters found themselves facing mysterious challenges that required courage and wit. "
                    f"After many twists and turns, they discovered a solution that brought everything to a satisfying conclusion."
                )
    except Exception as e:
        print(f"Error generating story: {e}")
        # Provide a fallback story
        story_text = (
            f"In a world where anything was possible, {story_prompt} became reality. "
            f"The adventure began unexpectedly one morning when strange events unfolded. "
            f"Characters found themselves facing mysterious challenges that required courage and wit. "
            f"After many twists and turns, they discovered a solution that brought everything to a satisfying conclusion."
        )

    # Split story into panels
    print("\nSplitting story into panels...")
    story_panels = split_story_into_panels(story_text, num_panels)
    for i, panel in enumerate(story_panels):
        print(f"Panel {i+1}: {panel}")

    try:
        # Load SDXL model
        pipe = load_sdxl_model()

        print("\nGenerating panel images...")
        panel_images = []
        for i, panel_text in enumerate(story_panels):
            print(f"Generating image for panel {i+1}...")
            # Create a prompt that helps SDXL understand it's a comic panel
            # Use the art style specified by the user
            prompt = f"A detailed {art_style} illustration for a comic panel showing: {panel_text} cinematic lighting, expressive characters, vibrant colors"

            # Generate the image
            image = pipe(
                prompt=prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                num_inference_steps=30
            ).images[0]
            panel_images.append(image)

            # Save the image
            image_path = f"panel_{i+1}.png"
            image.save(image_path)

        print("Comic generation complete! All panels saved as PNG files.")
        return story_text, story_panels, panel_images

    except Exception as e:
        print(f"Error in image generation: {e}")
        print("Unable to generate images. Please check if you have enough GPU memory and proper installations.")
        return story_text, story_panels, None

# Web UI using Gradio
def create_ui():
    with gr.Blocks() as app:
        gr.Markdown("# Story to Comic Generator")
        gr.Markdown("Enter a prompt to generate a unique story and convert it into comic panels")

        with gr.Row():
            with gr.Column():
                story_prompt = gr.Textbox(
                    label="Story Prompt",
                    placeholder="Write a story about a brave cat who learns to fly",
                    lines=2
                )
                num_panels = gr.Slider(minimum=2, maximum=8, value=4, step=1, label="Number of Panels")
                model_path = gr.Textbox(label="Optional: Custom Llama Model Path", placeholder="Leave empty for default")
                art_style = gr.Dropdown(
                    choices=[
                        "cartoon style",
                        "manga style",
                        "comic book style",
                        "watercolor style",
                        "3D rendered",
                        "pixel art style",
                        "oil painting style",
                        "storybook illustration style"
                    ],
                    value="cartoon style",
                    label="Art Style"
                )
                generate_button = gr.Button("Generate Comic")

            with gr.Column():
                story_output = gr.Textbox(label="Generated Story", lines=8)

        # Fixed Gallery component
        panel_gallery = gr.Gallery(label="Comic Panels", columns=2, rows=2, height=600)

        # Status output for error messages
        status_output = gr.Textbox(label="Status", lines=2)

        def run_generation(prompt, panels, path, style):
            if not prompt:
                return "Please enter a story prompt", [], "Error: No prompt provided"

            model_path_to_use = path if path.strip() else None

            try:
                story_text, story_panels, images = generate_comic(prompt, int(panels), model_path_to_use, style)

                if story_text and images:
                    status = "Success: Comic generated successfully!"
                elif story_text:
                    status = "Warning: Story generated but image creation failed"
                else:
                    status = "Error: Story generation failed"

                return story_text, images if images else [], status
            except Exception as e:
                return f"An error occurred: {str(e)}", [], f"Error: {str(e)}"

        generate_button.click(
            run_generation,
            inputs=[story_prompt, num_panels, model_path, art_style],
            outputs=[story_output, panel_gallery, status_output]
        )

    return app

# Main function to either run the generator directly or launch the web UI
def main(use_web_ui=True):
    if use_web_ui:
        app = create_ui()
        app.launch(share=True)  # Enable sharing for remote access
    else:
        # Example without web UI
        story_prompt = input("Enter a story prompt: ")
        num_panels = int(input("Enter number of panels (2-8): "))
        art_style = input("Enter art style (e.g., cartoon style, manga style): ")

        story, panels, images = generate_comic(story_prompt, num_panels, None, art_style)

        if images:
            for i, img in enumerate(images):
                plt.figure(figsize=(8, 8))
                plt.imshow(img)
                plt.title(f"Panel {i+1}")
                plt.axis('off')
                plt.show()

if __name__ == "__main__":
    main(use_web_ui=True)  # Set to False to use command line instead of web UI

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5a730fe9ca58cee57a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import os
import torch
import matplotlib.pyplot as plt
from llama_cpp import Llama
from diffusers import StableDiffusionXLPipeline
from PIL import Image
import gradio as gr

# Load the Llama Model
def load_llama_model(model_path=None):
    # If no model path provided, use default path
    if not model_path:
        model_path = "/content/drive/MyDrive/llama-2-7b.Q3_K_S.gguf"

    # Check if model exists at the specified path
    if not os.path.exists(model_path):
        print(f"Warning: Model not found at {model_path}")
        print("You may need to adjust the model path or download the model first.")
        raise FileNotFoundError(f"Model not found at {model_path}")

    model = Llama(
        model_path=model_path,
        n_ctx=2048,
        verbose=False  # Reduce the logging output
    )
    return model

# Enhanced story generation function
def generate_story(model, prompt, max_tokens=500):
    # Create a more specific prompt that encourages creative storytelling
    story_prompt = f"""<s>[INST] Write an original, engaging short story based on the following prompt:

    "{prompt}"

    Make it creative with a clear beginning, middle, and end. Include interesting characters,
    dialogue, and descriptive language. The story should be approximately 200-300 words. [/INST]"""

    try:
        # Increase temperature for more creative outputs
        response = model(
            story_prompt,
            max_tokens=max_tokens,
            temperature=0.85,    # Slightly higher temperature for creativity
            top_p=0.92,          # Slightly higher top_p for more varied outputs
            frequency_penalty=0.3,  # Reduce repetition
            presence_penalty=0.3,   # Encourage novel wording
            echo=False,
            stop=["</s>", "[INST]"]  # Stop at proper ending tokens
        )

        generated_text = response['choices'][0]['text'].strip()

        # Only fall back if the response is truly problematic
        if len(generated_text) < 50 or "[INST]" in generated_text:
            print("Warning: Low quality story generated. Retrying with different parameters...")

            # Second attempt with different parameters
            retry_response = model(
                story_prompt,
                max_tokens=max_tokens,
                temperature=0.75,  # More moderate temperature
                top_p=0.85,        # More focused sampling
                frequency_penalty=0.2,
                presence_penalty=0.2,
                echo=False,
                stop=["</s>", "[INST]"]
            )

            retry_text = retry_response['choices'][0]['text'].strip()

            if len(retry_text) >= 50 and "[INST]" not in retry_text:
                return retry_text
            else:
                # Only use fallback as a last resort
                print("Warning: Retry failed. Creating a dynamic fallback story.")
                # Create a more dynamic fallback story based on the prompt
                return create_dynamic_fallback_story(prompt)

        return generated_text
    except Exception as e:
        print(f"Error in story generation: {e}")
        return create_dynamic_fallback_story(prompt)

# Function to create a more varied fallback story when needed
def create_dynamic_fallback_story(prompt):
    # Extract potential characters or themes from the prompt
    words = prompt.lower().split()

    # Look for keywords that might be character types or settings
    character_keywords = ["about", "with", "featuring", "starring", "where"]
    setting_keywords = ["in", "at", "during", "through"]

    characters = "characters"
    setting = "a mysterious place"

    # Extract potential character information
    for keyword in character_keywords:
        if keyword in words:
            idx = words.index(keyword)
            if idx + 1 < len(words):
                characters = " ".join(words[idx+1:min(idx+6, len(words))])
                break

    # Extract potential setting information
    for keyword in setting_keywords:
        if keyword in words:
            idx = words.index(keyword)
            if idx + 1 < len(words):
                setting = " ".join(words[idx+1:min(idx+4, len(words))])
                break

    # Create one of several story templates using the extracted information
    import random
    templates = [
        f"The sun cast long shadows across {setting} as {characters} embarked on an unexpected journey. "
        f"What began as an ordinary day quickly transformed when they discovered a hidden secret that would change everything. "
        f"Through challenges and revelations, they found strength in each other they never knew existed. "
        f"By the journey's end, nothing would ever be the same for them again.",

        f"A chance encounter in {setting} brought {characters} together at precisely the right moment. "
        f"Neither could have anticipated how their lives would intertwine, nor the adventure that awaited them. "
        f"Facing impossible odds, they were forced to trust each other despite their differences. "
        f"Their story became one that would be told for generations to come.",

        f"Deep within {setting}, {characters} discovered something that should have remained hidden. "
        f"As they unraveled ancient mysteries, tension grew between them, testing the limits of their relationship. "
        f"A moment of truth revealed their true character when everything was at stake. "
        f"In the end, they emerged transformed, carrying both scars and wisdom from their extraordinary experience."
    ]

    # Return a randomly selected template
    return random.choice(templates)

# Function to split the story into panels
def split_story_into_panels(story, num_panels=4):
    # First check if we have enough content to split
    if len(story) < 100:
        print("Warning: Story is very short. Panel division may not be optimal.")

    # Try to split by paragraphs first for more natural divisions
    paragraphs = story.split('\n')
    paragraphs = [p for p in paragraphs if p.strip()]  # Remove empty paragraphs

    if len(paragraphs) >= num_panels:
        # We have enough paragraphs, so use them as natural divisions
        if len(paragraphs) > num_panels:
            # Combine paragraphs to match the desired number of panels
            step = len(paragraphs) / num_panels
            panels = []
            for i in range(num_panels):
                start_idx = int(i * step)
                end_idx = int((i + 1) * step) if i < num_panels - 1 else len(paragraphs)
                panel_text = " ".join(paragraphs[start_idx:end_idx])
                panels.append(panel_text)
            return panels
        else:
            # Perfect match or fewer paragraphs than panels
            return paragraphs[:num_panels]

    # If we don't have enough paragraphs, fall back to sentence splitting
    sentences = []
    # Split by common sentence endings
    for chunk in story.split('. '):
        sentences.extend([s.strip() + '.' for s in chunk.split('! ')])
    sentences = [s for s in sentences if len(s) > 1]  # Remove empty or too-short sentences

    # Handle edge case where there are fewer sentences than panels
    if len(sentences) < num_panels:
        # Create more sentences by splitting the longest ones
        while len(sentences) < num_panels:
            longest_idx = max(range(len(sentences)), key=lambda i: len(sentences[i]))
            longest = sentences[longest_idx]

            # Split at a comma or other natural break point if possible
            if ', ' in longest:
                parts = longest.split(', ', 1)
                sentences[longest_idx] = parts[0] + '.'
                sentences.insert(longest_idx + 1, parts[1])
            else:
                # If no good split point, duplicate the sentence with a variation
                sentences.append(f"Meanwhile, {longest.lower()}")

    # Calculate how many sentences per panel for a balanced distribution
    step = max(1, len(sentences) // num_panels)

    panels = []
    for i in range(num_panels):
        start_idx = i * step
        # For the last panel, include any remaining sentences
        end_idx = (i + 1) * step if i < num_panels - 1 else len(sentences)
        panel_text = " ".join(sentences[start_idx:end_idx])
        panels.append(panel_text)

    return panels

# Load the Stable Diffusion XL model
def load_sdxl_model():
    print("Loading SDXL model - this may take a while...")
    pipe = StableDiffusionXLPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True
    )

    # Check if CUDA is available, otherwise use CPU
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device == "cpu":
        print("Warning: CUDA not available, using CPU. This will be very slow.")
    pipe = pipe.to(device)

    return pipe

# Function to generate comic panels from any story prompt
def generate_comic(story_prompt, num_panels=4, model_path=None, art_style="cartoon style"):
    # Make sure the story prompt is compelling
    if len(story_prompt) < 10:
        story_prompt = f"Create an interesting story about {story_prompt}"

    try:
        # Load the Llama model
        print("Loading Llama model...")
        llama_model = load_llama_model(model_path)

        # Generate story from prompt with multiple attempts if needed
        print(f"Generating story from prompt: '{story_prompt}'")
        story_text = generate_story(llama_model, story_prompt)
        print("Story generated:")
        print(story_text)

        # Check if story generated properly
        if not story_text or len(story_text) < 50:
            print("Story generation produced insufficient content. Trying one more time with different approach.")
            # Try a more direct approach with the model
            direct_prompt = f"""<s>[INST] Write a creative short story for children about {story_prompt}. Make it exciting with interesting characters and events. [/INST]"""

            retry_response = llama_model(
                direct_prompt,
                max_tokens=600,
                temperature=0.8,
                top_p=0.9,
                echo=False,
                stop=["</s>", "[INST]"]
            )

            story_text = retry_response['choices'][0]['text'].strip()

            if len(story_text) < 50:
                print("Still unable to generate a good story. Using the last fallback option.")
                story_text = (
                    f"In a world where anything was possible, {story_prompt} became reality. "
                    f"The adventure began unexpectedly one morning when strange events unfolded. "
                    f"Characters found themselves facing mysterious challenges that required courage and wit. "
                    f"After many twists and turns, they discovered a solution that brought everything to a satisfying conclusion."
                )
    except Exception as e:
        print(f"Error generating story: {e}")
        # Provide a fallback story
        story_text = (
            f"In a world where anything was possible, {story_prompt} became reality. "
            f"The adventure began unexpectedly one morning when strange events unfolded. "
            f"Characters found themselves facing mysterious challenges that required courage and wit. "
            f"After many twists and turns, they discovered a solution that brought everything to a satisfying conclusion."
        )

    # Split story into panels
    print("\nSplitting story into panels...")
    story_panels = split_story_into_panels(story_text, num_panels)
    for i, panel in enumerate(story_panels):
        print(f"Panel {i+1}: {panel}")

    try:
        # Load SDXL model
        pipe = load_sdxl_model()

        print("\nGenerating panel images...")
        panel_images = []
        for i, panel_text in enumerate(story_panels):
            print(f"Generating image for panel {i+1}...")
            # Create a prompt that helps SDXL understand it's a comic panel
            # Use the art style specified by the user
            prompt = f"A detailed {art_style} illustration for a comic panel showing: {panel_text} cinematic lighting, expressive characters, vibrant colors"

            # Generate the image
            image = pipe(
                prompt=prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                num_inference_steps=30
            ).images[0]
            panel_images.append(image)

            # Save the image
            image_path = f"panel_{i+1}.png"
            image.save(image_path)

        print("Comic generation complete! All panels saved as PNG files.")
        return story_text, story_panels, panel_images

    except Exception as e:
        print(f"Error in image generation: {e}")
        print("Unable to generate images. Please check if you have enough GPU memory and proper installations.")
        return story_text, story_panels, None

# Web UI using Gradio
def create_ui():
    with gr.Blocks() as app:
        gr.Markdown("# Story to Comic Generator")
        gr.Markdown("Enter a prompt to generate a unique story and convert it into comic panels")

        with gr.Row():
            with gr.Column():
                story_prompt = gr.Textbox(
                    label="Story Prompt",
                    placeholder="Write a story about a brave cat who learns to fly",
                    lines=2
                )
                num_panels = gr.Slider(minimum=2, maximum=8, value=4, step=1, label="Number of Panels")
                model_path = gr.Textbox(label="Optional: Custom Llama Model Path", placeholder="Leave empty for default")
                art_style = gr.Dropdown(
                    choices=[
                        "cartoon style",
                        "manga style",
                        "comic book style",
                        "watercolor style",
                        "3D rendered",
                        "pixel art style",
                        "oil painting style",
                        "storybook illustration style"
                    ],
                    value="cartoon style",
                    label="Art Style"
                )
                generate_button = gr.Button("Generate Comic")

            with gr.Column():
                story_output = gr.Textbox(label="Generated Story", lines=8)

        # Fixed Gallery component
        panel_gallery = gr.Gallery(label="Comic Panels", columns=2, rows=2, height=600)

        # Status output for error messages
        status_output = gr.Textbox(label="Status", lines=2)

        def run_generation(prompt, panels, path, style):
            if not prompt:
                return "Please enter a story prompt", [], "Error: No prompt provided"

            model_path_to_use = path if path.strip() else None

            try:
                story_text, story_panels, images = generate_comic(prompt, int(panels), model_path_to_use, style)

                if story_text and images:
                    status = "Success: Comic generated successfully!"
                elif story_text:
                    status = "Warning: Story generated but image creation failed"
                else:
                    status = "Error: Story generation failed"

                return story_text, images if images else [], status
            except Exception as e:
                return f"An error occurred: {str(e)}", [], f"Error: {str(e)}"

        generate_button.click(
            run_generation,
            inputs=[story_prompt, num_panels, model_path, art_style],
            outputs=[story_output, panel_gallery, status_output]
        )

    return app

# Main function to either run the generator directly or launch the web UI
def main(use_web_ui=True):
    if use_web_ui:
        app = create_ui()
        app.launch(share=True)  # Enable sharing for remote access
    else:
        # Example without web UI
        story_prompt = input("Enter a story prompt: ")
        num_panels = int(input("Enter number of panels (2-8): "))
        art_style = input("Enter art style (e.g., cartoon style, manga style): ")

        story, panels, images = generate_comic(story_prompt, num_panels, None, art_style)

        if images:
            for i, img in enumerate(images):
                plt.figure(figsize=(8, 8))
                plt.imshow(img)
                plt.title(f"Panel {i+1}")
                plt.axis('off')
                plt.show()

if __name__ == "__main__":
    main(use_web_ui=True)  # Set to False to use command line instead of web UI

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://99acdb205225c2de02.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import os
import torch
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForCausalLM
from diffusers import StableDiffusionXLPipeline
from PIL import Image
import gradio as gr



# Load the fine-tuned model
def load_fine_tuned_model(model_path=None):
    # If no model path provided, use default path
    if not model_path:
        model_path = '/content/drive/MyDrive/fine_tuned_tiny_stories_model'

    try:
        print(f"Loading fine-tuned model from {model_path}...")
        model = AutoModelForCausalLM.from_pretrained(model_path)
        tokenizer = AutoTokenizer.from_pretrained(model_path)

        # Move model to GPU if available
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)

        print(f"Model loaded successfully and moved to {device}")
        return model, tokenizer
    except Exception as e:
        print(f"Error loading model: {e}")
        return None, None

# Helper functions for story generation
def extract_character_name(prompt):
    """Extract potential character names from the prompt"""
    # Common superhero/character names to check for
    character_names = ["Spider-Man", "Peter Parker", "Batman", "Superman", "Wonder Woman",
                       "Iron Man", "Captain America", "Thor", "Hulk", "Black Widow"]

    # Check if any known character names are in the prompt
    for name in character_names:
        if name.lower() in prompt.lower():
            return name

    # Otherwise extract words that might be names (capitalized words)
    words = prompt.split()
    for word in words:
        if word[0].isupper() and len(word) > 1 and word.lower() not in ["write", "story", "about"]:
            return word

    return None

def extract_topic(prompt):
    """Extract the main topic from the prompt"""
    # Remove common instruction words
    cleaned = prompt.lower().replace("write", "").replace("story", "").replace("about", "")

    # Look for interesting keywords
    keywords = ["adventure", "flying", "jumping", "fighting", "learning", "playing", "making friends"]

    for keyword in keywords:
        if keyword in cleaned:
            return keyword

    # Return the cleaned prompt if no specific keyword found
    return cleaned.strip()

def create_structured_childrens_story(character="the character", topic="having an adventure"):
    """Create a structured children's story with simple language and short sentences"""

    # Clean up inputs
    if not character or len(character) < 2:
        character = "the little hero"

    # Create a story with the structure of the Peter Parker example
    story = f"""The story of {character} began in the park. {character} was a little child who loved to play with friends.

One sunny day, a friend came over and said, "{character}, you must come and play with me! I'm the best at {topic}!"

{character} was so excited. {character} jumped up and down with happiness. They all played together and had lots of fun.

{character} learned how to be good at {topic}. Soon, everyone wanted to play with {character}.

{character} and the friends became the best of friends. They played every day after school.

{character} was so happy to have such good friends. They would be friends forever and have many more adventures together.
"""
    return story

# Enhanced story generation function using fine-tuned model
def generate_story(model, tokenizer, prompt, max_length=400):
    # Format prompt in a way that's more suitable for children's stories
    story_prompt = f"Write a simple children's story about {prompt}. Use simple language and short sentences."

    try:
        # Encode the prompt
        input_ids = tokenizer.encode(story_prompt, return_tensors='pt')

        # Move to the same device as the model
        device = next(model.parameters()).device
        input_ids = input_ids.to(device)

        # Generate text with different parameters more suited to simple children's stories
        output = model.generate(
            input_ids,
            max_length=max_length,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            top_k=40,
            top_p=0.9,
            temperature=0.7,  # Lower temperature for more consistent, simpler text
            do_sample=True
        )

        # Decode the generated text
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

        # Check quality of response
        if len(generated_text) < 50 or "In a world where anything was possible" in generated_text:
            print("Warning: Low quality story generated. Generating a structured children's story...")

            # Create a more structured children's story instead of using the default template
            character_name = extract_character_name(prompt) or "the character"
            topic = extract_topic(prompt) or "having an adventure"

            return create_structured_childrens_story(character_name, topic)

        return generated_text
    except Exception as e:
        print(f"Error in story generation: {e}")
        return create_structured_childrens_story(extract_character_name(prompt), extract_topic(prompt))

# Function to create a fallback story when needed
def create_dynamic_fallback_story(prompt):
    # Extract character and topics
    character = extract_character_name(prompt) or "the character"
    topic = extract_topic(prompt) or "having fun"

    # Create a story similar to the Peter Parker example
    story = f"""
The story of {character} began in the park. {character} was a little child who loved to play with friends.

One sunny day, a friend came over and said, "{character}, you must come and play with me! I'm the best at {topic}!"

{character} was so excited. {character} jumped up and down with happiness. They all played together and had lots of fun.

{character} learned how to be good at {topic}. Soon, everyone wanted to play with {character}.

{character} and the friends became the best of friends. They played every day after school.

{character} was so happy to have such good friends. They would be friends forever and have many more adventures together.
"""
    return story

# Function to split the story into panels (same as in your original code)
def split_story_into_panels(story, num_panels=4):
    # First check if we have enough content to split
    if len(story) < 100:
        print("Warning: Story is very short. Panel division may not be optimal.")

    # Try to split by paragraphs first for more natural divisions
    paragraphs = story.split('\n')
    paragraphs = [p for p in paragraphs if p.strip()]  # Remove empty paragraphs

    if len(paragraphs) >= num_panels:
        # We have enough paragraphs, so use them as natural divisions
        if len(paragraphs) > num_panels:
            # Combine paragraphs to match the desired number of panels
            step = len(paragraphs) / num_panels
            panels = []
            for i in range(num_panels):
                start_idx = int(i * step)
                end_idx = int((i + 1) * step) if i < num_panels - 1 else len(paragraphs)
                panel_text = " ".join(paragraphs[start_idx:end_idx])
                panels.append(panel_text)
            return panels
        else:
            # Perfect match or fewer paragraphs than panels
            return paragraphs[:num_panels]

    # If we don't have enough paragraphs, fall back to sentence splitting
    sentences = []
    # Split by common sentence endings
    for chunk in story.split('. '):
        sentences.extend([s.strip() + '.' for s in chunk.split('! ')])
    sentences = [s for s in sentences if len(s) > 1]  # Remove empty or too-short sentences

    # Handle edge case where there are fewer sentences than panels
    if len(sentences) < num_panels:
        # Create more sentences by splitting the longest ones
        while len(sentences) < num_panels:
            longest_idx = max(range(len(sentences)), key=lambda i: len(sentences[i]))
            longest = sentences[longest_idx]

            # Split at a comma or other natural break point if possible
            if ', ' in longest:
                parts = longest.split(', ', 1)
                sentences[longest_idx] = parts[0] + '.'
                sentences.insert(longest_idx + 1, parts[1])
            else:
                # If no good split point, duplicate the sentence with a variation
                sentences.append(f"Meanwhile, {longest.lower()}")

    # Calculate how many sentences per panel for a balanced distribution
    step = max(1, len(sentences) // num_panels)

    panels = []
    for i in range(num_panels):
        start_idx = i * step
        # For the last panel, include any remaining sentences
        end_idx = (i + 1) * step if i < num_panels - 1 else len(sentences)
        panel_text = " ".join(sentences[start_idx:end_idx])
        panels.append(panel_text)

    return panels

# Load the Stable Diffusion XL model (same as in your original code)
def load_sdxl_model():
    print("Loading SDXL model - this may take a while...")
    pipe = StableDiffusionXLPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True
    )

    # Check if CUDA is available, otherwise use CPU
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device == "cpu":
        print("Warning: CUDA not available, using CPU. This will be very slow.")
    pipe = pipe.to(device)

    return pipe

# Function to generate comic panels from story prompt using our fine-tuned model
def generate_comic(story_prompt, num_panels=4, model_path=None, art_style="cartoon style"):
    # Make sure the story prompt is compelling
    if len(story_prompt) < 10:
        story_prompt = f"Create an interesting story about {story_prompt}"

    try:
        # Load the fine-tuned model
        print("Loading fine-tuned model...")
        model, tokenizer = load_fine_tuned_model(model_path)

        if model is None or tokenizer is None:
            raise Exception("Failed to load the fine-tuned model")

        # Generate story from prompt with multiple attempts if needed
        print(f"Generating story from prompt: '{story_prompt}'")
        story_text = generate_story(model, tokenizer, story_prompt)
        print("Story generated:")
        print(story_text)

        # Check if story generated properly
        if not story_text or len(story_text) < 50:
            print("Story generation produced insufficient content. Trying one more time with different approach.")
            # Try a more direct approach with the model
            direct_prompt = f"Write a simple children's story about {story_prompt}. Use short sentences."

            story_text = generate_story(model, tokenizer, direct_prompt, max_length=500)

            if len(story_text) < 50:
                print("Still unable to generate a good story. Using the structured children's story.")
                character_name = extract_character_name(story_prompt) or "the character"
                topic = extract_topic(story_prompt) or "having an adventure"
                story_text = create_structured_childrens_story(character_name, topic)
    except Exception as e:
        print(f"Error generating story: {e}")
        # Provide a fallback story using our improved fallback generator
        character_name = extract_character_name(story_prompt) or "the character"
        topic = extract_topic(story_prompt) or "having an adventure"
        story_text = create_structured_childrens_story(character_name, topic)

    # Split story into panels
    print("\nSplitting story into panels...")
    story_panels = split_story_into_panels(story_text, num_panels)
    for i, panel in enumerate(story_panels):
        print(f"Panel {i+1}: {panel}")

    try:
        # Load SDXL model
        pipe = load_sdxl_model()

        print("\nGenerating panel images...")
        panel_images = []
        for i, panel_text in enumerate(story_panels):
            print(f"Generating image for panel {i+1}...")
            # Create a prompt that helps SDXL understand it's a comic panel for children
            # Use the art style specified by the user, with emphasis on children's book style
            prompt = f"A friendly {art_style} illustration for a children's storybook showing: {panel_text} bright colors, simple shapes, cute characters, child-friendly"

            # Generate the image
            image = pipe(
                prompt=prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                num_inference_steps=30
            ).images[0]
            panel_images.append(image)

            # Save the image
            image_path = f"panel_{i+1}.png"
            image.save(image_path)

        print("Comic generation complete! All panels saved as PNG files.")
        return story_text, story_panels, panel_images

    except Exception as e:
        print(f"Error in image generation: {e}")
        print("Unable to generate images. Please check if you have enough GPU memory and proper installations.")
        return story_text, story_panels, None

# Web UI using Gradio
def create_ui():
    with gr.Blocks() as app:
        gr.Markdown("# Story to Comic Generator")
        gr.Markdown("Enter a prompt to generate a story using your fine-tuned model and convert it into comic panels")

        with gr.Row():
            with gr.Column():
                story_prompt = gr.Textbox(
                    label="Story Prompt",
                    placeholder="Write a story about a brave cat who learns to fly",
                    lines=2
                )
                num_panels = gr.Slider(minimum=2, maximum=8, value=4, step=1, label="Number of Panels")
                model_path = gr.Textbox(
                    label="Custom Model Path",
                    placeholder="/content/drive/MyDrive/fine_tuned_tiny_stories_model",
                    value="/content/drive/MyDrive/fine_tuned_tiny_stories_model"
                )
                art_style = gr.Dropdown(
                    choices=[
                        "cartoon style",
                        "manga style",
                        "comic book style",
                        "watercolor style",
                        "3D rendered",
                        "pixel art style",
                        "oil painting style",
                        "storybook illustration style"
                    ],
                    value="cartoon style",
                    label="Art Style"
                )
                generate_button = gr.Button("Generate Comic")

            with gr.Column():
                story_output = gr.Textbox(label="Generated Story", lines=8)

        # Gallery component for displaying panels
        panel_gallery = gr.Gallery(label="Comic Panels", columns=2, rows=2, height=600)

        # Status output for error messages
        status_output = gr.Textbox(label="Status", lines=2)

        def run_generation(prompt, panels, path, style):
            if not prompt:
                return "Please enter a story prompt", [], "Error: No prompt provided"

            model_path_to_use = path if path.strip() else None

            try:
                story_text, story_panels, images = generate_comic(prompt, int(panels), model_path_to_use, style)

                if story_text and images:
                    status = "Success: Comic generated successfully!"
                elif story_text:
                    status = "Warning: Story generated but image creation failed"
                else:
                    status = "Error: Story generation failed"

                return story_text, images if images else [], status
            except Exception as e:
                return f"An error occurred: {str(e)}", [], f"Error: {str(e)}"

        generate_button.click(
            run_generation,
            inputs=[story_prompt, num_panels, model_path, art_style],
            outputs=[story_output, panel_gallery, status_output]
        )

    return app

# Main function to either run the generator directly or launch the web UI
def main(use_web_ui=True):
    if use_web_ui:
        app = create_ui()
        app.launch(share=True)  # Enable sharing for remote access
    else:
        # Example without web UI
        story_prompt = input("Enter a story prompt: ")
        num_panels = int(input("Enter number of panels (2-8): "))
        model_path = input("Enter model path (leave empty for default): ")
        art_style = input("Enter art style (e.g., cartoon style, manga style): ")

        story, panels, images = generate_comic(story_prompt, num_panels, model_path or None, art_style)

        if images:
            for i, img in enumerate(images):
                plt.figure(figsize=(8, 8))
                plt.imshow(img)
                plt.title(f"Panel {i+1}")
                plt.axis('off')
                plt.show()

if _name_ == "_main_":
    main(use_web_ui=True)  # Set to False to use command line instead of web UI

NameError: name '_name_' is not defined

In [ ]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import cv2
from transformers import AutoTokenizer, AutoModelForCausalLM
from diffusers import StableDiffusionXLPipeline
from PIL import Image, ImageDraw, ImageFont
import gradio as gr
from google.colab import drive


# Load the fine-tuned model
def load_fine_tuned_model(model_path=None):
    # If no model path provided, use default path
    if not model_path:
        model_path = '/content/drive/MyDrive/fine_tuned_tiny_stories_model'

    try:
        print(f"Loading fine-tuned model from {model_path}...")
        model = AutoModelForCausalLM.from_pretrained(model_path)
        tokenizer = AutoTokenizer.from_pretrained(model_path)

        # Move model to GPU if available
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)

        print(f"Model loaded successfully and moved to {device}")
        return model, tokenizer
    except Exception as e:
        print(f"Error loading model: {e}")
        return None, None

# Helper functions for story generation
def extract_character_name(prompt):
    """Extract potential character names from the prompt"""
    # Common superhero/character names to check for
    character_names = ["Spider-Man", "Peter Parker", "Batman", "Superman", "Wonder Woman",
                       "Iron Man", "Captain America", "Thor", "Hulk", "Black Widow"]

    # Check if any known character names are in the prompt
    for name in character_names:
        if name.lower() in prompt.lower():
            return name

    # Otherwise extract words that might be names (capitalized words)
    words = prompt.split()
    for word in words:
        if word[0].isupper() and len(word) > 1 and word.lower() not in ["write", "story", "about"]:
            return word

    return None

def extract_topic(prompt):
    """Extract the main topic from the prompt"""
    # Remove common instruction words
    cleaned = prompt.lower().replace("write", "").replace("story", "").replace("about", "")

    # Look for interesting keywords
    keywords = ["adventure", "flying", "jumping", "fighting", "learning", "playing", "making friends"]

    for keyword in keywords:
        if keyword in cleaned:
            return keyword

    # Return the cleaned prompt if no specific keyword found
    return cleaned.strip()

def create_structured_childrens_story(character="the character", topic="having an adventure"):
    """Create a structured children's story with simple language and short sentences"""

    # Clean up inputs
    if not character or len(character) < 2:
        character = "the little hero"

    # Create a story with the structure of the Peter Parker example
    story = f"""The story of {character} began in the park. {character} was a little child who loved to play with friends.

One sunny day, a friend came over and said, "{character}, you must come and play with me! I'm the best at {topic}!"

{character} was so excited. {character} jumped up and down with happiness. They all played together and had lots of fun.

{character} learned how to be good at {topic}. Soon, everyone wanted to play with {character}.

{character} and the friends became the best of friends. They played every day after school.

{character} was so happy to have such good friends. They would be friends forever and have many more adventures together.
"""
    return story

# Enhanced story generation function using fine-tuned model
def generate_story(model, tokenizer, prompt, max_length=400):
    # Format prompt in a way that's more suitable for children's stories
    story_prompt = f"Write a simple children's story about {prompt}. Use simple language and short sentences."

    try:
        # Encode the prompt
        input_ids = tokenizer.encode(story_prompt, return_tensors='pt')

        # Move to the same device as the model
        device = next(model.parameters()).device
        input_ids = input_ids.to(device)

        # Generate text with different parameters more suited to simple children's stories
        output = model.generate(
            input_ids,
            max_length=max_length,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            top_k=40,
            top_p=0.9,
            temperature=0.7,  # Lower temperature for more consistent, simpler text
            do_sample=True
        )

        # Decode the generated text
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

        # Check quality of response
        if len(generated_text) < 50 or "In a world where anything was possible" in generated_text:
            print("Warning: Low quality story generated. Generating a structured children's story...")

            # Create a more structured children's story instead of using the default template
            character_name = extract_character_name(prompt) or "the character"
            topic = extract_topic(prompt) or "having an adventure"

            return create_structured_childrens_story(character_name, topic)

        return generated_text
    except Exception as e:
        print(f"Error in story generation: {e}")
        return create_structured_childrens_story(extract_character_name(prompt), extract_topic(prompt))

# Function to create a fallback story when needed
def create_dynamic_fallback_story(prompt):
    # Extract character and topics
    character = extract_character_name(prompt) or "the character"
    topic = extract_topic(prompt) or "having fun"

    # Create a story similar to the Peter Parker example
    story = f"""
The story of {character} began in the park. {character} was a little child who loved to play with friends.

One sunny day, a friend came over and said, "{character}, you must come and play with me! I'm the best at {topic}!"

{character} was so excited. {character} jumped up and down with happiness. They all played together and had lots of fun.

{character} learned how to be good at {topic}. Soon, everyone wanted to play with {character}.

{character} and the friends became the best of friends. They played every day after school.

{character} was so happy to have such good friends. They would be friends forever and have many more adventures together.
"""
    return story

# Function to split the story into panels and also extract dialogues for speech bubbles
def split_story_into_panels(story, num_panels=4):
    # First check if we have enough content to split
    if len(story) < 100:
        print("Warning: Story is very short. Panel division may not be optimal.")

    # Try to split by paragraphs first for more natural divisions
    paragraphs = story.split('\n')
    paragraphs = [p for p in paragraphs if p.strip()]  # Remove empty paragraphs

    # Extract dialogues for speech bubbles
    dialogues = []
    panel_texts = []

    if len(paragraphs) >= num_panels:
        # We have enough paragraphs, so use them as natural divisions
        if len(paragraphs) > num_panels:
            # Combine paragraphs to match the desired number of panels
            step = len(paragraphs) / num_panels
            panels = []
            for i in range(num_panels):
                start_idx = int(i * step)
                end_idx = int((i + 1) * step) if i < num_panels - 1 else len(paragraphs)
                panel_text = " ".join(paragraphs[start_idx:end_idx])
                panels.append(panel_text)

                # Extract dialogue (text within quotes) for speech bubbles
                dialogue = extract_dialogue(panel_text)
                dialogues.append(dialogue)

                # Remove dialogue from panel text for better image prompts
                clean_text = panel_text.replace(dialogue, "") if dialogue else panel_text
                panel_texts.append(clean_text)
            return panels, panel_texts, dialogues
        else:
            # Perfect match or fewer paragraphs than panels
            panels = paragraphs[:num_panels]
            for panel in panels:
                dialogue = extract_dialogue(panel)
                dialogues.append(dialogue)
                clean_text = panel.replace(dialogue, "") if dialogue else panel
                panel_texts.append(clean_text)
            return panels, panel_texts, dialogues

    # If we don't have enough paragraphs, fall back to sentence splitting
    sentences = []
    # Split by common sentence endings
    for chunk in story.split('. '):
        sentences.extend([s.strip() + '.' for s in chunk.split('! ')])
    sentences = [s for s in sentences if len(s) > 1]  # Remove empty or too-short sentences

    # Handle edge case where there are fewer sentences than panels
    if len(sentences) < num_panels:
        # Create more sentences by splitting the longest ones
        while len(sentences) < num_panels:
            longest_idx = max(range(len(sentences)), key=lambda i: len(sentences[i]))
            longest = sentences[longest_idx]

            # Split at a comma or other natural break point if possible
            if ', ' in longest:
                parts = longest.split(', ', 1)
                sentences[longest_idx] = parts[0] + '.'
                sentences.insert(longest_idx + 1, parts[1])
            else:
                # If no good split point, duplicate the sentence with a variation
                sentences.append(f"Meanwhile, {longest.lower()}")

    # Calculate how many sentences per panel for a balanced distribution
    step = max(1, len(sentences) // num_panels)

    panels = []
    for i in range(num_panels):
        start_idx = i * step
        # For the last panel, include any remaining sentences
        end_idx = (i + 1) * step if i < num_panels - 1 else len(sentences)
        panel_text = " ".join(sentences[start_idx:end_idx])
        panels.append(panel_text)

        # Extract dialogue for speech bubbles
        dialogue = extract_dialogue(panel_text)
        dialogues.append(dialogue)

        # Remove dialogue from panel text for better image prompts
        clean_text = panel_text.replace(dialogue, "") if dialogue else panel_text
        panel_texts.append(clean_text)

    return panels, panel_texts, dialogues

# Function to extract dialogue from text (text within quotes)
def extract_dialogue(text):
    import re
    dialogue_matches = re.findall(r'"([^"]*)"', text)
    if dialogue_matches:
        # Return the first dialogue found
        return dialogue_matches[0]
    return ""

# Load the Stable Diffusion XL model
def load_sdxl_model():
    print("Loading SDXL model - this may take a while...")
    pipe = StableDiffusionXLPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True
    )

    # Check if CUDA is available, otherwise use CPU
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device == "cpu":
        print("Warning: CUDA not available, using CPU. This will be very slow.")
    pipe = pipe.to(device)

    return pipe

# Function to add speech bubbles to images
def add_speech_bubble(image, text, position=None):
    if not text:
        return image

    # Convert PIL image to numpy array for OpenCV operations
    img = np.array(image.convert("RGBA"))
    height, width, _ = img.shape

    # If position not provided, place the bubble at a suitable position
    if position is None:
        position = (width // 2, height // 4)

    # Create a transparent layer for the bubble
    bubble = np.zeros((height, width, 4), dtype=np.uint8)

    # Calculate bubble size based on text length
    bubble_width = min(300, max(150, len(text) * 10))
    bubble_height = min(130, max(60, (len(text) // 20 + 1) * 30))

    # Draw speech bubble
    cv2.ellipse(bubble, position, (bubble_width, bubble_height), 0, 0, 360, (255, 255, 255, 255), -1)

    # Add a pointer to make it look like a speech bubble
    pointer_x = position[0]
    pointer_y = position[1] + bubble_height
    triangle_pts = np.array([
        [pointer_x - 20, pointer_y - 10],
        [pointer_x + 20, pointer_y - 10],
        [pointer_x, pointer_y + 30]
    ])
    cv2.fillPoly(bubble, [triangle_pts], (255, 255, 255, 255))

    bubble_pil = Image.fromarray(bubble)

    # Draw text
    draw = ImageDraw.Draw(bubble_pil)

    # Try to load a font, fall back to default if not available
    try:
        font = ImageFont.truetype("arial.ttf", 24)
    except:
        try:
            font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 24)
        except:
            font = ImageFont.load_default()

    # Wrap text to fit in the bubble
    wrapped_text = wrap_text(text, font, bubble_width - 40)

    # Calculate text position for centering
    text_x = position[0] - (bubble_width // 2) + 20
    text_y = position[1] - (bubble_height // 2) + 10

    # Draw the wrapped text
    for i, line in enumerate(wrapped_text):
        draw.text((text_x, text_y + i * 30), line, fill="black", font=font)

    # Combine the original image with the bubble
    return Image.alpha_composite(image.convert("RGBA"), bubble_pil)

# Function to wrap text to fit within a certain width
def wrap_text(text, font, max_width):
    words = text.split()
    wrapped_lines = []
    current_line = []

    for word in words:
        # Test if adding this word would exceed the width
        test_line = ' '.join(current_line + [word])
        try:
            line_width = font.getsize(test_line)[0]
        except:
            # If getsize not available, estimate based on character count
            line_width = len(test_line) * 10

        if line_width <= max_width:
            current_line.append(word)
        else:
            wrapped_lines.append(' '.join(current_line))
            current_line = [word]

    # Add the last line
    if current_line:
        wrapped_lines.append(' '.join(current_line))

    return wrapped_lines

# Function to generate comic panels from story prompt using our fine-tuned model
def generate_comic(story_prompt, num_panels=4, model_path=None, art_style="cartoon style"):
    # Make sure the story prompt is compelling
    if len(story_prompt) < 10:
        story_prompt = f"Create an interesting story about {story_prompt}"

    try:
        # Load the fine-tuned model
        print("Loading fine-tuned model...")
        model, tokenizer = load_fine_tuned_model(model_path)

        if model is None or tokenizer is None:
            raise Exception("Failed to load the fine-tuned model")

        # Generate story from prompt with multiple attempts if needed
        print(f"Generating story from prompt: '{story_prompt}'")
        story_text = generate_story(model, tokenizer, story_prompt)
        print("Story generated:")
        print(story_text)

        # Check if story generated properly
        if not story_text or len(story_text) < 50:
            print("Story generation produced insufficient content. Trying one more time with different approach.")
            # Try a more direct approach with the model
            direct_prompt = f"Write a simple children's story about {story_prompt}. Use short sentences."

            story_text = generate_story(model, tokenizer, direct_prompt, max_length=500)

            if len(story_text) < 50:
                print("Still unable to generate a good story. Using the structured children's story.")
                character_name = extract_character_name(story_prompt) or "the character"
                topic = extract_topic(story_prompt) or "having an adventure"
                story_text = create_structured_childrens_story(character_name, topic)
    except Exception as e:
        print(f"Error generating story: {e}")
        # Provide a fallback story using our improved fallback generator
        character_name = extract_character_name(story_prompt) or "the character"
        topic = extract_topic(story_prompt) or "having an adventure"
        story_text = create_structured_childrens_story(character_name, topic)

    # Split story into panels and extract dialogues
    print("\nSplitting story into panels...")
    story_panels, panel_texts, dialogues = split_story_into_panels(story_text, num_panels)
    for i, panel in enumerate(story_panels):
        print(f"Panel {i+1}: {panel}")
        if dialogues[i]:
            print(f"Dialogue for panel {i+1}: \"{dialogues[i]}\"")

    try:
        # Load SDXL model
        pipe = load_sdxl_model()

        print("\nGenerating panel images...")
        panel_images = []

        # Extract character name for consistency
        character_name = extract_character_name(story_prompt) or "the character"

        # Generate a base character image for consistency across panels
        print("Generating base character image for consistency...")
        base_prompt = f"A friendly {art_style} illustration of {character_name}, whole body visible, simple background, bright colors, child-friendly"
        base_image = pipe(
            prompt=base_prompt,
            negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
            num_inference_steps=30
        ).images[0]

        # Resize base image for consistency
        base_image = base_image.resize((768, 768))

        # Generate each panel
        for i, panel_text in enumerate(panel_texts):
            print(f"Generating image for panel {i+1}...")
            # Create a prompt that helps SDXL understand it's a comic panel for children
            prompt = f"A friendly {art_style} illustration for a children's storybook showing: {panel_text}, featuring {character_name}, bright colors, simple shapes, cute characters, child-friendly"

            # Generate the image using the base character for consistency
            image = pipe(
                prompt=prompt,
                image=base_image,  # Use base image for character consistency
                strength=0.7,      # Keep some elements of base image but allow for new scene
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                num_inference_steps=30
            ).images[0]

            # Resize to standard panel size
            image = image.resize((512, 512))

            # Add speech bubble if there's dialogue
            if dialogues[i]:
                # Calculate a position for the speech bubble based on panel number
                position = (256, 100 + (i % 2) * 50)  # Vary position slightly between panels
                image = add_speech_bubble(image, dialogues[i], position)

            panel_images.append(image)

            # Save the individual panel
            image_path = f"panel_{i+1}.png"
            image.save(image_path)

        # Create a combined comic strip
        print("Creating combined comic strip...")
        rows = int(np.ceil(num_panels / 2))  # Calculate how many rows we need
        columns = min(2, num_panels)         # Max 2 columns

        final_width = 512 * columns
        final_height = 512 * rows

        comic_strip = Image.new("RGB", (final_width, final_height), (255, 255, 255))

        for i, image in enumerate(panel_images):
            x_offset = (i % 2) * 512
            y_offset = (i // 2) * 512
            comic_strip.paste(image, (x_offset, y_offset))

        # Save the combined comic strip
        comic_strip.save("comic_strip.png")
        print("Comic generation complete! All panels saved as PNG files and combined into comic_strip.png")

        return story_text, story_panels, panel_images, comic_strip

    except Exception as e:
        print(f"Error in image generation: {e}")
        print("Unable to generate images. Please check if you have enough GPU memory and proper installations.")
        return story_text, story_panels, None, None

# Web UI using Gradio
def create_ui():
    with gr.Blocks() as app:
        gr.Markdown("# Story to Comic Generator")
        gr.Markdown("Enter a prompt to generate a story and convert it into comic panels with speech bubbles")

        with gr.Row():
            with gr.Column():
                story_prompt = gr.Textbox(
                    label="Story Prompt",
                    placeholder="Write a story about a brave cat who learns to fly",
                    lines=2
                )
                num_panels = gr.Slider(minimum=2, maximum=8, value=4, step=1, label="Number of Panels")
                model_path = gr.Textbox(
                    label="Custom Model Path",
                    placeholder="/content/drive/MyDrive/fine_tuned_tiny_stories_model",
                    value="/content/drive/MyDrive/fine_tuned_tiny_stories_model"
                )
                art_style = gr.Dropdown(
                    choices=[
                        "cartoon style",
                        "manga style",
                        "comic book style",
                        "watercolor style",
                        "3D rendered",
                        "pixel art style",
                        "oil painting style",
                        "storybook illustration style"
                    ],
                    value="cartoon style",
                    label="Art Style"
                )
                generate_button = gr.Button("Generate Comic")

            with gr.Column():
                story_output = gr.Textbox(label="Generated Story", lines=8)

        # Gallery component for displaying individual panels
        panel_gallery = gr.Gallery(label="Comic Panels", columns=2, rows=2, height=500)

        # Image component for displaying the combined comic strip
        comic_strip_image = gr.Image(label="Combined Comic Strip", height=600)

        # Status output for error messages
        status_output = gr.Textbox(label="Status", lines=2)

        def run_generation(prompt, panels, path, style):
            if not prompt:
                return "Please enter a story prompt", [], None, "Error: No prompt provided"

            model_path_to_use = path if path.strip() else None

            try:
                story_text, story_panels, images, comic_strip = generate_comic(prompt, int(panels), model_path_to_use, style)

                if story_text and images and comic_strip:
                    status = "Success: Comic generated successfully with speech bubbles!"
                elif story_text and images:
                    status = "Success: Individual panels generated but combined strip failed"
                elif story_text:
                    status = "Warning: Story generated but image creation failed"
                else:
                    status = "Error: Story generation failed"

                return story_text, images if images else [], comic_strip, status
            except Exception as e:
                return f"An error occurred: {str(e)}", [], None, f"Error: {str(e)}"

        generate_button.click(
            run_generation,
            inputs=[story_prompt, num_panels, model_path, art_style],
            outputs=[story_output, panel_gallery, comic_strip_image, status_output]
        )

    return app

# Main function to either run the generator directly or launch the web UI
def main(use_web_ui=True):
    if use_web_ui:
        app = create_ui()
        app.launch(share=True)  # Enable sharing for remote access
    else:
        # Example without web UI
        story_prompt = input("Enter a story prompt: ")
        num_panels = int(input("Enter number of panels (2-8): "))
        model_path = input("Enter model path (leave empty for default): ")
        art_style = input("Enter art style (e.g., cartoon style, manga style): ")

        story, panels, images, comic_strip = generate_comic(story_prompt, num_panels, model_path or None, art_style)

        if images:
            for i, img in enumerate(images):
                plt.figure(figsize=(8, 8))
                plt.imshow(img)
                plt.title(f"Panel {i+1}")
                plt.axis('off')
                plt.show()

            # Display combined comic strip
            if comic_strip:
                plt.figure(figsize=(12, 12))
                plt.imshow(comic_strip)
                plt.title("Complete Comic Strip")
                plt.axis('off')
                plt.show()

if __name__ == "__main__":
    main(use_web_ui=True)  # Set to False to use command line instead of web UI

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f55e72022b729581bb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import cv2
from transformers import AutoTokenizer, AutoModelForCausalLM
from diffusers import StableDiffusionPipeline, StableDiffusionXLPipeline
from PIL import Image, ImageDraw, ImageFont
import gradio as gr
from google.colab import drive


# Load the fine-tuned model
def load_fine_tuned_model(model_path=None):
    # If no model path provided, use default path
    if not model_path:
        model_path = '/content/drive/MyDrive/fine_tuned_tiny_stories_model'

    try:
        print(f"Loading fine-tuned model from {model_path}...")
        model = AutoModelForCausalLM.from_pretrained(model_path)
        tokenizer = AutoTokenizer.from_pretrained(model_path)

        # Move model to GPU if available
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)

        print(f"Model loaded successfully and moved to {device}")
        return model, tokenizer
    except Exception as e:
        print(f"Error loading model: {e}")
        return None, None

# Helper functions for story generation
def extract_character_name(prompt):
    """Extract potential character names from the prompt"""
    # Common superhero/character names to check for
    character_names = ["Spider-Man", "Peter Parker", "Batman", "Superman", "Wonder Woman",
                       "Iron Man", "Captain America", "Thor", "Hulk", "Black Widow"]

    # Check if any known character names are in the prompt
    for name in character_names:
        if name.lower() in prompt.lower():
            return name

    # Otherwise extract words that might be names (capitalized words)
    words = prompt.split()
    for word in words:
        if word[0].isupper() and len(word) > 1 and word.lower() not in ["write", "story", "about"]:
            return word

    return None

def extract_topic(prompt):
    """Extract the main topic from the prompt"""
    # Remove common instruction words
    cleaned = prompt.lower().replace("write", "").replace("story", "").replace("about", "")

    # Look for interesting keywords
    keywords = ["adventure", "flying", "jumping", "fighting", "learning", "playing", "making friends"]

    for keyword in keywords:
        if keyword in cleaned:
            return keyword

    # Return the cleaned prompt if no specific keyword found
    return cleaned.strip()

def create_structured_childrens_story(character="the character", topic="having an adventure"):
    """Create a structured children's story with simple language and short sentences"""

    # Clean up inputs
    if not character or len(character) < 2:
        character = "the little hero"

    # Create a story with the structure of the Peter Parker example
    story = f"""The story of {character} began in the park. {character} was a little child who loved to play with friends.

One sunny day, a friend came over and said, "{character}, you must come and play with me! I'm the best at {topic}!"

{character} was so excited. {character} jumped up and down with happiness. They all played together and had lots of fun.

{character} learned how to be good at {topic}. Soon, everyone wanted to play with {character}.

{character} and the friends became the best of friends. They played every day after school.

{character} was so happy to have such good friends. They would be friends forever and have many more adventures together.
"""
    return story

# Enhanced story generation function using fine-tuned model
def generate_story(model, tokenizer, prompt, max_length=400):
    # Format prompt in a way that's more suitable for children's stories
    story_prompt = f"Write a simple children's story about {prompt}. Use simple language and short sentences."

    try:
        # Encode the prompt
        input_ids = tokenizer.encode(story_prompt, return_tensors='pt')

        # Move to the same device as the model
        device = next(model.parameters()).device
        input_ids = input_ids.to(device)

        # Generate text with different parameters more suited to simple children's stories
        output = model.generate(
            input_ids,
            max_length=max_length,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            top_k=40,
            top_p=0.9,
            temperature=0.7,  # Lower temperature for more consistent, simpler text
            do_sample=True
        )

        # Decode the generated text
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

        # Check quality of response
        if len(generated_text) < 50 or "In a world where anything was possible" in generated_text:
            print("Warning: Low quality story generated. Generating a structured children's story...")

            # Create a more structured children's story instead of using the default template
            character_name = extract_character_name(prompt) or "the character"
            topic = extract_topic(prompt) or "having an adventure"

            return create_structured_childrens_story(character_name, topic)

        return generated_text
    except Exception as e:
        print(f"Error in story generation: {e}")
        return create_structured_childrens_story(extract_character_name(prompt), extract_topic(prompt))

# Function to create a fallback story when needed
def create_dynamic_fallback_story(prompt):
    # Extract character and topics
    character = extract_character_name(prompt) or "the character"
    topic = extract_topic(prompt) or "having fun"

    # Create a story similar to the Peter Parker example
    story = f"""
The story of {character} began in the park. {character} was a little child who loved to play with friends.

One sunny day, a friend came over and said, "{character}, you must come and play with me! I'm the best at {topic}!"

{character} was so excited. {character} jumped up and down with happiness. They all played together and had lots of fun.

{character} learned how to be good at {topic}. Soon, everyone wanted to play with {character}.

{character} and the friends became the best of friends. They played every day after school.

{character} was so happy to have such good friends. They would be friends forever and have many more adventures together.
"""
    return story

# Function to split the story into panels and also extract dialogues for speech bubbles
def split_story_into_panels(story, num_panels=4):
    # First check if we have enough content to split
    if len(story) < 100:
        print("Warning: Story is very short. Panel division may not be optimal.")

    # Try to split by paragraphs first for more natural divisions
    paragraphs = story.split('\n')
    paragraphs = [p for p in paragraphs if p.strip()]  # Remove empty paragraphs

    # Extract dialogues for speech bubbles
    dialogues = []
    panel_texts = []

    if len(paragraphs) >= num_panels:
        # We have enough paragraphs, so use them as natural divisions
        if len(paragraphs) > num_panels:
            # Combine paragraphs to match the desired number of panels
            step = len(paragraphs) / num_panels
            panels = []
            for i in range(num_panels):
                start_idx = int(i * step)
                end_idx = int((i + 1) * step) if i < num_panels - 1 else len(paragraphs)
                panel_text = " ".join(paragraphs[start_idx:end_idx])
                panels.append(panel_text)

                # Extract dialogue (text within quotes) for speech bubbles
                dialogue = extract_dialogue(panel_text)
                dialogues.append(dialogue)

                # Remove dialogue from panel text for better image prompts
                clean_text = panel_text.replace(dialogue, "") if dialogue else panel_text
                panel_texts.append(clean_text)
            return panels, panel_texts, dialogues
        else:
            # Perfect match or fewer paragraphs than panels
            panels = paragraphs[:num_panels]
            for panel in panels:
                dialogue = extract_dialogue(panel)
                dialogues.append(dialogue)
                clean_text = panel.replace(dialogue, "") if dialogue else panel
                panel_texts.append(clean_text)
            return panels, panel_texts, dialogues

    # If we don't have enough paragraphs, fall back to sentence splitting
    sentences = []
    # Split by common sentence endings
    for chunk in story.split('. '):
        sentences.extend([s.strip() + '.' for s in chunk.split('! ')])
    sentences = [s for s in sentences if len(s) > 1]  # Remove empty or too-short sentences

    # Handle edge case where there are fewer sentences than panels
    if len(sentences) < num_panels:
        # Create more sentences by splitting the longest ones
        while len(sentences) < num_panels:
            longest_idx = max(range(len(sentences)), key=lambda i: len(sentences[i]))
            longest = sentences[longest_idx]

            # Split at a comma or other natural break point if possible
            if ', ' in longest:
                parts = longest.split(', ', 1)
                sentences[longest_idx] = parts[0] + '.'
                sentences.insert(longest_idx + 1, parts[1])
            else:
                # If no good split point, duplicate the sentence with a variation
                sentences.append(f"Meanwhile, {longest.lower()}")

    # Calculate how many sentences per panel for a balanced distribution
    step = max(1, len(sentences) // num_panels)

    panels = []
    for i in range(num_panels):
        start_idx = i * step
        # For the last panel, include any remaining sentences
        end_idx = (i + 1) * step if i < num_panels - 1 else len(sentences)
        panel_text = " ".join(sentences[start_idx:end_idx])
        panels.append(panel_text)

        # Extract dialogue for speech bubbles
        dialogue = extract_dialogue(panel_text)
        dialogues.append(dialogue)

        # Remove dialogue from panel text for better image prompts
        clean_text = panel_text.replace(dialogue, "") if dialogue else panel_text
        panel_texts.append(clean_text)

    return panels, panel_texts, dialogues

# Function to extract dialogue from text (text within quotes)
def extract_dialogue(text):
    import re
    dialogue_matches = re.findall(r'"([^"]*)"', text)
    if dialogue_matches:
        # Return the first dialogue found
        return dialogue_matches[0]
    return ""

# Load the custom InkoloRA model
def load_custom_model():
    print("Loading InkoloRA model - this may take a while...")

    # Define custom model path
    custom_model_path = "/content/drive/MyDrive/InkoloRA.safetensors"

    # Check if the model file exists
    if not os.path.exists(custom_model_path):
        print(f"Warning: Custom model not found at {custom_model_path}. Falling back to Stable Diffusion XL.")
        return load_sdxl_model()

    try:
        # Load the model - adjust the model_id based on what base model your safetensors file is trained on
        # Assuming it's SD 1.5 based; modify if it's based on a different architecture
        pipe = StableDiffusionPipeline.from_pretrained(
            "runwayml/stable-diffusion-v1-5",
            torch_dtype=torch.float16,
            safety_checker=None
        )

        # Load the custom weights
        pipe.unet.load_attn_procs(custom_model_path)

        # Check if CUDA is available, otherwise use CPU
        device = "cuda" if torch.cuda.is_available() else "cpu"
        if device == "cpu":
            print("Warning: CUDA not available, using CPU. This will be very slow.")
        pipe = pipe.to(device)

        print("InkoloRA model loaded successfully!")
        return pipe
    except Exception as e:
        print(f"Error loading custom model: {e}")
        print("Falling back to Stable Diffusion XL...")
        return load_sdxl_model()

# Original SDXL model load function as fallback
def load_sdxl_model():
    print("Loading SDXL model - this may take a while...")
    pipe = StableDiffusionXLPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True
    )

    # Check if CUDA is available, otherwise use CPU
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device == "cpu":
        print("Warning: CUDA not available, using CPU. This will be very slow.")
    pipe = pipe.to(device)

    return pipe

# Function to add speech bubbles to images
def add_speech_bubble(image, text, position=None):
    if not text:
        return image

    # Convert PIL image to numpy array for OpenCV operations
    img = np.array(image.convert("RGBA"))
    height, width, _ = img.shape

    # If position not provided, place the bubble at a suitable position
    if position is None:
        position = (width // 2, height // 4)

    # Create a transparent layer for the bubble
    bubble = np.zeros((height, width, 4), dtype=np.uint8)

    # Calculate bubble size based on text length
    bubble_width = min(300, max(150, len(text) * 10))
    bubble_height = min(130, max(60, (len(text) // 20 + 1) * 30))

    # Draw speech bubble
    cv2.ellipse(bubble, position, (bubble_width, bubble_height), 0, 0, 360, (255, 255, 255, 255), -1)

    # Add a pointer to make it look like a speech bubble
    pointer_x = position[0]
    pointer_y = position[1] + bubble_height
    triangle_pts = np.array([
        [pointer_x - 20, pointer_y - 10],
        [pointer_x + 20, pointer_y - 10],
        [pointer_x, pointer_y + 30]
    ])
    cv2.fillPoly(bubble, [triangle_pts], (255, 255, 255, 255))

    bubble_pil = Image.fromarray(bubble)

    # Draw text
    draw = ImageDraw.Draw(bubble_pil)

    # Try to load a font, fall back to default if not available
    try:
        font = ImageFont.truetype("arial.ttf", 24)
    except:
        try:
            font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 24)
        except:
            font = ImageFont.load_default()

    # Wrap text to fit in the bubble
    wrapped_text = wrap_text(text, font, bubble_width - 40)

    # Calculate text position for centering
    text_x = position[0] - (bubble_width // 2) + 20
    text_y = position[1] - (bubble_height // 2) + 10

    # Draw the wrapped text
    for i, line in enumerate(wrapped_text):
        draw.text((text_x, text_y + i * 30), line, fill="black", font=font)

    # Combine the original image with the bubble
    return Image.alpha_composite(image.convert("RGBA"), bubble_pil)

# Function to wrap text to fit within a certain width
def wrap_text(text, font, max_width):
    words = text.split()
    wrapped_lines = []
    current_line = []

    for word in words:
        # Test if adding this word would exceed the width
        test_line = ' '.join(current_line + [word])
        try:
            line_width = font.getsize(test_line)[0]
        except:
            # If getsize not available, estimate based on character count
            line_width = len(test_line) * 10

        if line_width <= max_width:
            current_line.append(word)
        else:
            wrapped_lines.append(' '.join(current_line))
            current_line = [word]

    # Add the last line
    if current_line:
        wrapped_lines.append(' '.join(current_line))

    return wrapped_lines

# Function to generate comic panels from story prompt using our fine-tuned model
def generate_comic(story_prompt, num_panels=4, model_path=None, art_style="cartoon style"):
    # Make sure the story prompt is compelling
    if len(story_prompt) < 10:
        story_prompt = f"Create an interesting story about {story_prompt}"

    try:
        # Load the fine-tuned model
        print("Loading fine-tuned model...")
        model, tokenizer = load_fine_tuned_model(model_path)

        if model is None or tokenizer is None:
            raise Exception("Failed to load the fine-tuned model")

        # Generate story from prompt with multiple attempts if needed
        print(f"Generating story from prompt: '{story_prompt}'")
        story_text = generate_story(model, tokenizer, story_prompt)
        print("Story generated:")
        print(story_text)

        # Check if story generated properly
        if not story_text or len(story_text) < 50:
            print("Story generation produced insufficient content. Trying one more time with different approach.")
            # Try a more direct approach with the model
            direct_prompt = f"Write a simple children's story about {story_prompt}. Use short sentences."

            story_text = generate_story(model, tokenizer, direct_prompt, max_length=500)

            if len(story_text) < 50:
                print("Still unable to generate a good story. Using the structured children's story.")
                character_name = extract_character_name(story_prompt) or "the character"
                topic = extract_topic(story_prompt) or "having an adventure"
                story_text = create_structured_childrens_story(character_name, topic)
    except Exception as e:
        print(f"Error generating story: {e}")
        # Provide a fallback story using our improved fallback generator
        character_name = extract_character_name(story_prompt) or "the character"
        topic = extract_topic(story_prompt) or "having an adventure"
        story_text = create_structured_childrens_story(character_name, topic)

    # Split story into panels and extract dialogues
    print("\nSplitting story into panels...")
    story_panels, panel_texts, dialogues = split_story_into_panels(story_text, num_panels)
    for i, panel in enumerate(story_panels):
        print(f"Panel {i+1}: {panel}")
        if dialogues[i]:
            print(f"Dialogue for panel {i+1}: \"{dialogues[i]}\"")

    try:
        # Load our custom InkoloRA model instead of SDXL
        pipe = load_custom_model()

        print("\nGenerating panel images...")
        panel_images = []

        # Extract character name for consistency
        character_name = extract_character_name(story_prompt) or "the character"

        # Generate a base character image for consistency across panels
        print("Generating base character image for consistency...")
        base_prompt = f"A friendly {art_style} illustration of {character_name}, whole body visible, simple background, bright colors, child-friendly"

        # Check if using StableDiffusionXLPipeline or StableDiffusionPipeline (custom model may use different API)
        is_sdxl = isinstance(pipe, StableDiffusionXLPipeline)

        if is_sdxl:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                num_inference_steps=30
            ).images[0]
        else:
            # For regular SD pipeline
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                num_inference_steps=30,
                guidance_scale=7.5
            ).images[0]

        # Resize base image for consistency
        base_image = base_image.resize((768, 768))

        # Generate each panel
        for i, panel_text in enumerate(panel_texts):
            print(f"Generating image for panel {i+1}...")
            # Create a prompt that helps the model understand it's a comic panel for children
            prompt = f"A friendly {art_style} illustration for a children's storybook showing: {panel_text}, featuring {character_name}, bright colors, simple shapes, cute characters, child-friendly"

            # Generate the image using the base character for consistency
            if is_sdxl:
                image = pipe(
                    prompt=prompt,
                    image=base_image,  # Use base image for character consistency
                    strength=0.7,      # Keep some elements of base image but allow for new scene
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                    num_inference_steps=30
                ).images[0]
            else:
                # For regular SD pipeline
                image = pipe(
                    prompt=prompt,
                    image=base_image,  # Use base image for character consistency
                    strength=0.7,      # Keep some elements of base image but allow for new scene
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                    num_inference_steps=30,
                    guidance_scale=7.5
                ).images[0]

            # Resize to standard panel size
            image = image.resize((512, 512))

            # Add speech bubble if there's dialogue
            if dialogues[i]:
                # Calculate a position for the speech bubble based on panel number
                position = (256, 100 + (i % 2) * 50)  # Vary position slightly between panels
                image = add_speech_bubble(image, dialogues[i], position)

            panel_images.append(image)

            # Save the individual panel
            image_path = f"panel_{i+1}.png"
            image.save(image_path)

        # Create a combined comic strip
        print("Creating combined comic strip...")
        rows = int(np.ceil(num_panels / 2))  # Calculate how many rows we need
        columns = min(2, num_panels)         # Max 2 columns

        final_width = 512 * columns
        final_height = 512 * rows

        comic_strip = Image.new("RGB", (final_width, final_height), (255, 255, 255))

        for i, image in enumerate(panel_images):
            x_offset = (i % 2) * 512
            y_offset = (i // 2) * 512
            comic_strip.paste(image, (x_offset, y_offset))

        # Save the combined comic strip
        comic_strip.save("comic_strip.png")
        print("Comic generation complete! All panels saved as PNG files and combined into comic_strip.png")

        return story_text, story_panels, panel_images, comic_strip

    except Exception as e:
        print(f"Error in image generation: {e}")
        print("Unable to generate images. Please check if you have enough GPU memory and proper installations.")
        return story_text, story_panels, None, None

# Web UI using Gradio
def create_ui():
    with gr.Blocks() as app:
        gr.Markdown("# Story to Comic Generator with InkoloRA")
        gr.Markdown("Enter a prompt to generate a story and convert it into comic panels with speech bubbles using the InkoloRA model")

        with gr.Row():
            with gr.Column():
                story_prompt = gr.Textbox(
                    label="Story Prompt",
                    placeholder="Write a story about a brave cat who learns to fly",
                    lines=2
                )
                num_panels = gr.Slider(minimum=2, maximum=8, value=4, step=1, label="Number of Panels")
                model_path = gr.Textbox(
                    label="Story Model Path (optional)",
                    placeholder="/content/drive/MyDrive/fine_tuned_tiny_stories_model",
                    value="/content/drive/MyDrive/fine_tuned_tiny_stories_model"
                )
                art_style = gr.Dropdown(
                    choices=[
                        "cartoon style",
                        "manga style",
                        "comic book style",
                        "watercolor style",
                        "3D rendered",
                        "pixel art style",
                        "oil painting style",
                        "storybook illustration style",
                        "Inkolo style" # Added new option for your custom model's style
                    ],
                    value="Inkolo style", # Set your custom style as default
                    label="Art Style"
                )
                generate_button = gr.Button("Generate Comic")

            with gr.Column():
                story_output = gr.Textbox(label="Generated Story", lines=8)

        # Gallery component for displaying individual panels
        panel_gallery = gr.Gallery(label="Comic Panels", columns=2, rows=2, height=500)

        # Image component for displaying the combined comic strip
        comic_strip_image = gr.Image(label="Combined Comic Strip", height=600)

        # Status output for error messages
        status_output = gr.Textbox(label="Status", lines=2)

        def run_generation(prompt, panels, path, style):
            if not prompt:
                return "Please enter a story prompt", [], None, "Error: No prompt provided"

            model_path_to_use = path if path.strip() else None

            try:
                story_text, story_panels, images, comic_strip = generate_comic(prompt, int(panels), model_path_to_use, style)

                if story_text and images and comic_strip:
                    status = "Success: Comic generated successfully with InkoloRA model and speech bubbles!"
                elif story_text and images:
                    status = "Success: Individual panels generated but combined strip failed"
                elif story_text:
                    status = "Warning: Story generated but image creation failed"
                else:
                    status = "Error: Story generation failed"

                return story_text, images if images else [], comic_strip, status
            except Exception as e:
                return f"An error occurred: {str(e)}", [], None, f"Error: {str(e)}"

        generate_button.click(
            run_generation,
            inputs=[story_prompt, num_panels, model_path, art_style],
            outputs=[story_output, panel_gallery, comic_strip_image, status_output]
        )

    return app

# Main function to either run the generator directly or launch the web UI
def main(use_web_ui=True):
    # Mount Google Drive to access the models
    try:
        drive.mount('/content/drive')
        print("Google Drive mounted successfully")
    except Exception as e:
        print(f"Error mounting Google Drive: {e}")
        print("Continuing without mounting drive - this may cause issues accessing models")

    if use_web_ui:
        app = create_ui()
        app.launch(share=True)  # Enable sharing for remote access
    else:
        # Example without web UI
        story_prompt = input("Enter a story prompt: ")
        num_panels = int(input("Enter number of panels (2-8): "))
        model_path = input("Enter story model path (leave empty for default): ")
        art_style = input("Enter art style (e.g., cartoon style, manga style, Inkolo style): ")

        story, panels, images, comic_strip = generate_comic(story_prompt, num_panels, model_path or None, art_style)

        if images:
            for i, img in enumerate(images):
                plt.figure(figsize=(8, 8))
                plt.imshow(img)
                plt.title(f"Panel {i+1}")
                plt.axis('off')
                plt.show()

            # Display combined comic strip
            if comic_strip:
                plt.figure(figsize=(12, 12))
                plt.imshow(comic_strip)
                plt.title("Complete Comic Strip")
                plt.axis('off')
                plt.show()

if __name__ == "__main__":
    main(use_web_ui=True)  # Set to False to use command line instead of web UI

ModuleNotFoundError: No module named 'gradio'

In [ ]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.4 MB/s eta 0:00:00


In [ ]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import cv2
from transformers import AutoTokenizer, AutoModelForCausalLM
from diffusers import StableDiffusionPipeline, StableDiffusionXLPipeline
from PIL import Image, ImageDraw, ImageFont
import gradio as gr
from google.colab import drive


# Load the fine-tuned model
def load_fine_tuned_model(model_path=None):
    # If no model path provided, use default path
    if not model_path:
        model_path = '/content/drive/MyDrive/fine_tuned_tiny_stories_model'

    try:
        print(f"Loading fine-tuned model from {model_path}...")
        model = AutoModelForCausalLM.from_pretrained(model_path)
        tokenizer = AutoTokenizer.from_pretrained(model_path)

        # Move model to GPU if available
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)

        print(f"Model loaded successfully and moved to {device}")
        return model, tokenizer
    except Exception as e:
        print(f"Error loading model: {e}")
        return None, None

# Helper functions for story generation
def extract_character_name(prompt):
    """Extract potential character names from the prompt"""
    # Common superhero/character names to check for
    character_names = ["Spider-Man", "Peter Parker", "Batman", "Superman", "Wonder Woman",
                       "Iron Man", "Captain America", "Thor", "Hulk", "Black Widow"]

    # Check if any known character names are in the prompt
    for name in character_names:
        if name.lower() in prompt.lower():
            return name

    # Otherwise extract words that might be names (capitalized words)
    words = prompt.split()
    for word in words:
        if word[0].isupper() and len(word) > 1 and word.lower() not in ["write", "story", "about"]:
            return word

    return None

def extract_topic(prompt):
    """Extract the main topic from the prompt"""
    # Remove common instruction words
    cleaned = prompt.lower().replace("write", "").replace("story", "").replace("about", "")

    # Look for interesting keywords
    keywords = ["adventure", "flying", "jumping", "fighting", "learning", "playing", "making friends"]

    for keyword in keywords:
        if keyword in cleaned:
            return keyword

    # Return the cleaned prompt if no specific keyword found
    return cleaned.strip()

def create_structured_childrens_story(character="the character", topic="having an adventure"):
    """Create a structured children's story with simple language and short sentences"""

    # Clean up inputs
    if not character or len(character) < 2:
        character = "the little hero"

    # Create a story with the structure of the Peter Parker example
    story = f"""The story of {character} began in the park. {character} was a little child who loved to play with friends.

One sunny day, a friend came over and said, "{character}, you must come and play with me! I'm the best at {topic}!"

{character} was so excited. {character} jumped up and down with happiness. They all played together and had lots of fun.

{character} learned how to be good at {topic}. Soon, everyone wanted to play with {character}.

{character} and the friends became the best of friends. They played every day after school.

{character} was so happy to have such good friends. They would be friends forever and have many more adventures together.
"""
    return story

# Enhanced story generation function using fine-tuned model
def generate_story(model, tokenizer, prompt, max_length=400):
    # Format prompt in a way that's more suitable for children's stories
    story_prompt = f"Write a simple children's story about {prompt}. Use simple language and short sentences."

    try:
        # Encode the prompt
        input_ids = tokenizer.encode(story_prompt, return_tensors='pt')

        # Move to the same device as the model
        device = next(model.parameters()).device
        input_ids = input_ids.to(device)

        # Generate text with different parameters more suited to simple children's stories
        output = model.generate(
            input_ids,
            max_length=max_length,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            top_k=40,
            top_p=0.9,
            temperature=0.7,  # Lower temperature for more consistent, simpler text
            do_sample=True
        )

        # Decode the generated text
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

        # Check quality of response
        if len(generated_text) < 50 or "In a world where anything was possible" in generated_text:
            print("Warning: Low quality story generated. Generating a structured children's story...")

            # Create a more structured children's story instead of using the default template
            character_name = extract_character_name(prompt) or "the character"
            topic = extract_topic(prompt) or "having an adventure"

            return create_structured_childrens_story(character_name, topic)

        return generated_text
    except Exception as e:
        print(f"Error in story generation: {e}")
        return create_structured_childrens_story(extract_character_name(prompt), extract_topic(prompt))

# Function to create a fallback story when needed
def create_dynamic_fallback_story(prompt):
    # Extract character and topics
    character = extract_character_name(prompt) or "the character"
    topic = extract_topic(prompt) or "having fun"

    # Create a story similar to the Peter Parker example
    story = f"""
The story of {character} began in the park. {character} was a little child who loved to play with friends.

One sunny day, a friend came over and said, "{character}, you must come and play with me! I'm the best at {topic}!"

{character} was so excited. {character} jumped up and down with happiness. They all played together and had lots of fun.

{character} learned how to be good at {topic}. Soon, everyone wanted to play with {character}.

{character} and the friends became the best of friends. They played every day after school.

{character} was so happy to have such good friends. They would be friends forever and have many more adventures together.
"""
    return story

# Function to split the story into panels and also extract dialogues for speech bubbles
def split_story_into_panels(story, num_panels=4):
    # First check if we have enough content to split
    if len(story) < 100:
        print("Warning: Story is very short. Panel division may not be optimal.")

    # Try to split by paragraphs first for more natural divisions
    paragraphs = story.split('\n')
    paragraphs = [p for p in paragraphs if p.strip()]  # Remove empty paragraphs

    # Extract dialogues for speech bubbles
    dialogues = []
    panel_texts = []

    if len(paragraphs) >= num_panels:
        # We have enough paragraphs, so use them as natural divisions
        if len(paragraphs) > num_panels:
            # Combine paragraphs to match the desired number of panels
            step = len(paragraphs) / num_panels
            panels = []
            for i in range(num_panels):
                start_idx = int(i * step)
                end_idx = int((i + 1) * step) if i < num_panels - 1 else len(paragraphs)
                panel_text = " ".join(paragraphs[start_idx:end_idx])
                panels.append(panel_text)

                # Extract dialogue (text within quotes) for speech bubbles
                dialogue = extract_dialogue(panel_text)
                dialogues.append(dialogue)

                # Remove dialogue from panel text for better image prompts
                clean_text = panel_text.replace(dialogue, "") if dialogue else panel_text
                panel_texts.append(clean_text)
            return panels, panel_texts, dialogues
        else:
            # Perfect match or fewer paragraphs than panels
            panels = paragraphs[:num_panels]
            for panel in panels:
                dialogue = extract_dialogue(panel)
                dialogues.append(dialogue)
                clean_text = panel.replace(dialogue, "") if dialogue else panel
                panel_texts.append(clean_text)
            return panels, panel_texts, dialogues

    # If we don't have enough paragraphs, fall back to sentence splitting
    sentences = []
    # Split by common sentence endings
    for chunk in story.split('. '):
        sentences.extend([s.strip() + '.' for s in chunk.split('! ')])
    sentences = [s for s in sentences if len(s) > 1]  # Remove empty or too-short sentences

    # Handle edge case where there are fewer sentences than panels
    if len(sentences) < num_panels:
        # Create more sentences by splitting the longest ones
        while len(sentences) < num_panels:
            longest_idx = max(range(len(sentences)), key=lambda i: len(sentences[i]))
            longest = sentences[longest_idx]

            # Split at a comma or other natural break point if possible
            if ', ' in longest:
                parts = longest.split(', ', 1)
                sentences[longest_idx] = parts[0] + '.'
                sentences.insert(longest_idx + 1, parts[1])
            else:
                # If no good split point, duplicate the sentence with a variation
                sentences.append(f"Meanwhile, {longest.lower()}")

    # Calculate how many sentences per panel for a balanced distribution
    step = max(1, len(sentences) // num_panels)

    panels = []
    for i in range(num_panels):
        start_idx = i * step
        # For the last panel, include any remaining sentences
        end_idx = (i + 1) * step if i < num_panels - 1 else len(sentences)
        panel_text = " ".join(sentences[start_idx:end_idx])
        panels.append(panel_text)

        # Extract dialogue for speech bubbles
        dialogue = extract_dialogue(panel_text)
        dialogues.append(dialogue)

        # Remove dialogue from panel text for better image prompts
        clean_text = panel_text.replace(dialogue, "") if dialogue else panel_text
        panel_texts.append(clean_text)

    return panels, panel_texts, dialogues

# Function to extract dialogue from text (text within quotes)
def extract_dialogue(text):
    import re
    dialogue_matches = re.findall(r'"([^"]*)"', text)
    if dialogue_matches:
        # Return the first dialogue found
        return dialogue_matches[0]
    return ""

# Load the custom InkoloRA model
def load_custom_model():
    print("Loading InkoloRA model - this may take a while...")

    # Define custom model path
    custom_model_path = "/content/drive/MyDrive/InkoloRA.safetensors"

    # Check if the model file exists
    if not os.path.exists(custom_model_path):
        print(f"Warning: Custom model not found at {custom_model_path}. Falling back to Stable Diffusion XL.")
        return load_sdxl_model()

    try:
        # Load the model - adjust the model_id based on what base model your safetensors file is trained on
        # Assuming it's SD 1.5 based; modify if it's based on a different architecture
        pipe = StableDiffusionPipeline.from_pretrained(
            "runwayml/stable-diffusion-v1-5",
            torch_dtype=torch.float16,
            safety_checker=None
        )

        # Load the custom weights
        pipe.unet.load_attn_procs(custom_model_path)

        # Check if CUDA is available, otherwise use CPU
        device = "cuda" if torch.cuda.is_available() else "cpu"
        if device == "cpu":
            print("Warning: CUDA not available, using CPU. This will be very slow.")
        pipe = pipe.to(device)

        print("InkoloRA model loaded successfully!")
        return pipe
    except Exception as e:
        print(f"Error loading custom model: {e}")
        print("Falling back to Stable Diffusion XL...")
        return load_sdxl_model()

# Original SDXL model load function as fallback
def load_sdxl_model():
    print("Loading SDXL model - this may take a while...")
    pipe = StableDiffusionXLPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True
    )

    # Check if CUDA is available, otherwise use CPU
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device == "cpu":
        print("Warning: CUDA not available, using CPU. This will be very slow.")
    pipe = pipe.to(device)

    return pipe

# Function to add speech bubbles to images
def add_speech_bubble(image, text, position=None):
    if not text:
        return image

    # Convert PIL image to numpy array for OpenCV operations
    img = np.array(image.convert("RGBA"))
    height, width, _ = img.shape

    # If position not provided, place the bubble at a suitable position
    if position is None:
        position = (width // 2, height // 4)

    # Create a transparent layer for the bubble
    bubble = np.zeros((height, width, 4), dtype=np.uint8)

    # Calculate bubble size based on text length
    bubble_width = min(300, max(150, len(text) * 10))
    bubble_height = min(130, max(60, (len(text) // 20 + 1) * 30))

    # Draw speech bubble
    cv2.ellipse(bubble, position, (bubble_width, bubble_height), 0, 0, 360, (255, 255, 255, 255), -1)

    # Add a pointer to make it look like a speech bubble
    pointer_x = position[0]
    pointer_y = position[1] + bubble_height
    triangle_pts = np.array([
        [pointer_x - 20, pointer_y - 10],
        [pointer_x + 20, pointer_y - 10],
        [pointer_x, pointer_y + 30]
    ])
    cv2.fillPoly(bubble, [triangle_pts], (255, 255, 255, 255))

    bubble_pil = Image.fromarray(bubble)

    # Draw text
    draw = ImageDraw.Draw(bubble_pil)

    # Try to load a font, fall back to default if not available
    try:
        font = ImageFont.truetype("arial.ttf", 24)
    except:
        try:
            font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 24)
        except:
            font = ImageFont.load_default()

    # Wrap text to fit in the bubble
    wrapped_text = wrap_text(text, font, bubble_width - 40)

    # Calculate text position for centering
    text_x = position[0] - (bubble_width // 2) + 20
    text_y = position[1] - (bubble_height // 2) + 10

    # Draw the wrapped text
    for i, line in enumerate(wrapped_text):
        draw.text((text_x, text_y + i * 30), line, fill="black", font=font)

    # Combine the original image with the bubble
    return Image.alpha_composite(image.convert("RGBA"), bubble_pil)

# Function to wrap text to fit within a certain width
def wrap_text(text, font, max_width):
    words = text.split()
    wrapped_lines = []
    current_line = []

    for word in words:
        # Test if adding this word would exceed the width
        test_line = ' '.join(current_line + [word])
        try:
            line_width = font.getsize(test_line)[0]
        except:
            # If getsize not available, estimate based on character count
            line_width = len(test_line) * 10

        if line_width <= max_width:
            current_line.append(word)
        else:
            wrapped_lines.append(' '.join(current_line))
            current_line = [word]

    # Add the last line
    if current_line:
        wrapped_lines.append(' '.join(current_line))

    return wrapped_lines

# Function to generate comic panels from story prompt using our fine-tuned model
def generate_comic(story_prompt, num_panels=4, model_path=None, art_style="cartoon style"):
    # Make sure the story prompt is compelling
    if len(story_prompt) < 10:
        story_prompt = f"Create an interesting story about {story_prompt}"

    try:
        # Load the fine-tuned model
        print("Loading fine-tuned model...")
        model, tokenizer = load_fine_tuned_model(model_path)

        if model is None or tokenizer is None:
            raise Exception("Failed to load the fine-tuned model")

        # Generate story from prompt with multiple attempts if needed
        print(f"Generating story from prompt: '{story_prompt}'")
        story_text = generate_story(model, tokenizer, story_prompt)
        print("Story generated:")
        print(story_text)

        # Check if story generated properly
        if not story_text or len(story_text) < 50:
            print("Story generation produced insufficient content. Trying one more time with different approach.")
            # Try a more direct approach with the model
            direct_prompt = f"Write a simple children's story about {story_prompt}. Use short sentences."

            story_text = generate_story(model, tokenizer, direct_prompt, max_length=500)

            if len(story_text) < 50:
                print("Still unable to generate a good story. Using the structured children's story.")
                character_name = extract_character_name(story_prompt) or "the character"
                topic = extract_topic(story_prompt) or "having an adventure"
                story_text = create_structured_childrens_story(character_name, topic)
    except Exception as e:
        print(f"Error generating story: {e}")
        # Provide a fallback story using our improved fallback generator
        character_name = extract_character_name(story_prompt) or "the character"
        topic = extract_topic(story_prompt) or "having an adventure"
        story_text = create_structured_childrens_story(character_name, topic)

    # Split story into panels and extract dialogues
    print("\nSplitting story into panels...")
    story_panels, panel_texts, dialogues = split_story_into_panels(story_text, num_panels)
    for i, panel in enumerate(story_panels):
        print(f"Panel {i+1}: {panel}")
        if dialogues[i]:
            print(f"Dialogue for panel {i+1}: \"{dialogues[i]}\"")

    try:
        # Load our custom InkoloRA model instead of SDXL
        pipe = load_custom_model()

        print("\nGenerating panel images...")
        panel_images = []

        # Extract character name for consistency
        character_name = extract_character_name(story_prompt) or "the character"

        # Generate a base character image for consistency across panels
        print("Generating base character image for consistency...")
        base_prompt = f"A friendly {art_style} illustration of {character_name}, whole body visible, simple background, bright colors, child-friendly"

        # Check if using StableDiffusionXLPipeline or StableDiffusionPipeline (custom model may use different API)
        is_sdxl = isinstance(pipe, StableDiffusionXLPipeline)

        if is_sdxl:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                num_inference_steps=30
            ).images[0]
        else:
            # For regular SD pipeline
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                num_inference_steps=30,
                guidance_scale=7.5
            ).images[0]

        # Resize base image for consistency
        base_image = base_image.resize((768, 768))

        # Generate each panel
        for i, panel_text in enumerate(panel_texts):
            print(f"Generating image for panel {i+1}...")
            # Create a prompt that helps the model understand it's a comic panel for children
            prompt = f"A friendly {art_style} illustration for a children's storybook showing: {panel_text}, featuring {character_name}, bright colors, simple shapes, cute characters, child-friendly"

            # Generate the image using the base character for consistency
            if is_sdxl:
                image = pipe(
                    prompt=prompt,
                    image=base_image,  # Use base image for character consistency
                    strength=0.7,      # Keep some elements of base image but allow for new scene
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                    num_inference_steps=30
                ).images[0]
            else:
                # For regular SD pipeline
                image = pipe(
                    prompt=prompt,
                    image=base_image,  # Use base image for character consistency
                    strength=0.7,      # Keep some elements of base image but allow for new scene
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                    num_inference_steps=30,
                    guidance_scale=7.5
                ).images[0]

            # Resize to standard panel size
            image = image.resize((512, 512))

            # Add speech bubble if there's dialogue
            if dialogues[i]:
                # Calculate a position for the speech bubble based on panel number
                position = (256, 100 + (i % 2) * 50)  # Vary position slightly between panels
                image = add_speech_bubble(image, dialogues[i], position)

            panel_images.append(image)

            # Save the individual panel
            image_path = f"panel_{i+1}.png"
            image.save(image_path)

        # Create a combined comic strip
        print("Creating combined comic strip...")
        rows = int(np.ceil(num_panels / 2))  # Calculate how many rows we need
        columns = min(2, num_panels)         # Max 2 columns

        final_width = 512 * columns
        final_height = 512 * rows

        comic_strip = Image.new("RGB", (final_width, final_height), (255, 255, 255))

        for i, image in enumerate(panel_images):
            x_offset = (i % 2) * 512
            y_offset = (i // 2) * 512
            comic_strip.paste(image, (x_offset, y_offset))

        # Save the combined comic strip
        comic_strip.save("comic_strip.png")
        print("Comic generation complete! All panels saved as PNG files and combined into comic_strip.png")

        return story_text, story_panels, panel_images, comic_strip

    except Exception as e:
        print(f"Error in image generation: {e}")
        print("Unable to generate images. Please check if you have enough GPU memory and proper installations.")
        return story_text, story_panels, None, None

# Web UI using Gradio
def create_ui():
    with gr.Blocks() as app:
        gr.Markdown("# Story to Comic Generator with InkoloRA")
        gr.Markdown("Enter a prompt to generate a story and convert it into comic panels with speech bubbles using the InkoloRA model")

        with gr.Row():
            with gr.Column():
                story_prompt = gr.Textbox(
                    label="Story Prompt",
                    placeholder="Write a story about a brave cat who learns to fly",
                    lines=2
                )
                num_panels = gr.Slider(minimum=2, maximum=8, value=4, step=1, label="Number of Panels")
                model_path = gr.Textbox(
                    label="Story Model Path (optional)",
                    placeholder="/content/drive/MyDrive/fine_tuned_tiny_stories_model",
                    value="/content/drive/MyDrive/fine_tuned_tiny_stories_model"
                )
                art_style = gr.Dropdown(
                    choices=[
                        "cartoon style",
                        "manga style",
                        "comic book style",
                        "watercolor style",
                        "3D rendered",
                        "pixel art style",
                        "oil painting style",
                        "storybook illustration style",
                        "Inkolo style" # Added new option for your custom model's style
                    ],
                    value="Inkolo style", # Set your custom style as default
                    label="Art Style"
                )
                generate_button = gr.Button("Generate Comic")

            with gr.Column():
                story_output = gr.Textbox(label="Generated Story", lines=8)

        # Gallery component for displaying individual panels
        panel_gallery = gr.Gallery(label="Comic Panels", columns=2, rows=2, height=500)

        # Image component for displaying the combined comic strip
        comic_strip_image = gr.Image(label="Combined Comic Strip", height=600)

        # Status output for error messages
        status_output = gr.Textbox(label="Status", lines=2)

        def run_generation(prompt, panels, path, style):
            if not prompt:
                return "Please enter a story prompt", [], None, "Error: No prompt provided"

            model_path_to_use = path if path.strip() else None

            try:
                story_text, story_panels, images, comic_strip = generate_comic(prompt, int(panels), model_path_to_use, style)

                if story_text and images and comic_strip:
                    status = "Success: Comic generated successfully with InkoloRA model and speech bubbles!"
                elif story_text and images:
                    status = "Success: Individual panels generated but combined strip failed"
                elif story_text:
                    status = "Warning: Story generated but image creation failed"
                else:
                    status = "Error: Story generation failed"

                return story_text, images if images else [], comic_strip, status
            except Exception as e:
                return f"An error occurred: {str(e)}", [], None, f"Error: {str(e)}"

        generate_button.click(
            run_generation,
            inputs=[story_prompt, num_panels, model_path, art_style],
            outputs=[story_output, panel_gallery, comic_strip_image, status_output]
        )

    return app

# Main function to either run the generator directly or launch the web UI
def main(use_web_ui=True):
    # Mount Google Drive to access the models
    try:
        drive.mount('/content/drive')
        print("Google Drive mounted successfully")
    except Exception as e:
        print(f"Error mounting Google Drive: {e}")
        print("Continuing without mounting drive - this may cause issues accessing models")

    if use_web_ui:
        app = create_ui()
        app.launch(share=True)  # Enable sharing for remote access
    else:
        # Example without web UI
        story_prompt = input("Enter a story prompt: ")
        num_panels = int(input("Enter number of panels (2-8): "))
        model_path = input("Enter story model path (leave empty for default): ")
        art_style = input("Enter art style (e.g., cartoon style, manga style, Inkolo style): ")

        story, panels, images, comic_strip = generate_comic(story_prompt, num_panels, model_path or None, art_style)

        if images:
            for i, img in enumerate(images):
                plt.figure(figsize=(8, 8))
                plt.imshow(img)
                plt.title(f"Panel {i+1}")
                plt.axis('off')
                plt.show()

            # Display combined comic strip
            if comic_strip:
                plt.figure(figsize=(12, 12))
                plt.imshow(comic_strip)
                plt.title("Complete Comic Strip")
                plt.axis('off')
                plt.show()

if __name__ == "__main__":
    main(use_web_ui=True)  # Set to False to use command line instead of web UI

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dbb76da67e00670c30.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import cv2
from transformers import AutoTokenizer, AutoModelForCausalLM
from diffusers import StableDiffusionPipeline, StableDiffusionXLPipeline
from PIL import Image, ImageDraw, ImageFont
import gradio as gr
from google.colab import drive


# Load the fine-tuned model
def load_fine_tuned_model(model_path=None):
    # If no model path provided, use default path
    if not model_path:
        model_path = '/content/drive/MyDrive/fine_tuned_tiny_stories_model'

    try:
        print(f"Loading fine-tuned model from {model_path}...")
        model = AutoModelForCausalLM.from_pretrained(model_path)
        tokenizer = AutoTokenizer.from_pretrained(model_path)

        # Move model to GPU if available
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)

        print(f"Model loaded successfully and moved to {device}")
        return model, tokenizer
    except Exception as e:
        print(f"Error loading model: {e}")
        return None, None

# Helper functions for story generation
def extract_character_name(prompt):
    """Extract potential character names from the prompt"""
    # Common superhero/character names to check for
    character_names = ["Spider-Man", "Peter Parker", "Batman", "Superman", "Wonder Woman",
                       "Iron Man", "Captain America", "Thor", "Hulk", "Black Widow"]

    # Check if any known character names are in the prompt
    for name in character_names:
        if name.lower() in prompt.lower():
            return name

    # Otherwise extract words that might be names (capitalized words)
    words = prompt.split()
    for word in words:
        if word[0].isupper() and len(word) > 1 and word.lower() not in ["write", "story", "about"]:
            return word

    return None

def extract_topic(prompt):
    """Extract the main topic from the prompt"""
    # Remove common instruction words
    cleaned = prompt.lower().replace("write", "").replace("story", "").replace("about", "")

    # Look for interesting keywords
    keywords = ["adventure", "flying", "jumping", "fighting", "learning", "playing", "making friends"]

    for keyword in keywords:
        if keyword in cleaned:
            return keyword

    # Return the cleaned prompt if no specific keyword found
    return cleaned.strip()

def create_structured_childrens_story(character="the character", topic="having an adventure"):
    """Create a structured children's story with simple language and short sentences"""

    # Clean up inputs
    if not character or len(character) < 2:
        character = "the little hero"

    # Create a story with the structure of the Peter Parker example
    story = f"""The story of {character} began in the park. {character} was a little child who loved to play with friends.

One sunny day, a friend came over and said, "{character}, you must come and play with me! I'm the best at {topic}!"

{character} was so excited. {character} jumped up and down with happiness. They all played together and had lots of fun.

{character} learned how to be good at {topic}. Soon, everyone wanted to play with {character}.

{character} and the friends became the best of friends. They played every day after school.

{character} was so happy to have such good friends. They would be friends forever and have many more adventures together.
"""
    return story

# Enhanced story generation function using fine-tuned model
def generate_story(model, tokenizer, prompt, max_length=400):
    # Format prompt in a way that's more suitable for children's stories
    story_prompt = f"Write a simple children's story about {prompt}. Use simple language and short sentences."

    try:
        # Encode the prompt
        input_ids = tokenizer.encode(story_prompt, return_tensors='pt')

        # Move to the same device as the model
        device = next(model.parameters()).device
        input_ids = input_ids.to(device)

        # Generate text with different parameters more suited to simple children's stories
        output = model.generate(
            input_ids,
            max_length=max_length,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            top_k=40,
            top_p=0.9,
            temperature=0.7,  # Lower temperature for more consistent, simpler text
            do_sample=True
        )

        # Decode the generated text
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

        # Check quality of response
        if len(generated_text) < 50 or "In a world where anything was possible" in generated_text:
            print("Warning: Low quality story generated. Generating a structured children's story...")

            # Create a more structured children's story instead of using the default template
            character_name = extract_character_name(prompt) or "the character"
            topic = extract_topic(prompt) or "having an adventure"

            return create_structured_childrens_story(character_name, topic)

        return generated_text
    except Exception as e:
        print(f"Error in story generation: {e}")
        return create_structured_childrens_story(extract_character_name(prompt), extract_topic(prompt))

# Function to split the story into panels and also extract dialogues for speech bubbles
def split_story_into_panels(story, num_panels=4):
    # First check if we have enough content to split
    if len(story) < 100:
        print("Warning: Story is very short. Panel division may not be optimal.")

    # Try to split by paragraphs first for more natural divisions
    paragraphs = story.split('\n')
    paragraphs = [p for p in paragraphs if p.strip()]  # Remove empty paragraphs

    # Extract dialogues for speech bubbles
    dialogues = []
    panel_texts = []

    if len(paragraphs) >= num_panels:
        # We have enough paragraphs, so use them as natural divisions
        if len(paragraphs) > num_panels:
            # Combine paragraphs to match the desired number of panels
            step = len(paragraphs) / num_panels
            panels = []
            for i in range(num_panels):
                start_idx = int(i * step)
                end_idx = int((i + 1) * step) if i < num_panels - 1 else len(paragraphs)
                panel_text = " ".join(paragraphs[start_idx:end_idx])
                panels.append(panel_text)

                # Extract dialogue (text within quotes) for speech bubbles
                dialogue = extract_dialogue(panel_text)
                dialogues.append(dialogue)

                # Remove dialogue from panel text for better image prompts
                clean_text = panel_text.replace(dialogue, "") if dialogue else panel_text
                panel_texts.append(clean_text)
            return panels, panel_texts, dialogues
        else:
            # Perfect match or fewer paragraphs than panels
            panels = paragraphs[:num_panels]
            for panel in panels:
                dialogue = extract_dialogue(panel)
                dialogues.append(dialogue)
                clean_text = panel.replace(dialogue, "") if dialogue else panel
                panel_texts.append(clean_text)
            return panels, panel_texts, dialogues

    # If we don't have enough paragraphs, fall back to sentence splitting
    sentences = []
    # Split by common sentence endings
    for chunk in story.split('. '):
        sentences.extend([s.strip() + '.' for s in chunk.split('! ')])
    sentences = [s for s in sentences if len(s) > 1]  # Remove empty or too-short sentences

    # Handle edge case where there are fewer sentences than panels
    if len(sentences) < num_panels:
        # Create more sentences by splitting the longest ones
        while len(sentences) < num_panels:
            longest_idx = max(range(len(sentences)), key=lambda i: len(sentences[i]))
            longest = sentences[longest_idx]

            # Split at a comma or other natural break point if possible
            if ', ' in longest:
                parts = longest.split(', ', 1)
                sentences[longest_idx] = parts[0] + '.'
                sentences.insert(longest_idx + 1, parts[1])
            else:
                # If no good split point, duplicate the sentence with a variation
                sentences.append(f"Meanwhile, {longest.lower()}")

    # Calculate how many sentences per panel for a balanced distribution
    step = max(1, len(sentences) // num_panels)

    panels = []
    for i in range(num_panels):
        start_idx = i * step
        # For the last panel, include any remaining sentences
        end_idx = (i + 1) * step if i < num_panels - 1 else len(sentences)
        panel_text = " ".join(sentences[start_idx:end_idx])
        panels.append(panel_text)

        # Extract dialogue for speech bubbles
        dialogue = extract_dialogue(panel_text)
        dialogues.append(dialogue)

        # Remove dialogue from panel text for better image prompts
        clean_text = panel_text.replace(dialogue, "") if dialogue else panel_text
        panel_texts.append(clean_text)

    return panels, panel_texts, dialogues

# Function to extract dialogue from text (text within quotes)
def extract_dialogue(text):
    import re
    dialogue_matches = re.findall(r'"([^"]*)"', text)
    if dialogue_matches:
        # Return the first dialogue found
        return dialogue_matches[0]
    return ""

# Load the custom InkoloRA model
def load_custom_model():
    print("Loading InkoloRA model - this may take a while...")

    # Define custom model path
    custom_model_path = "/content/drive/MyDrive/InkoloRA.safetensors"

    # Check if the model file exists
    if not os.path.exists(custom_model_path):
        print(f"Warning: Custom model not found at {custom_model_path}. Falling back to Stable Diffusion XL.")
        return load_sdxl_model()

    try:
        # Load the model - adjust the model_id based on what base model your safetensors file is trained on
        # Assuming it's SD 1.5 based; modify if it's based on a different architecture
        pipe = StableDiffusionPipeline.from_pretrained(
            "runwayml/stable-diffusion-v1-5",
            torch_dtype=torch.float16,
            safety_checker=None
        )

        # Load the custom weights
        pipe.unet.load_attn_procs(custom_model_path)

        # Check if CUDA is available, otherwise use CPU
        device = "cuda" if torch.cuda.is_available() else "cpu"
        if device == "cpu":
            print("Warning: CUDA not available, using CPU. This will be very slow.")
        pipe = pipe.to(device)

        print("InkoloRA model loaded successfully!")
        return pipe
    except Exception as e:
        print(f"Error loading custom model: {e}")
        print("Falling back to Stable Diffusion XL...")
        return load_sdxl_model()

# Original SDXL model load function as fallback
def load_sdxl_model():
    print("Loading SDXL model - this may take a while...")
    pipe = StableDiffusionXLPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True
    )

    # Check if CUDA is available, otherwise use CPU
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device == "cpu":
        print("Warning: CUDA not available, using CPU. This will be very slow.")
    pipe = pipe.to(device)

    return pipe

# Function to add speech bubbles to images
def add_speech_bubble(image, text, position=None):
    if not text:
        return image

    # Convert PIL image to numpy array for OpenCV operations
    img = np.array(image.convert("RGBA"))
    height, width, _ = img.shape

    # If position not provided, place the bubble at a suitable position
    if position is None:
        position = (width // 2, height // 4)

    # Create a transparent layer for the bubble
    bubble = np.zeros((height, width, 4), dtype=np.uint8)

    # Calculate bubble size based on text length
    bubble_width = min(300, max(150, len(text) * 10))
    bubble_height = min(130, max(60, (len(text) // 20 + 1) * 30))

    # Draw speech bubble
    cv2.ellipse(bubble, position, (bubble_width, bubble_height), 0, 0, 360, (255, 255, 255, 255), -1)

    # Add a pointer to make it look like a speech bubble
    pointer_x = position[0]
    pointer_y = position[1] + bubble_height
    triangle_pts = np.array([
        [pointer_x - 20, pointer_y - 10],
        [pointer_x + 20, pointer_y - 10],
        [pointer_x, pointer_y + 30]
    ])
    cv2.fillPoly(bubble, [triangle_pts], (255, 255, 255, 255))

    bubble_pil = Image.fromarray(bubble)

    # Draw text
    draw = ImageDraw.Draw(bubble_pil)

    # Try to load a font, fall back to default if not available
    try:
        font = ImageFont.truetype("arial.ttf", 24)
    except:
        try:
            font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 24)
        except:
            font = ImageFont.load_default()

    # Wrap text to fit in the bubble
    wrapped_text = wrap_text(text, font, bubble_width - 40)

    # Calculate text position for centering
    text_x = position[0] - (bubble_width // 2) + 20
    text_y = position[1] - (bubble_height // 2) + 10

    # Draw the wrapped text
    for i, line in enumerate(wrapped_text):
        draw.text((text_x, text_y + i * 30), line, fill="black", font=font)

    # Combine the original image with the bubble
    return Image.alpha_composite(image.convert("RGBA"), bubble_pil)

# Function to wrap text to fit within a certain width
def wrap_text(text, font, max_width):
    words = text.split()
    wrapped_lines = []
    current_line = []

    for word in words:
        # Test if adding this word would exceed the width
        test_line = ' '.join(current_line + [word])
        try:
            # Use getbbox or getlength instead of deprecated getsize
            try:
                bbox = font.getbbox(test_line)
                line_width = bbox[2] - bbox[0]  # right - left
            except AttributeError:
                try:
                    line_width = font.getlength(test_line)
                except AttributeError:
                    line_width = font.getsize(test_line)[0]  # Fallback to deprecated method
        except:
            # If all methods fail, estimate based on character count
            line_width = len(test_line) * 10

        if line_width <= max_width:
            current_line.append(word)
        else:
            wrapped_lines.append(' '.join(current_line))
            current_line = [word]

    # Add the last line
    if current_line:
        wrapped_lines.append(' '.join(current_line))

    return wrapped_lines

# NEW FEATURE: Predefined comic templates
def get_template_data(template_name):
    """Return predefined prompts and dialogue for various comic templates"""
    templates = {
        "batman": {
            "prompts": [
                "Batman standing on a dark Gotham rooftop, his cape flowing in the wind, the Bat-Signal glowing in the sky behind him.",
                "Close-up shot of Batman's determined face, shadows partially covering his mask, eyes glowing in the darkness.",
                "Batman leaping from a rooftop, his cape spread like wings as he descends towards a group of criminals.",
                "Batman delivering a powerful punch to a thug, the action captured in a comic-style motion blur."
            ],
            "dialogues": [
                "Gotham never sleeps, and neither do I.",
                "Justice isn't a choice. It's a responsibility.",
                "Fear is a tool. I use it against them.",
                "Crime has no place in my city."
            ],
            "positions": [(250, 70), (300, 50), (220, 120), (280, 140)]
        },
        "spiderman": {
            "prompts": [
                "Spider-Man perched on the edge of a skyscraper, overlooking New York City, his iconic red and blue suit vibrant against the skyline.",
                "Spider-Man shooting webs from his wrists, swinging between buildings with great agility.",
                "Spider-Man facing off against a villain, in a dynamic action pose, ready to fight.",
                "Spider-Man helping a civilian, showcasing his friendly neighborhood hero nature."
            ],
            "dialogues": [
                "With great power comes great responsibility.",
                "Just your friendly neighborhood Spider-Man!",
                "My spider-sense is tingling!",
                "I'm not going to let anyone get hurt on my watch."
            ],
            "positions": [(250, 70), (280, 80), (240, 110), (260, 90)]
        },
        "wonderwoman": {
            "prompts": [
                "Wonder Woman standing heroically, her golden lasso and bracelets gleaming, an island paradise visible in the background.",
                "Wonder Woman deflecting bullets with her bracelets, her expression determined and fierce.",
                "Wonder Woman wielding her sword and shield, engaged in an epic battle against mythological creatures.",
                "Wonder Woman using her lasso of truth, the golden rope glowing with magical energy."
            ],
            "dialogues": [
                "I am Diana of Themyscira, daughter of Hippolyta.",
                "I fight for those who cannot fight for themselves.",
                "Peace is a virtue to be shown, not peace enforced.",
                "The truth will set you free."
            ],
            "positions": [(250, 70), (270, 60), (230, 100), (260, 80)]
        }
    }

    return templates.get(template_name.lower(), None)

# Function to generate comic using a template
def generate_template_comic(template_name, num_panels=4):
    template_data = get_template_data(template_name.lower())

    if not template_data:
        print(f"Template '{template_name}' not found. Available templates: {', '.join(get_available_templates())}")
        return None, None, None, None

    try:
        # Load SDXL model (templates work better with SDXL)
        pipe = load_sdxl_model()

        print(f"\nGenerating {template_name} comic panels...")
        panel_images = []

        # Generate a base character image for consistency
        print("Generating base character image...")
        base_prompt = f"A detailed illustration of {template_name} in a dynamic pose, full body visible"

        # Check if using SDXL or regular SD pipeline
        is_sdxl = isinstance(pipe, StableDiffusionXLPipeline)

        if is_sdxl:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                num_inference_steps=30
            ).images[0]
        else:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                num_inference_steps=30,
                guidance_scale=7.5
            ).images[0]

        # Resize base image for consistency
        base_image = base_image.resize((768, 768))

        # Get the prompts, dialogues, and positions from template data
        prompts = template_data["prompts"][:num_panels]
        dialogues = template_data["dialogues"][:num_panels]
        positions = template_data["positions"][:num_panels]

        # Fill with defaults if not enough data
        while len(prompts) < num_panels:
            prompts.append(f"{template_name} in an action scene")
        while len(dialogues) < num_panels:
            dialogues.append("")
        while len(positions) < num_panels:
            positions.append((256, 100))

        # Generate each panel
        for i, prompt in enumerate(prompts):
            print(f"Generating image for panel {i+1}...")

            if is_sdxl:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.65,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                    num_inference_steps=30
                ).images[0]
            else:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.65,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                    num_inference_steps=30,
                    guidance_scale=7.5
                ).images[0]

            # Resize to standard panel size
            image = image.resize((512, 512))

            # Add speech bubble if there's dialogue
            if dialogues[i]:
                image = add_speech_bubble(image, dialogues[i], positions[i])

            panel_images.append(image)

            # Save the individual panel
            image_path = f"panel_{i+1}.png"
            image.save(image_path)

        # Create a combined comic strip
        print("Creating combined comic strip...")
        rows = int(np.ceil(num_panels / 2))
        columns = min(2, num_panels)

        final_width = 512 * columns
        final_height = 512 * rows

        comic_strip = Image.new("RGB", (final_width, final_height), (255, 255, 255))

        for i, image in enumerate(panel_images):
            x_offset = (i % 2) * 512
            y_offset = (i // 2) * 512
            comic_strip.paste(image, (x_offset, y_offset))

        # Save the combined comic strip
        comic_strip.save(f"{template_name}_comic_strip.png")
        print(f"{template_name} comic generation complete!")

        # Create a dummy story text for consistency with the generate_comic function
        story_text = f"A {template_name} adventure comic generated using a template."
        story_panels = prompts

        return story_text, story_panels, panel_images, comic_strip

    except Exception as e:
        print(f"Error generating template comic: {e}")
        return None, None, None, None

# Function to get available templates
def get_available_templates():
    return ["Batman", "Spiderman", "WonderWoman"]

# Function to generate comic panels from story prompt using our fine-tuned model
def generate_comic(story_prompt, num_panels=4, model_path=None, art_style="cartoon style", use_template=None):
    # Check if using a template
    if use_template and use_template.lower() != "none":
        return generate_template_comic(use_template, num_panels)

    # Original story-based comic generation
    # Make sure the story prompt is compelling
    if len(story_prompt) < 10:
        story_prompt = f"Create an interesting story about {story_prompt}"

    try:
        # Load the fine-tuned model
        print("Loading fine-tuned model...")
        model, tokenizer = load_fine_tuned_model(model_path)

        if model is None or tokenizer is None:
            raise Exception("Failed to load the fine-tuned model")

        # Generate story from prompt with multiple attempts if needed
        print(f"Generating story from prompt: '{story_prompt}'")
        story_text = generate_story(model, tokenizer, story_prompt)
        print("Story generated:")
        print(story_text)

        # Check if story generated properly
        if not story_text or len(story_text) < 50:
            print("Story generation produced insufficient content. Trying one more time with different approach.")
            # Try a more direct approach with the model
            direct_prompt = f"Write a simple children's story about {story_prompt}. Use short sentences."

            story_text = generate_story(model, tokenizer, direct_prompt, max_length=500)

            if len(story_text) < 50:
                print("Still unable to generate a good story. Using the structured children's story.")
                character_name = extract_character_name(story_prompt) or "the character"
                topic = extract_topic(story_prompt) or "having an adventure"
                story_text = create_structured_childrens_story(character_name, topic)
    except Exception as e:
        print(f"Error generating story: {e}")
        # Provide a fallback story using our improved fallback generator
        character_name = extract_character_name(story_prompt) or "the character"
        topic = extract_topic(story_prompt) or "having an adventure"
        story_text = create_structured_childrens_story(character_name, topic)

    # Split story into panels and extract dialogues
    print("\nSplitting story into panels...")
    story_panels, panel_texts, dialogues = split_story_into_panels(story_text, num_panels)
    for i, panel in enumerate(story_panels):
        print(f"Panel {i+1}: {panel}")
        if dialogues[i]:
            print(f"Dialogue for panel {i+1}: \"{dialogues[i]}\"")

    try:
        # Load our custom InkoloRA model instead of SDXL
        pipe = load_custom_model()

        print("\nGenerating panel images...")
        panel_images = []

        # Extract character name for consistency
        character_name = extract_character_name(story_prompt) or "the character"

        # Generate a base character image for consistency across panels
        print("Generating base character image for consistency...")
        base_prompt = f"A friendly {art_style} illustration of {character_name}, whole body visible, simple background, bright colors, child-friendly"

        # Check if using StableDiffusionXLPipeline or StableDiffusionPipeline (custom model may use different API)
        is_sdxl = isinstance(pipe, StableDiffusionXLPipeline)

        if is_sdxl:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                num_inference_steps=30
            ).images[0]
        else:
            # For regular SD pipeline
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                num_inference_steps=30,
                guidance_scale=7.5
            ).images[0]

        # Resize base image for consistency
        base_image = base_image.resize((768, 768))

        # Generate each panel
        for i, panel_text in enumerate(panel_texts):
            print(f"Generating image for panel {i+1}...")
            # Create a prompt that helps the model understand it's a comic panel for children
            prompt = f"A friendly {art_style} illustration for a children's storybook showing: {panel_text}, featuring {character_name}, bright colors, simple shapes, cute characters, child-friendly"

            # Generate the image using the base character for consistency
            if is_sdxl:
                image = pipe(
                    prompt=prompt,
                    image=base_image,  # Use base image for character consistency
                    strength=0.7,      # Keep some elements of base image but allow for new scene
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                    num_inference_steps=30
                ).images[0]
            else:
                # For regular SD pipeline
                image = pipe(
                    prompt=prompt,
                    image=base_image,  # Use base image for character consistency
                    strength=0.7,      # Keep some elements of base image but allow for new scene
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                    num_inference_steps=30,
                    guidance_scale=7.5
                ).images[0]

            # Resize to standard panel size
            image = image.resize((512, 512))

            # Add speech bubble if there's dialogue
            if dialogues[i]:
                # Calculate a position for the speech bubble based on panel number
                position = (256, 100 + (i % 2) * 50)  # Vary position slightly between panels
                image = add_speech_bubble(image, dialogues[i], position)

            panel_images.append(image)

            # Save the individual panel
            image_path = f"panel_{i+1}.png"
            image.save(image_path)

        # Create a combined comic strip
        print("Creating combined comic strip...")
        rows = int(np.ceil(num_panels / 2))  # Calculate how many rows we need
        columns = min(2, num_panels)         # Max 2 columns

        final_width = 512 * columns
        final_height = 512 * rows

        comic_strip = Image.new("RGB", (final_width, final_height), (255, 255, 255))

        for i, image in enumerate(panel_images):
            x_offset = (i % 2) * 512
            y_offset = (i // 2) * 512
            comic_strip.paste(image, (x_offset, y_offset))

        # Save the combined comic strip
        comic_strip.save("comic_strip.png")
        print("Comic generation complete! All panels saved as PNG files and combined into comic_strip.png")

        return story_text, story_panels, panel_images, comic_strip

    except Exception as e:
        print(f"Error in image generation: {e}")
        print("Unable to generate images. Please check if you have enough GPU memory and proper installations.")
        return story_text, story_panels, None, None

# Web UI using Gradio
def create_ui():
    with gr.Blocks() as app:
        gr.Markdown("# Story to Comic Generator with InkoloRA")
        gr.Markdown("Enter a prompt to generate a story and convert it into comic panels with speech bubbles using the InkoloRA model")

        with gr.Row():
            with gr.Column():
                # Add template selection dropdown
                template_dropdown = gr.Dropdown(
                    choices=["None"] + get_available_templates(),

SyntaxError: incomplete input (<ipython-input-4-298a8b64de67>, line 720)

In [ ]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import cv2
from transformers import AutoTokenizer, AutoModelForCausalLM
from diffusers import StableDiffusionPipeline, StableDiffusionXLPipeline
from PIL import Image, ImageDraw, ImageFont
import gradio as gr
from google.colab import drive


# Load the fine-tuned model
def load_fine_tuned_model(model_path=None):
    # If no model path provided, use default path
    if not model_path:
        model_path = '/content/drive/MyDrive/fine_tuned_tiny_stories_model'

    try:
        print(f"Loading fine-tuned model from {model_path}...")
        model = AutoModelForCausalLM.from_pretrained(model_path)
        tokenizer = AutoTokenizer.from_pretrained(model_path)

        # Move model to GPU if available
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)

        print(f"Model loaded successfully and moved to {device}")
        return model, tokenizer
    except Exception as e:
        print(f"Error loading model: {e}")
        return None, None

# Helper functions for story generation
def extract_character_name(prompt):
    """Extract potential character names from the prompt"""
    # Common superhero/character names to check for
    character_names = ["Spider-Man", "Peter Parker", "Batman", "Superman", "Wonder Woman",
                       "Iron Man", "Captain America", "Thor", "Hulk", "Black Widow"]

    # Check if any known character names are in the prompt
    for name in character_names:
        if name.lower() in prompt.lower():
            return name

    # Otherwise extract words that might be names (capitalized words)
    words = prompt.split()
    for word in words:
        if word[0].isupper() and len(word) > 1 and word.lower() not in ["write", "story", "about"]:
            return word

    return None

def extract_topic(prompt):
    """Extract the main topic from the prompt"""
    # Remove common instruction words
    cleaned = prompt.lower().replace("write", "").replace("story", "").replace("about", "")

    # Look for interesting keywords
    keywords = ["adventure", "flying", "jumping", "fighting", "learning", "playing", "making friends"]

    for keyword in keywords:
        if keyword in cleaned:
            return keyword

    # Return the cleaned prompt if no specific keyword found
    return cleaned.strip()

def create_structured_childrens_story(character="the character", topic="having an adventure"):
    """Create a structured children's story with simple language and short sentences"""

    # Clean up inputs
    if not character or len(character) < 2:
        character = "the little hero"

    # Create a story with the structure of the Peter Parker example
    story = f"""The story of {character} began in the park. {character} was a little child who loved to play with friends.

One sunny day, a friend came over and said, "{character}, you must come and play with me! I'm the best at {topic}!"

{character} was so excited. {character} jumped up and down with happiness. They all played together and had lots of fun.

{character} learned how to be good at {topic}. Soon, everyone wanted to play with {character}.

{character} and the friends became the best of friends. They played every day after school.

{character} was so happy to have such good friends. They would be friends forever and have many more adventures together.
"""
    return story

# Enhanced story generation function using fine-tuned model
def generate_story(model, tokenizer, prompt, max_length=400):
    # Format prompt in a way that's more suitable for children's stories
    story_prompt = f"Write a simple children's story about {prompt}. Use simple language and short sentences."

    try:
        # Encode the prompt
        input_ids = tokenizer.encode(story_prompt, return_tensors='pt')

        # Move to the same device as the model
        device = next(model.parameters()).device
        input_ids = input_ids.to(device)

        # Generate text with different parameters more suited to simple children's stories
        output = model.generate(
            input_ids,
            max_length=max_length,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            top_k=40,
            top_p=0.9,
            temperature=0.7,  # Lower temperature for more consistent, simpler text
            do_sample=True
        )

        # Decode the generated text
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

        # Check quality of response
        if len(generated_text) < 50 or "In a world where anything was possible" in generated_text:
            print("Warning: Low quality story generated. Generating a structured children's story...")

            # Create a more structured children's story instead of using the default template
            character_name = extract_character_name(prompt) or "the character"
            topic = extract_topic(prompt) or "having an adventure"

            return create_structured_childrens_story(character_name, topic)

        return generated_text
    except Exception as e:
        print(f"Error in story generation: {e}")
        return create_structured_childrens_story(extract_character_name(prompt), extract_topic(prompt))

# Function to split the story into panels and also extract dialogues for speech bubbles
def split_story_into_panels(story, num_panels=4):
    # First check if we have enough content to split
    if len(story) < 100:
        print("Warning: Story is very short. Panel division may not be optimal.")

    # Try to split by paragraphs first for more natural divisions
    paragraphs = story.split('\n')
    paragraphs = [p for p in paragraphs if p.strip()]  # Remove empty paragraphs

    # Extract dialogues for speech bubbles
    dialogues = []
    panel_texts = []

    if len(paragraphs) >= num_panels:
        # We have enough paragraphs, so use them as natural divisions
        if len(paragraphs) > num_panels:
            # Combine paragraphs to match the desired number of panels
            step = len(paragraphs) / num_panels
            panels = []
            for i in range(num_panels):
                start_idx = int(i * step)
                end_idx = int((i + 1) * step) if i < num_panels - 1 else len(paragraphs)
                panel_text = " ".join(paragraphs[start_idx:end_idx])
                panels.append(panel_text)

                # Extract dialogue (text within quotes) for speech bubbles
                dialogue = extract_dialogue(panel_text)
                dialogues.append(dialogue)

                # Remove dialogue from panel text for better image prompts
                clean_text = panel_text.replace(dialogue, "") if dialogue else panel_text
                panel_texts.append(clean_text)
            return panels, panel_texts, dialogues
        else:
            # Perfect match or fewer paragraphs than panels
            panels = paragraphs[:num_panels]
            for panel in panels:
                dialogue = extract_dialogue(panel)
                dialogues.append(dialogue)
                clean_text = panel.replace(dialogue, "") if dialogue else panel
                panel_texts.append(clean_text)
            return panels, panel_texts, dialogues

    # If we don't have enough paragraphs, fall back to sentence splitting
    sentences = []
    # Split by common sentence endings
    for chunk in story.split('. '):
        sentences.extend([s.strip() + '.' for s in chunk.split('! ')])
    sentences = [s for s in sentences if len(s) > 1]  # Remove empty or too-short sentences

    # Handle edge case where there are fewer sentences than panels
    if len(sentences) < num_panels:
        # Create more sentences by splitting the longest ones
        while len(sentences) < num_panels:
            longest_idx = max(range(len(sentences)), key=lambda i: len(sentences[i]))
            longest = sentences[longest_idx]

            # Split at a comma or other natural break point if possible
            if ', ' in longest:
                parts = longest.split(', ', 1)
                sentences[longest_idx] = parts[0] + '.'
                sentences.insert(longest_idx + 1, parts[1])
            else:
                # If no good split point, duplicate the sentence with a variation
                sentences.append(f"Meanwhile, {longest.lower()}")

    # Calculate how many sentences per panel for a balanced distribution
    step = max(1, len(sentences) // num_panels)

    panels = []
    for i in range(num_panels):
        start_idx = i * step
        # For the last panel, include any remaining sentences
        end_idx = (i + 1) * step if i < num_panels - 1 else len(sentences)
        panel_text = " ".join(sentences[start_idx:end_idx])
        panels.append(panel_text)

        # Extract dialogue for speech bubbles
        dialogue = extract_dialogue(panel_text)
        dialogues.append(dialogue)

        # Remove dialogue from panel text for better image prompts
        clean_text = panel_text.replace(dialogue, "") if dialogue else panel_text
        panel_texts.append(clean_text)

    return panels, panel_texts, dialogues

# Function to extract dialogue from text (text within quotes)
def extract_dialogue(text):
    import re
    dialogue_matches = re.findall(r'"([^"]*)"', text)
    if dialogue_matches:
        # Return the first dialogue found
        return dialogue_matches[0]
    return ""

# Load the custom InkoloRA model
def load_custom_model():
    print("Loading InkoloRA model - this may take a while...")

    # Define custom model path
    custom_model_path = "/content/drive/MyDrive/InkoloRA.safetensors"

    # Check if the model file exists
    if not os.path.exists(custom_model_path):
        print(f"Warning: Custom model not found at {custom_model_path}. Falling back to Stable Diffusion XL.")
        return load_sdxl_model()

    try:
        # Load the model - adjust the model_id based on what base model your safetensors file is trained on
        # Assuming it's SD 1.5 based; modify if it's based on a different architecture
        pipe = StableDiffusionPipeline.from_pretrained(
            "runwayml/stable-diffusion-v1-5",
            torch_dtype=torch.float16,
            safety_checker=None
        )

        # Load the custom weights
        pipe.unet.load_attn_procs(custom_model_path)

        # Check if CUDA is available, otherwise use CPU
        device = "cuda" if torch.cuda.is_available() else "cpu"
        if device == "cpu":
            print("Warning: CUDA not available, using CPU. This will be very slow.")
        pipe = pipe.to(device)

        print("InkoloRA model loaded successfully!")
        return pipe
    except Exception as e:
        print(f"Error loading custom model: {e}")
        print("Falling back to Stable Diffusion XL...")
        return load_sdxl_model()

# Original SDXL model load function as fallback
def load_sdxl_model():
    print("Loading SDXL model - this may take a while...")
    pipe = StableDiffusionXLPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True
    )

    # Check if CUDA is available, otherwise use CPU
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device == "cpu":
        print("Warning: CUDA not available, using CPU. This will be very slow.")
    pipe = pipe.to(device)

    return pipe

# Function to add speech bubbles to images
def add_speech_bubble(image, text, position=None):
    if not text:
        return image

    # Convert PIL image to numpy array for OpenCV operations
    img = np.array(image.convert("RGBA"))
    height, width, _ = img.shape

    # If position not provided, place the bubble at a suitable position
    if position is None:
        position = (width // 2, height // 4)

    # Create a transparent layer for the bubble
    bubble = np.zeros((height, width, 4), dtype=np.uint8)

    # Calculate bubble size based on text length
    bubble_width = min(300, max(150, len(text) * 10))
    bubble_height = min(130, max(60, (len(text) // 20 + 1) * 30))

    # Draw speech bubble
    cv2.ellipse(bubble, position, (bubble_width, bubble_height), 0, 0, 360, (255, 255, 255, 255), -1)

    # Add a pointer to make it look like a speech bubble
    pointer_x = position[0]
    pointer_y = position[1] + bubble_height
    triangle_pts = np.array([
        [pointer_x - 20, pointer_y - 10],
        [pointer_x + 20, pointer_y - 10],
        [pointer_x, pointer_y + 30]
    ])
    cv2.fillPoly(bubble, [triangle_pts], (255, 255, 255, 255))

    bubble_pil = Image.fromarray(bubble)

    # Draw text
    draw = ImageDraw.Draw(bubble_pil)

    # Try to load a font, fall back to default if not available
    try:
        font = ImageFont.truetype("arial.ttf", 24)
    except:
        try:
            font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 24)
        except:
            font = ImageFont.load_default()

    # Wrap text to fit in the bubble
    wrapped_text = wrap_text(text, font, bubble_width - 40)

    # Calculate text position for centering
    text_x = position[0] - (bubble_width // 2) + 20
    text_y = position[1] - (bubble_height // 2) + 10

    # Draw the wrapped text
    for i, line in enumerate(wrapped_text):
        draw.text((text_x, text_y + i * 30), line, fill="black", font=font)

    # Combine the original image with the bubble
    return Image.alpha_composite(image.convert("RGBA"), bubble_pil)

# Function to wrap text to fit within a certain width
def wrap_text(text, font, max_width):
    words = text.split()
    wrapped_lines = []
    current_line = []

    for word in words:
        # Test if adding this word would exceed the width
        test_line = ' '.join(current_line + [word])
        try:
            # Use getbbox or getlength instead of deprecated getsize
            try:
                bbox = font.getbbox(test_line)
                line_width = bbox[2] - bbox[0]  # right - left
            except AttributeError:
                try:
                    line_width = font.getlength(test_line)
                except AttributeError:
                    line_width = font.getsize(test_line)[0]  # Fallback to deprecated method
        except:
            # If all methods fail, estimate based on character count
            line_width = len(test_line) * 10

        if line_width <= max_width:
            current_line.append(word)
        else:
            wrapped_lines.append(' '.join(current_line))
            current_line = [word]

    # Add the last line
    if current_line:
        wrapped_lines.append(' '.join(current_line))

    return wrapped_lines

# NEW FEATURE: Predefined comic templates
def get_template_data(template_name):
    """Return predefined prompts and dialogue for various comic templates"""
    templates = {
        "batman": {
            "prompts": [
                "Batman standing on a dark Gotham rooftop, his cape flowing in the wind, the Bat-Signal glowing in the sky behind him.",
                "Close-up shot of Batman's determined face, shadows partially covering his mask, eyes glowing in the darkness.",
                "Batman leaping from a rooftop, his cape spread like wings as he descends towards a group of criminals.",
                "Batman delivering a powerful punch to a thug, the action captured in a comic-style motion blur."
            ],
            "dialogues": [
                "Gotham never sleeps, and neither do I.",
                "Justice isn't a choice. It's a responsibility.",
                "Fear is a tool. I use it against them.",
                "Crime has no place in my city."
            ],
            "positions": [(250, 70), (300, 50), (220, 120), (280, 140)]
        },
        "spiderman": {
            "prompts": [
                "Spider-Man perched on the edge of a skyscraper, overlooking New York City, his iconic red and blue suit vibrant against the skyline.",
                "Spider-Man shooting webs from his wrists, swinging between buildings with great agility.",
                "Spider-Man facing off against a villain, in a dynamic action pose, ready to fight.",
                "Spider-Man helping a civilian, showcasing his friendly neighborhood hero nature."
            ],
            "dialogues": [
                "With great power comes great responsibility.",
                "Just your friendly neighborhood Spider-Man!",
                "My spider-sense is tingling!",
                "I'm not going to let anyone get hurt on my watch."
            ],
            "positions": [(250, 70), (280, 80), (240, 110), (260, 90)]
        },
        "wonderwoman": {
            "prompts": [
                "Wonder Woman standing heroically, her golden lasso and bracelets gleaming, an island paradise visible in the background.",
                "Wonder Woman deflecting bullets with her bracelets, her expression determined and fierce.",
                "Wonder Woman wielding her sword and shield, engaged in an epic battle against mythological creatures.",
                "Wonder Woman using her lasso of truth, the golden rope glowing with magical energy."
            ],
            "dialogues": [
                "I am Diana of Themyscira, daughter of Hippolyta.",
                "I fight for those who cannot fight for themselves.",
                "Peace is a virtue to be shown, not peace enforced.",
                "The truth will set you free."
            ],
            "positions": [(250, 70), (270, 60), (230, 100), (260, 80)]
        }
    }

    return templates.get(template_name.lower(), None)

# Function to generate comic using a template
def generate_template_comic(template_name, num_panels=4):
    template_data = get_template_data(template_name.lower())

    if not template_data:
        print(f"Template '{template_name}' not found. Available templates: {', '.join(get_available_templates())}")
        return None, None, None, None

    try:
        # Load SDXL model (templates work better with SDXL)
        pipe = load_sdxl_model()

        print(f"\nGenerating {template_name} comic panels...")
        panel_images = []

        # Generate a base character image for consistency
        print("Generating base character image...")
        base_prompt = f"A detailed illustration of {template_name} in a dynamic pose, full body visible"

        # Check if using SDXL or regular SD pipeline
        is_sdxl = isinstance(pipe, StableDiffusionXLPipeline)

        if is_sdxl:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                num_inference_steps=30
            ).images[0]
        else:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                num_inference_steps=30,
                guidance_scale=7.5
            ).images[0]

        # Resize base image for consistency
        base_image = base_image.resize((768, 768))

        # Get the prompts, dialogues, and positions from template data
        prompts = template_data["prompts"][:num_panels]
        dialogues = template_data["dialogues"][:num_panels]
        positions = template_data["positions"][:num_panels]

        # Fill with defaults if not enough data
        while len(prompts) < num_panels:
            prompts.append(f"{template_name} in an action scene")
        while len(dialogues) < num_panels:
            dialogues.append("")
        while len(positions) < num_panels:
            positions.append((256, 100))

        # Generate each panel
        for i, prompt in enumerate(prompts):
            print(f"Generating image for panel {i+1}...")

            if is_sdxl:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.65,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                    num_inference_steps=30
                ).images[0]
            else:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.65,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                    num_inference_steps=30,
                    guidance_scale=7.5
                ).images[0]

            # Resize to standard panel size
            image = image.resize((512, 512))

            # Add speech bubble if there's dialogue
            if dialogues[i]:
                image = add_speech_bubble(image, dialogues[i], positions[i])

            panel_images.append(image)

            # Save the individual panel
            image_path = f"panel_{i+1}.png"
            image.save(image_path)

        # Create a combined comic strip
        print("Creating combined comic strip...")
        rows = int(np.ceil(num_panels / 2))
        columns = min(2, num_panels)

        final_width = 512 * columns
        final_height = 512 * rows

        comic_strip = Image.new("RGB", (final_width, final_height), (255, 255, 255))

        for i, image in enumerate(panel_images):
            x_offset = (i % 2) * 512
            y_offset = (i // 2) * 512
            comic_strip.paste(image, (x_offset, y_offset))

        # Save the combined comic strip
        comic_strip.save(f"{template_name}_comic_strip.png")
        print(f"{template_name} comic generation complete!")

        # Create a dummy story text for consistency with the generate_comic function
        story_text = f"A {template_name} adventure comic generated using a template."
        story_panels = prompts

        return story_text, story_panels, panel_images, comic_strip

    except Exception as e:
        print(f"Error generating template comic: {e}")
        return None, None, None, None

# Function to get available templates
def get_available_templates():
    return ["Batman", "Spiderman", "WonderWoman"]

# Function to generate comic panels from story prompt using our fine-tuned model
def generate_comic(story_prompt, num_panels=4, model_path=None, art_style="cartoon style", use_template=None):
    # Check if using a template
    if use_template and use_template.lower() != "none":
        return generate_template_comic(use_template, num_panels)

    # Original story-based comic generation
    # Make sure the story prompt is compelling
    if len(story_prompt) < 10:
        story_prompt = f"Create an interesting story about {story_prompt}"

    try:
        # Load the fine-tuned model
        print("Loading fine-tuned model...")
        model, tokenizer = load_fine_tuned_model(model_path)

        if model is None or tokenizer is None:
            raise Exception("Failed to load the fine-tuned model")

        # Generate story from prompt with multiple attempts if needed
        print(f"Generating story from prompt: '{story_prompt}'")
        story_text = generate_story(model, tokenizer, story_prompt)
        print("Story generated:")
        print(story_text)

        # Check if story generated properly
        if not story_text or len(story_text) < 50:
            print("Story generation produced insufficient content. Trying one more time with different approach.")
            # Try a more direct approach with the model
            direct_prompt = f"Write a simple children's story about {story_prompt}. Use short sentences."

            story_text = generate_story(model, tokenizer, direct_prompt, max_length=500)

            if len(story_text) < 50:
                print("Still unable to generate a good story. Using the structured children's story.")
                character_name = extract_character_name(story_prompt) or "the character"
                topic = extract_topic(story_prompt) or "having an adventure"
                story_text = create_structured_childrens_story(character_name, topic)
    except Exception as e:
        print(f"Error generating story: {e}")
        # Provide a fallback story using our improved fallback generator
        character_name = extract_character_name(story_prompt) or "the character"
        topic = extract_topic(story_prompt) or "having an adventure"
        story_text = create_structured_childrens_story(character_name, topic)

    # Split story into panels and extract dialogues
    print("\nSplitting story into panels...")
    story_panels, panel_texts, dialogues = split_story_into_panels(story_text, num_panels)
    for i, panel in enumerate(story_panels):
        print(f"Panel {i+1}: {panel}")
        if dialogues[i]:
            print(f"Dialogue for panel {i+1}: \"{dialogues[i]}\"")

    try:
        # Load our custom InkoloRA model instead of SDXL
        pipe = load_custom_model()

        print("\nGenerating panel images...")
        panel_images = []

        # Extract character name for consistency
        character_name = extract_character_name(story_prompt) or "the character"

        # Generate a base character image for consistency across panels
        print("Generating base character image for consistency...")
        base_prompt = f"A friendly {art_style} illustration of {character_name}, whole body visible, simple background, bright colors, child-friendly"

        # Check if using StableDiffusionXLPipeline or StableDiffusionPipeline (custom model may use different API)
        is_sdxl = isinstance(pipe, StableDiffusionXLPipeline)

        if is_sdxl:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                num_inference_steps=30
            ).images[0]
        else:
            # For regular SD pipeline
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                num_inference_steps=30,
                guidance_scale=7.5
            ).images[0]

        # Resize base image for consistency
        base_image = base_image.resize((768, 768))

        # Generate each panel
        for i, panel_text in enumerate(panel_texts):
            print(f"Generating image for panel {i+1}...")
            # Create a prompt that helps the model understand it's a comic panel for children
            prompt = f"A friendly {art_style} illustration for a children's storybook showing: {panel_text}, featuring {character_name}, bright colors, simple shapes, cute characters, child-friendly"

            # Generate the image using the base character for consistency
            if is_sdxl:
                image = pipe(
                    prompt=prompt,
                    image=base_image,  # Use base image for character consistency
                    strength=0.7,      # Keep some elements of base image but allow for new scene
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                    num_inference_steps=30
                ).images[0]
            else:
                # For regular SD pipeline
                image = pipe(
                    prompt=prompt,
                    image=base_image,  # Use base image for character consistency
                    strength=0.7,      # Keep some elements of base image but allow for new scene
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                    num_inference_steps=30,
                    guidance_scale=7.5
                ).images[0]

            # Resize to standard panel size
            image = image.resize((512, 512))

            # Add speech bubble if there's dialogue
            if dialogues[i]:
                # Calculate a position for the speech bubble based on panel number
                position = (256, 100 + (i % 2) * 50)  # Vary position slightly between panels
                image = add_speech_bubble(image, dialogues[i], position)

            panel_images.append(image)

            # Save the individual panel
            image_path = f"panel_{i+1}.png"
            image.save(image_path)

        # Create a combined comic strip
        print("Creating combined comic strip...")
        rows = int(np.ceil(num_panels / 2))  # Calculate how many rows we need
        columns = min(2, num_panels)         # Max 2 columns

        final_width = 512 * columns
        final_height = 512 * rows

        comic_strip = Image.new("RGB", (final_width, final_height), (255, 255, 255))

        for i, image in enumerate(panel_images):
            x_offset = (i % 2) * 512
            y_offset = (i // 2) * 512
            comic_strip.paste(image, (x_offset, y_offset))

        # Save the combined comic strip
        comic_strip.save("comic_strip.png")
        print("Comic generation complete! All panels saved as PNG files and combined into comic_strip.png")

        return story_text, story_panels, panel_images, comic_strip

    except Exception as e:
        print(f"Error in image generation: {e}")
        print("Unable to generate images. Please check if you have enough GPU memory and proper installations.")
        return story_text, story_panels, None, None

# Web UI using Gradio
def create_ui():
    with gr.Blocks() as app:
        gr.Markdown("# Story to Comic Generator with InkoloRA")
        gr.Markdown("Enter a prompt to generate a story and convert it into comic panels with speech bubbles using the InkoloRA model")

        with gr.Row():
            with gr.Column():
                # Add template selection dropdown
                template_dropdown = gr.Dropdown(
                    choices=["None"] + get_available_templates(),
                    label="Choose Template (optional)",
                    value="None"
                )

                story_prompt = gr.Textbox(
                    label="Story Prompt",
                    placeholder="Enter a prompt for the children's story (e.g. 'a little boy who learns to fly')",
                    lines=2
                )

                num_panels = gr.Slider(
                    label="Number of Panels",
                    minimum=2,
                    maximum=8,
                    value=4,
                    step=1
                )

                art_style = gr.Dropdown(
                                        choices=["Cartoon Style", "Realistic", "Anime", "Sketch"],
                    label="Art Style",
                    value="Cartoon Style"
                )

                generate_button = gr.Button("Generate Comic")

            with gr.Column():
                output_text = gr.Textbox(
                    label="Generated Story",
                    placeholder="The generated story will appear here...",
                    lines=10,
                    interactive=False
                )

                output_panels = gr.Gallery(
                    label="Comic Panels",
                    show_label=True,
                    elem_id="comic-panels",
                    height=200
                ).style(grid=2)

                output_comic = gr.Image(
                    label="Combined Comic Strip",
                    type="pil"
                )

        # Define the action for the button click
        def generate_comic_action(story_prompt, num_panels, art_style, use_template):
            story_text, story_panels, panel_images, comic_strip = generate_comic(
                story_prompt, num_panels, art_style=art_style, use_template=use_template
            )
            output_text.update(story_text)
            output_panels.update(panel_images)
            output_comic.update(comic_strip)
            return

        generate_button.click(
            generate_comic_action,
            inputs=[story_prompt, num_panels, art_style, template_dropdown],
            outputs=[output_text, output_panels, output_comic]
        )

    app.launch()

# Call the function to create the UI
create_ui()

AttributeError: 'Gallery' object has no attribute 'style'

In [ ]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import cv2
from transformers import AutoTokenizer, AutoModelForCausalLM
from diffusers import StableDiffusionPipeline, StableDiffusionXLPipeline
from PIL import Image, ImageDraw, ImageFont
import gradio as gr
from google.colab import drive

# Load the fine-tuned model
def load_fine_tuned_model(model_path=None):
    if not model_path:
        model_path = '/content/drive/MyDrive/fine_tuned_tiny_stories_model'

    try:
        print(f"Loading fine-tuned model from {model_path}...")
        model = AutoModelForCausalLM.from_pretrained(model_path)
        tokenizer = AutoTokenizer.from_pretrained(model_path)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)

        print(f"Model loaded successfully and moved to {device}")
        return model, tokenizer
    except Exception as e:
        print(f"Error loading model: {e}")
        return None, None

# Helper functions for story generation
def extract_character_name(prompt):
    character_names = ["Spider-Man", "Peter Parker", "Batman", "Superman", "Wonder Woman",
                       "Iron Man", "Captain America", "Thor", "Hulk", "Black Widow"]

    for name in character_names:
        if name.lower() in prompt.lower():
            return name

    words = prompt.split()
    for word in words:
        if word[0].isupper() and len(word) > 1 and word.lower() not in ["write", "story", "about"]:
            return word

    return None

def extract_topic(prompt):
    cleaned = prompt.lower().replace("write", "").replace("story", "").replace("about", "")
    keywords = ["adventure", "flying", "jumping", "fighting", "learning", "playing", "making friends"]

    for keyword in keywords:
        if keyword in cleaned:
            return keyword

    return cleaned.strip()

def create_structured_childrens_story(character="the character", topic="having an adventure"):
    if not character or len(character) < 2:
        character = "the little hero"

    story = f"""The story of {character} began in the park. {character} was a little child who loved to play with friends.

One sunny day, a friend came over and said, "{character}, you must come and play with me! I'm the best at {topic}!"

{character} was so excited. {character} jumped up and down with happiness. They all played together and had lots of fun.

{character} learned how to be good at {topic}. Soon, everyone wanted to play with {character}.

{character} and the friends became the best of friends. They played every day after school.

{character} was so happy to have such good friends. They would be friends forever and have many more adventures together.
"""
    return story

def generate_story(model, tokenizer, prompt, max_length=400):
    story_prompt = f"Write a simple children's story about {prompt}. Use simple language and short sentences."

    try:
        input_ids = tokenizer.encode(story_prompt, return_tensors='pt')
        device = next(model.parameters()).device
        input_ids = input_ids.to(device)

        output = model.generate(
            input_ids,
            max_length=max_length,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            top_k=40,
            top_p=0.9,
            temperature=0.7,
            do_sample=True
        )

        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

        if len(generated_text) < 50 or "In a world where anything was possible" in generated_text:
            print("Warning: Low quality story generated. Generating a structured children's story...")
            character_name = extract_character_name(prompt) or "the character"
            topic = extract_topic(prompt) or "having an adventure"
            return create_structured_childrens_story(character_name, topic)

        return generated_text
    except Exception as e:
        print(f"Error in story generation: {e}")
        return create_structured_childrens_story(extract_character_name(prompt), extract_topic(prompt))

def split_story_into_panels(story, num_panels=4):
    if len(story) < 100:
        print("Warning: Story is very short. Panel division may not be optimal.")

    paragraphs = story.split('\n')
    paragraphs = [p for p in paragraphs if p.strip()]

    dialogues = []
    panel_texts = []

    if len(paragraphs) >= num_panels:
        if len(paragraphs) > num_panels:
            step = len(paragraphs) / num_panels
            panels = []
            for i in range(num_panels):
                start_idx = int(i * step)
                end_idx = int((i + 1) * step) if i < num_panels - 1 else len(paragraphs)
                panel_text = " ".join(paragraphs[start_idx:end_idx])
                panels.append(panel_text)

                dialogue = extract_dialogue(panel_text)
                dialogues.append(dialogue)

                clean_text = panel_text.replace(dialogue, "") if dialogue else panel_text
                panel_texts.append(clean_text)
            return panels, panel_texts, dialogues
        else:
            panels = paragraphs[:num_panels]
            for panel in panels:
                dialogue = extract_dialogue(panel)
                dialogues.append(dialogue)
                clean_text = panel.replace(dialogue, "") if dialogue else panel
                panel_texts.append(clean_text)
            return panels, panel_texts, dialogues

    sentences = []
    for chunk in story.split('. '):
        sentences.extend([s.strip() + '.' for s in chunk.split('! ')])
    sentences = [s for s in sentences if len(s) > 1]

    if len(sentences) < num_panels:
        while len(sentences) < num_panels:
            longest_idx = max(range(len(sentences)), key=lambda i: len(sentences[i]))
            longest = sentences[longest_idx]

            if ', ' in longest:
                parts = longest.split(', ', 1)
                sentences[longest_idx] = parts[0] + '.'
                sentences.insert(longest_idx + 1, parts[1])
            else:
                sentences.append(f"Meanwhile, {longest.lower()}")

    step = max(1, len(sentences) // num_panels)

    panels = []
    for i in range(num_panels):
        start_idx = i * step
        end_idx = (i + 1) * step if i < num_panels - 1 else len(sentences)
        panel_text = " ".join(sentences[start_idx:end_idx])
        panels.append(panel_text)

        dialogue = extract_dialogue(panel_text)
        dialogues.append(dialogue)

        clean_text = panel_text.replace(dialogue, "") if dialogue else panel_text
        panel_texts.append(clean_text)

    return panels, panel_texts, dialogues

def extract_dialogue(text):
    import re
    dialogue_matches = re.findall(r'"([^"]*)"', text)
    if dialogue_matches:
        return dialogue_matches[0]
    return ""

def load_custom_model():
    print("Loading InkoloRA model - this may take a while...")

    custom_model_path = "/content/drive/MyDrive/InkoloRA.safetensors"

    if not os.path.exists(custom_model_path):
        print(f"Warning: Custom model not found at {custom_model_path}. Falling back to Stable Diffusion XL.")
        return load_sdxl_model()

    try:
        pipe = StableDiffusionPipeline.from_pretrained(
            "runwayml/stable-diffusion-v1-5",
            torch_dtype=torch.float16,
            safety_checker=None
        )

        pipe.unet.load_attn_procs(custom_model_path)

        device = "cuda" if torch.cuda.is_available() else "cpu"
        if device == "cpu":
            print("Warning: CUDA not available, using CPU. This will be very slow.")
        pipe = pipe.to(device)

        print("InkoloRA model loaded successfully!")
        return pipe
    except Exception as e:
        print(f"Error loading custom model: {e}")
        print("Falling back to Stable Diffusion XL...")
        return load_sdxl_model()

def load_sdxl_model():
    print("Loading SDXL model - this may take a while...")
    pipe = StableDiffusionXLPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True
    )

    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device == "cpu":
        print("Warning: CUDA not available, using CPU. This will be very slow.")
    pipe = pipe.to(device)

    return pipe

def add_speech_bubble(image, text, position=None):
    if not text:
        return image

    img = np.array(image.convert("RGBA"))
    height, width, _ = img.shape

    if position is None:
        position = (width // 2, height // 4)

    bubble = np.zeros((height, width, 4), dtype=np.uint8)

    bubble_width = min(300, max(150, len(text) * 10))
    bubble_height = min(130, max(60, (len(text) // 20 + 1) * 30))

    cv2.ellipse(bubble, position, (bubble_width, bubble_height), 0, 0, 360, (255, 255, 255, 255), -1)

    pointer_x = position[0]
    pointer_y = position[1] + bubble_height
    triangle_pts = np.array([
        [pointer_x - 20, pointer_y - 10],
        [pointer_x + 20, pointer_y - 10],
        [pointer_x, pointer_y + 30]
    ])
    cv2.fillPoly(bubble, [triangle_pts], (255, 255, 255, 255))

    bubble_pil = Image.fromarray(bubble)

    draw = ImageDraw.Draw(bubble_pil)

    try:
        font = ImageFont.truetype("arial.ttf", 24)
    except:
        try:
            font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 24)
        except:
            font = ImageFont.load_default()

    wrapped_text = wrap_text(text, font, bubble_width - 40)

    text_x = position[0] - (bubble_width // 2) + 20
    text_y = position[1] - (bubble_height // 2) + 10

    for i, line in enumerate(wrapped_text):
        draw.text((text_x, text_y + i * 30), line, fill="black", font=font)

    return Image.alpha_composite(image.convert("RGBA"), bubble_pil)

def wrap_text(text, font, max_width):
    words = text.split()
    wrapped_lines = []
    current_line = []

    for word in words:
        test_line = ' '.join(current_line + [word])
        try:
            try:
                bbox = font.getbbox(test_line)
                line_width = bbox[2] - bbox[0]
            except AttributeError:
                try:
                    line_width = font.getlength(test_line)
                except AttributeError:
                    line_width = font.getsize(test_line)[0]
        except:
            line_width = len(test_line) * 10

        if line_width <= max_width:
            current_line.append(word)
        else:
            wrapped_lines.append(' '.join(current_line))
            current_line = [word]

    if current_line:
        wrapped_lines.append(' '.join(current_line))

    return wrapped_lines

def get_template_data(template_name):
    templates = {
        "batman": {
            "prompts": [
                "Batman standing on a dark Gotham rooftop, his cape flowing in the wind, the Bat-Signal glowing in the sky behind him.",
                "Close-up shot of Batman's determined face, shadows partially covering his mask, eyes glowing in the darkness.",
                "Batman leaping from a rooftop, his cape spread like wings as he descends towards a group of criminals.",
                "Batman delivering a powerful punch to a thug, the action captured in a comic-style motion blur."
            ],
            "dialogues": [
                "Gotham never sleeps, and neither do I.",
                "Justice isn't a choice. It's a responsibility.",
                "Fear is a tool. I use it against them.",
                "Crime has no place in my city."
            ],
            "positions": [(250, 70), (300, 50), (220, 120), (280, 140)]
        },
        "spiderman": {
            "prompts": [
                "Spider-Man perched on the edge of a skyscraper, overlooking New York City, his iconic red and blue suit vibrant against the skyline.",
                "Spider-Man shooting webs from his wrists, swinging between buildings with great agility.",
                "Spider-Man facing off against a villain, in a dynamic action pose, ready to fight.",
                "Spider-Man helping a civilian, showcasing his friendly neighborhood hero nature."
            ],
            "dialogues": [
                "With great power comes great responsibility.",
                "Just your friendly neighborhood Spider-Man!",
                "My spider-sense is tingling!",
                "I'm not going to let anyone get hurt on my watch."
            ],
            "positions": [(250, 70), (280, 80), (240, 110), (260, 90)]
        },
        "wonderwoman": {
            "prompts": [
                "Wonder Woman standing heroically, her golden lasso and bracelets gleaming, an island paradise visible in the background.",
                "Wonder Woman deflecting bullets with her bracelets, her expression determined and fierce.",
                "Wonder Woman wielding her sword and shield, engaged in an epic battle against mythological creatures.",
                "Wonder Woman using her lasso of truth, the golden rope glowing with magical energy."
            ],
            "dialogues": [
                "I am Diana of Themyscira, daughter of Hippolyta.",
                "I fight for those who cannot fight for themselves.",
                "Peace is a virtue to be shown, not peace enforced.",
                "The truth will set you free."
            ],
            "positions": [(250, 70), (270, 60), (230, 100), (260, 80)]
        }
    }

    return templates.get(template_name.lower(), None)

def generate_template_comic(template_name, num_panels=4):
    template_data = get_template_data(template_name.lower())

    if not template_data:
        print(f"Template '{template_name}' not found. Available templates: {', '.join(get_available_templates())}")
        return None, None, None, None

    try:
        pipe = load_sdxl_model()

        print(f"\nGenerating {template_name} comic panels...")
        panel_images = []

        print("Generating base character image...")
        base_prompt = f"A detailed illustration of {template_name} in a dynamic pose, full body visible"

        is_sdxl = isinstance(pipe, StableDiffusionXLPipeline)

        if is_sdxl:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                num_inference_steps=30
            ).images[0]
        else:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                num_inference_steps=30,
                guidance_scale=7.5
            ).images[0]

        base_image = base_image.resize((768, 768))

        prompts = template_data["prompts"][:num_panels]
        dialogues = template_data["dialogues"][:num_panels]
        positions = template_data["positions"][:num_panels]

        while len(prompts) < num_panels:
            prompts.append(f"{template_name} in an action scene")
        while len(dialogues) < num_panels:
            dialogues.append("")
        while len(positions) < num_panels:
            positions.append((256, 100))

        for i, prompt in enumerate(prompts):
            print(f"Generating image for panel {i+1}...")

            if is_sdxl:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.65,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                    num_inference_steps=30
                ).images[0]
            else:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.65,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                    num_inference_steps=30,
                    guidance_scale=7.5
                ).images[0]

            image = image.resize((512, 512))

            if dialogues[i]:
                image = add_speech_bubble(image, dialogues[i], positions[i])

            panel_images.append(image)

            image_path = f"panel_{i+1}.png"
            image.save(image_path)

        print("Creating combined comic strip...")
        rows = int(np.ceil(num_panels / 2))
        columns = min(2, num_panels)

        final_width = 512 * columns
        final_height = 512 * rows

        comic_strip = Image.new("RGB", (final_width, final_height), (255, 255, 255))

        for i, image in enumerate(panel_images):
            x_offset = (i % 2) * 512
            y_offset = (i // 2) * 512
            comic_strip.paste(image, (x_offset, y_offset))

        comic_strip.save(f"{template_name}_comic_strip.png")
        print(f"{template_name} comic generation complete!")

        story_text = f"A {template_name} adventure comic generated using a template."
        story_panels = prompts

        return story_text, story_panels, panel_images, comic_strip

    except Exception as e:
        print(f"Error generating template comic: {e}")
        return None, None, None, None

def get_available_templates():
    return ["Batman", "Spiderman", "WonderWoman"]

def generate_comic(story_prompt, num_panels=4, model_path=None, art_style="cartoon style", use_template=None):
    if use_template and use_template.lower() != "none":
        return generate_template_comic(use_template, num_panels)

    if len(story_prompt) < 10:
        story_prompt = f"Create an interesting story about {story_prompt}"

    try:
        print("Loading fine-tuned model...")
        model, tokenizer = load_fine_tuned_model(model_path)

        if model is None or tokenizer is None:
            raise Exception("Failed to load the fine-tuned model")

        print(f"Generating story from prompt: '{story_prompt}'")
        story_text = generate_story(model, tokenizer, story_prompt)
        print("Story generated:")
        print(story_text)

        if not story_text or len(story_text) < 50:
            print("Story generation produced insufficient content. Trying one more time with different approach.")
            direct_prompt = f"Write a simple children's story about {story_prompt}. Use short sentences."

            story_text = generate_story(model, tokenizer, direct_prompt, max_length=500)

            if len(story_text) < 50:
                print("Still unable to generate a good story. Using the structured children's story.")
                character_name = extract_character_name(story_prompt) or "the character"
                topic = extract_topic(story_prompt) or "having an adventure"
                story_text = create_structured_childrens_story(character_name, topic)
    except Exception as e:
        print(f"Error generating story: {e}")
        character_name = extract_character_name(story_prompt) or "the character"
        topic = extract_topic(story_prompt) or "having an adventure"
        story_text = create_structured_childrens_story(character_name, topic)

    print("\nSplitting story into panels...")
    story_panels, panel_texts, dialogues = split_story_into_panels(story_text, num_panels)
    for i, panel in enumerate(story_panels):
        print(f"Panel {i+1}: {panel}")
        if dialogues[i]:
            print(f"Dialogue for panel {i+1}: \"{dialogues[i]}\"")

    try:
        pipe = load_custom_model()

        print("\nGenerating panel images...")
        panel_images = []

        character_name = extract_character_name(story_prompt) or "the character"

        print("Generating base character image for consistency...")
        base_prompt = f"A friendly {art_style} illustration of {character_name}, whole body visible, simple background, bright colors, child-friendly"

        is_sdxl = isinstance(pipe, StableDiffusionXLPipeline)

        if is_sdxl:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                num_inference_steps=30
            ).images[0]
        else:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                num_inference_steps=30,
                guidance_scale=7.5
            ).images[0]

        base_image = base_image.resize((768, 768))

        for i, panel_text in enumerate(panel_texts):
            print(f"Generating image for panel {i+1}...")
            prompt = f"A friendly {art_style} illustration for a children's storybook showing: {panel_text}, featuring {character_name}, bright colors, simple shapes, cute characters, child-friendly"

            if is_sdxl:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.7,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                    num_inference_steps=30
                ).images[0]
            else:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.7,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                    num_inference_steps=30,
                    guidance_scale=7.5
                ).images[0]

            image = image.resize((512, 512))

            if dialogues[i]:
                position = (256, 100 + (i % 2) * 50)
                image = add_speech_bubble(image, dialogues[i], position)

            panel_images.append(image)

            image_path = f"panel_{i+1}.png"
            image.save(image_path)

        print("Creating combined comic strip...")
        rows = int(np.ceil(num_panels / 2))
        columns = min(2, num_panels)

        final_width = 512 * columns
        final_height = 512 * rows

        comic_strip = Image.new("RGB", (final_width, final_height), (255, 255, 255))

        for i, image in enumerate(panel_images):
            x_offset = (i % 2) * 512
            y_offset = (i // 2) * 512
            comic_strip.paste(image, (x_offset, y_offset))

        comic_strip.save("comic_strip.png")
        print("Comic generation complete! All panels saved as PNG files and combined into comic_strip.png")

        return story_text, story_panels, panel_images, comic_strip

    except Exception as e:
        print(f"Error in image generation: {e}")
        print("Unable to generate images. Please check if you have enough GPU memory and proper installations.")
        return story_text, story_panels, None, None

# Web UI using Gradio
def create_ui():
    with gr.Blocks() as app:
        gr.Markdown("# Story to Comic Generator with InkoloRA")
        gr.Markdown("Enter a prompt to generate a story and convert it into comic panels with speech bubbles using the InkoloRA model")

        with gr.Row():
            with gr.Column():
                template_dropdown = gr.Dropdown(
                    choices=["None"] + get_available_templates(),
                    label="Choose Template (optional)",
                    value="None"
                )

                story_prompt = gr.Textbox(
                    label="Story Prompt",
                    placeholder="Enter a prompt for the children's story (e.g. 'a little boy who learns to fly')",
                    lines=2
                )

                num_panels = gr.Slider(
                    label="Number of Panels",
                    minimum=2,
                    maximum=8,
                    value=4,
                    step=1
                )

                art_style = gr.Dropdown(
                    choices=["Cartoon Style", "Realistic", "Anime", "Sketch"],
                    label="Art Style",
                    value="Cartoon Style"
                )

                generate_button = gr.Button("Generate Comic")

            with gr.Column():
                output_text = gr.Textbox(
                    label="Generated Story",
                    placeholder="The generated story will appear here...",
                    lines=10,
                    interactive=False
                )

                output_panels = gr.Gallery(
                    label="Comic Panels",
                    show_label=True,
                    elem_id="comic-panels",
                    height=200,
                    columns=2  # Set the number of columns for the gallery
                )

                output_comic = gr.Image(
                    label="Combined Comic Strip",
                    type="pil"
                )

        def generate_comic_action(story_prompt, num_panels, art_style, use_template):
            story_text, story_panels, panel_images, comic_strip = generate_comic(
                story_prompt, num_panels, art_style=art_style, use_template=use_template
            )
            output_text.update(story_text)
            output_panels.update(panel_images)
            output_comic.update(comic_strip)
            return

        generate_button.click(
            generate_comic_action,
            inputs=[story_prompt, num_panels, art_style, template_dropdown],
            outputs=[output_text, output_panels, output_comic]
        )

    app.launch()

# Call the function to create the UI
create_ui()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0dfef397bacc03490e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import cv2
from transformers import AutoTokenizer, AutoModelForCausalLM
from diffusers import StableDiffusionPipeline, StableDiffusionXLPipeline
from PIL import Image, ImageDraw, ImageFont
import gradio as gr
from google.colab import drive

# Load the fine-tuned model
def load_fine_tuned_model(model_path=None):
    if not model_path:
        model_path = '/content/drive/MyDrive/fine_tuned_tiny_stories_model'

    try:
        print(f"Loading fine-tuned model from {model_path}...")
        model = AutoModelForCausalLM.from_pretrained(model_path)
        tokenizer = AutoTokenizer.from_pretrained(model_path)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)

        print(f"Model loaded successfully and moved to {device}")
        return model, tokenizer
    except Exception as e:
        print(f"Error loading model: {e}")
        return None, None

# Helper functions for story generation
def extract_character_name(prompt):
    character_names = ["Spider-Man", "Peter Parker", "Batman", "Superman", "Wonder Woman",
                       "Iron Man", "Captain America", "Thor", "Hulk", "Black Widow"]

    for name in character_names:
        if name.lower() in prompt.lower():
            return name

    words = prompt.split()
    for word in words:
        if word[0].isupper() and len(word) > 1 and word.lower() not in ["write", "story", "about"]:
            return word

    return None

def extract_topic(prompt):
    cleaned = prompt.lower().replace("write", "").replace("story", "").replace("about", "")
    keywords = ["adventure", "flying", "jumping", "fighting", "learning", "playing", "making friends"]

    for keyword in keywords:
        if keyword in cleaned:
            return keyword

    return cleaned.strip()

def create_structured_childrens_story(character="the character", topic="having an adventure"):
    if not character or len(character) < 2:
        character = "the little hero"

    story = f"""The story of {character} began in the park. {character} was a little child who loved to play with friends.

One sunny day, a friend came over and said, "{character}, you must come and play with me! I'm the best at {topic}!"

{character} was so excited. {character} jumped up and down with happiness. They all played together and had lots of fun.

{character} learned how to be good at {topic}. Soon, everyone wanted to play with {character}.

{character} and the friends became the best of friends. They played every day after school.

{character} was so happy to have such good friends. They would be friends forever and have many more adventures together.
"""
    return story

def generate_story(model, tokenizer, prompt, max_length=400):
    story_prompt = f"Write a simple children's story about {prompt}. Use simple language and short sentences."

    try:
        input_ids = tokenizer.encode(story_prompt, return_tensors='pt')
        device = next(model.parameters()).device
        input_ids = input_ids.to(device)

        output = model.generate(
            input_ids,
            max_length=max_length,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            top_k=40,
            top_p=0.9,
            temperature=0.7,
            do_sample=True
        )

        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

        if len(generated_text) < 50 or "In a world where anything was possible" in generated_text:
            print("Warning: Low quality story generated. Generating a structured children's story...")
            character_name = extract_character_name(prompt) or "the character"
            topic = extract_topic(prompt) or "having an adventure"
            return create_structured_childrens_story(character_name, topic)

        return generated_text
    except Exception as e:
        print(f"Error in story generation: {e}")
        return create_structured_childrens_story(extract_character_name(prompt), extract_topic(prompt))

def split_story_into_panels(story, num_panels=4):
    if len(story) < 100:
        print("Warning: Story is very short. Panel division may not be optimal.")

    paragraphs = story.split('\n')
    paragraphs = [p for p in paragraphs if p.strip()]

    dialogues = []
    panel_texts = []

    if len(paragraphs) >= num_panels:
        if len(paragraphs) > num_panels:
            step = len(paragraphs) / num_panels
            panels = []
            for i in range(num_panels):
                start_idx = int(i * step)
                end_idx = int((i + 1) * step) if i < num_panels - 1 else len(paragraphs)
                panel_text = " ".join(paragraphs[start_idx:end_idx])
                panels.append(panel_text)

                dialogue = extract_dialogue(panel_text)
                dialogues.append(dialogue)

                clean_text = panel_text.replace(dialogue, "") if dialogue else panel_text
                panel_texts.append(clean_text)
            return panels, panel_texts, dialogues
        else:
            panels = paragraphs[:num_panels]
            for panel in panels:
                dialogue = extract_dialogue(panel)
                dialogues.append(dialogue)
                clean_text = panel.replace(dialogue, "") if dialogue else panel
                panel_texts.append(clean_text)
            return panels, panel_texts, dialogues

    sentences = []
    for chunk in story.split('. '):
        sentences.extend([s.strip() + '.' for s in chunk.split('! ')])
    sentences = [s for s in sentences if len(s) > 1]

    if len(sentences) < num_panels:
        while len(sentences) < num_panels:
            longest_idx = max(range(len(sentences)), key=lambda i: len(sentences[i]))
            longest = sentences[longest_idx]

            if ', ' in longest:
                parts = longest.split(', ', 1)
                sentences[longest_idx] = parts[0] + '.'
                sentences.insert(longest_idx + 1, parts[1])
            else:
                sentences.append(f"Meanwhile, {longest.lower()}")

    step = max(1, len(sentences) // num_panels)

    panels = []
    for i in range(num_panels):
        start_idx = i * step
        end_idx = (i + 1) * step if i < num_panels - 1 else len(sentences)
        panel_text = " ".join(sentences[start_idx:end_idx])
        panels.append(panel_text)

        dialogue = extract_dialogue(panel_text)
        dialogues.append(dialogue)

        clean_text = panel_text.replace(dialogue, "") if dialogue else panel_text
        panel_texts.append(clean_text)

    return panels, panel_texts, dialogues

def extract_dialogue(text):
    import re
    dialogue_matches = re.findall(r'"([^"]*)"', text)
    if dialogue_matches:
        return dialogue_matches[0]
    return ""

def load_custom_model():
    print("Loading InkoloRA model - this may take a while...")

    custom_model_path = "/content/drive/MyDrive/InkoloRA.safetensors"

    if not os.path.exists(custom_model_path):
        print(f"Warning: Custom model not found at {custom_model_path}. Falling back to Stable Diffusion XL.")
        return load_sdxl_model()

    try:
        pipe = StableDiffusionPipeline.from_pretrained(
            "runwayml/stable-diffusion-v1-5",
            torch_dtype=torch.float16,
            safety_checker=None
        )

        pipe.unet.load_attn_procs(custom_model_path)

        device = "cuda" if torch.cuda.is_available() else "cpu"
        if device == "cpu":
            print("Warning: CUDA not available, using CPU. This will be very slow.")
        pipe = pipe.to(device)

        print("InkoloRA model loaded successfully!")
        return pipe
    except Exception as e:
        print(f"Error loading custom model: {e}")
        print("Falling back to Stable Diffusion XL...")
        return load_sdxl_model()

def load_sdxl_model():
    print("Loading SDXL model - this may take a while...")
    pipe = StableDiffusionXLPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True
    )

    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device == "cpu":
        print("Warning: CUDA not available, using CPU. This will be very slow.")
    pipe = pipe.to(device)

    return pipe

def add_speech_bubble(image, text, position=None):
    if not text:
        return image

    img = np.array(image.convert("RGBA"))
    height, width, _ = img.shape

    if position is None:
        position = (width // 2, height // 4)

    bubble = np.zeros((height, width, 4), dtype=np.uint8)

    bubble_width = min(300, max(150, len(text) * 10))
    bubble_height = min(130, max(60, (len(text) // 20 + 1) * 30))

    cv2.ellipse(bubble, position, (bubble_width, bubble_height), 0, 0, 360, (255, 255, 255, 255), -1)

    pointer_x = position[0]
    pointer_y = position[1] + bubble_height
    triangle_pts = np.array([
        [pointer_x - 20, pointer_y - 10],
        [pointer_x + 20, pointer_y - 10],
        [pointer_x, pointer_y + 30]
    ])
    cv2.fillPoly(bubble, [triangle_pts], (255, 255, 255, 255))

    bubble_pil = Image.fromarray(bubble)

    draw = ImageDraw.Draw(bubble_pil)

    try:
        font = ImageFont.truetype("arial.ttf", 24)
    except:
        try:
            font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 24)
        except:
            font = ImageFont.load_default()

    wrapped_text = wrap_text(text, font, bubble_width - 40)

    text_x = position[0] - (bubble_width // 2) + 20
    text_y = position[1] - (bubble_height // 2) + 10

    for i, line in enumerate(wrapped_text):
        draw.text((text_x, text_y + i * 30), line, fill="black", font=font)

    return Image.alpha_composite(image.convert("RGBA"), bubble_pil)

def wrap_text(text, font, max_width):
    words = text.split()
    wrapped_lines = []
    current_line = []

    for word in words:
        test_line = ' '.join(current_line + [word])
        try:
            try:
                bbox = font.getbbox(test_line)
                line_width = bbox[2] - bbox[0]
            except AttributeError:
                try:
                    line_width = font.getlength(test_line)
                except AttributeError:
                    line_width = font.getsize(test_line)[0]
        except:
            line_width = len(test_line) * 10

        if line_width <= max_width:
            current_line.append(word)
        else:
            wrapped_lines.append(' '.join(current_line))
            current_line = [word]

    if current_line:
        wrapped_lines.append(' '.join(current_line))

    return wrapped_lines

def get_template_data(template_name):
    templates = {
        "batman": {
            "prompts": [
                "Batman standing on a dark Gotham rooftop, his cape flowing in the wind, the Bat-Signal glowing in the sky behind him.",
                "Close-up shot of Batman's determined face, shadows partially covering his mask, eyes glowing in the darkness.",
                "Batman leaping from a rooftop, his cape spread like wings as he descends towards a group of criminals.",
                "Batman delivering a powerful punch to a thug, the action captured in a comic-style motion blur."
            ],
            "dialogues": [
                "Gotham never sleeps, and neither do I.",
                "Justice isn't a choice. It's a responsibility.",
                "Fear is a tool. I use it against them.",
                "Crime has no place in my city."
            ],
            "positions": [(250, 70), (300, 50), (220, 120), (280, 140)]
        },
        "spiderman": {
            "prompts": [
                "Spider-Man perched on the edge of a skyscraper, overlooking New York City, his iconic red and blue suit vibrant against the skyline.",
                "Spider-Man shooting webs from his wrists, swinging between buildings with great agility.",
                "Spider-Man facing off against a villain, in a dynamic action pose, ready to fight.",
                "Spider-Man helping a civilian, showcasing his friendly neighborhood hero nature."
            ],
            "dialogues": [
                "With great power comes great responsibility.",
                "Just your friendly neighborhood Spider-Man!",
                "My spider-sense is tingling!",
                "I'm not going to let anyone get hurt on my watch."
            ],
            "positions": [(250, 70), (280, 80), (240, 110), (260, 90)]
        },
        "wonderwoman": {
            "prompts": [
                "Wonder Woman standing heroically, her golden lasso and bracelets gleaming, an island paradise visible in the background.",
                "Wonder Woman deflecting bullets with her bracelets, her expression determined and fierce.",
                "Wonder Woman wielding her sword and shield, engaged in an epic battle against mythological creatures.",
                "Wonder Woman using her lasso of truth, the golden rope glowing with magical energy."
            ],
            "dialogues": [
                "I am Diana of Themyscira, daughter of Hippolyta.",
                "I fight for those who cannot fight for themselves.",
                "Peace is a virtue to be shown, not peace enforced.",
                "The truth will set you free."
            ],
            "positions": [(250, 70), (270, 60), (230, 100), (260, 80)]
        }
    }

    return templates.get(template_name.lower(), None)

def get_available_templates():
    return ["Batman", "Spiderman", "WonderWoman"]

def generate_template_comic(template_name, num_panels=4):
    template_data = get_template_data(template_name.lower())

    if not template_data:
        print(f"Template '{template_name}' not found. Available templates: {', '.join(get_available_templates())}")
        return None, None, None, None

    try:
        pipe = load_sdxl_model()

        print(f"\nGenerating {template_name} comic panels...")
        panel_images = []

        print("Generating base character image...")
        base_prompt = f"A detailed illustration of {template_name} in a dynamic pose, full body visible"

        is_sdxl = isinstance(pipe, StableDiffusionXLPipeline)

        if is_sdxl:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                num_inference_steps=30
            ).images[0]
        else:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                num_inference_steps=30,
                guidance_scale=7.5
            ).images[0]

        base_image = base_image.resize((768, 768))

        prompts = template_data["prompts"][:num_panels]
        dialogues = template_data["dialogues"][:num_panels]
        positions = template_data["positions"][:num_panels]

        while len(prompts) < num_panels:
            prompts.append(f"{template_name} in an action scene")
        while len(dialogues) < num_panels:
            dialogues.append("")
        while len(positions) < num_panels:
            positions.append((256, 100))

        for i, prompt in enumerate(prompts):
            print(f"Generating image for panel {i+1}...")

            if is_sdxl:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.65,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                    num_inference_steps=30
                ).images[0]
            else:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.65,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                    num_inference_steps=30,
                    guidance_scale=7.5
                ).images[0]

            image = image.resize((512, 512))

            if dialogues[i]:
                image = add_speech_bubble(image, dialogues[i], positions[i])

            panel_images.append(image)

            image_path = f"panel_{i+1}.png"
            image.save(image_path)

        print("Creating combined comic strip...")
        rows = int(np.ceil(num_panels / 2))
        columns = min(2, num_panels)

        final_width = 512 * columns
        final_height = 512 * rows

        comic_strip = Image.new("RGB", (final_width, final_height), (255, 255, 255))

        for i, image in enumerate(panel_images):
            x_offset = (i % 2) * 512
            y_offset = (i // 2) * 512
            comic_strip.paste(image, (x_offset, y_offset))

        comic_strip.save(f"{template_name}_comic_strip.png")
        print(f"{template_name} comic generation complete!")

        story_text = f"A {template_name} adventure comic generated using a template."
        story_panels = prompts

        return story_text, story_panels, panel_images, comic_strip

    except Exception as e:
        print(f"Error generating template comic: {e}")
        return None, None, None, None

def generate_comic(story_prompt, num_panels=4, model_path=None, art_style="cartoon style", use_template=None):
    if use_template and use_template.lower() != "none":
        return generate_template_comic(use_template, num_panels)

    if len(story_prompt) < 10:
        story_prompt = f"Create an interesting story about {story_prompt}"

    try:
        print("Loading fine-tuned model...")
        model, tokenizer = load_fine_tuned_model(model_path)

        if model is None or tokenizer is None:
            raise Exception("Failed to load the fine-tuned model")

        print(f"Generating story from prompt: '{story_prompt}'")
        story_text = generate_story(model, tokenizer, story_prompt)
        print("Story generated:")
        print(story_text)

        if not story_text or len(story_text) < 50:
            print("Story generation produced insufficient content. Trying one more time with different approach.")
            direct_prompt = f"Write a simple children's story about {story_prompt}. Use short sentences."

            story_text = generate_story(model, tokenizer, direct_prompt, max_length=500)

            if len(story_text) < 50:
                print("Still unable to generate a good story. Using the structured children's story.")
                character_name = extract_character_name(story_prompt) or "the character"
                topic = extract_topic(story_prompt) or "having an adventure"
                story_text = create_structured_childrens_story(character_name, topic)
    except Exception as e:
        print(f"Error generating story: {e}")
        character_name = extract_character_name(story_prompt) or "the character"
        topic = extract_topic(story_prompt) or "having an adventure"
        story_text = create_structured_childrens_story(character_name, topic)

    print("\nSplitting story into panels...")
    story_panels, panel_texts, dialogues = split_story_into_panels(story_text, num_panels)
    for i, panel in enumerate(story_panels):
        print(f"Panel {i+1}: {panel}")
        if dialogues[i]:
            print(f"Dialogue for panel {i+1}: \"{dialogues[i]}\"")

    try:
        pipe = load_custom_model()

        print("\nGenerating panel images...")
        panel_images = []

        character_name = extract_character_name(story_prompt) or "the character"

        print("Generating base character image for consistency...")
        base_prompt = f"A friendly {art_style} illustration of {character_name}, whole body visible, simple background, bright colors, child-friendly"

        is_sdxl = isinstance(pipe, StableDiffusionXLPipeline)

        if is_sdxl:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                num_inference_steps=30
            ).images[0]
        else:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                num_inference_steps=30,
                guidance_scale=7.5
            ).images[0]

        base_image = base_image.resize((768, 768))

        for i, panel_text in enumerate(panel_texts):
            print(f"Generating image for panel {i+1}...")
            prompt = f"A friendly {art_style} illustration for a children's storybook showing: {panel_text}, featuring {character_name}, bright colors, simple shapes, cute characters, child-friendly"

            if is_sdxl:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.7,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                    num_inference_steps=30
                ).images[0]
            else:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.7,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                    num_inference_steps=30,
                    guidance_scale=7.5
                ).images[0]

            image = image.resize((512, 512))

            if dialogues[i]:
                position = (256, 100 + (i % 2) * 50)
                image = add_speech_bubble(image, dialogues[i], position)

            panel_images.append(image)

            image_path = f"panel_{i+1}.png"
            image.save(image_path)

        print("Creating combined comic strip...")
        rows = int(np.ceil(num_panels / 2))
        columns = min(2, num_panels)

        final_width = 512 * columns
        final_height = 512 * rows

        comic_strip = Image.new("RGB", (final_width, final_height), (255, 255, 255))

        for i, image in enumerate(panel_images):
            x_offset = (i % 2) * 512
            y_offset = (i // 2) * 512
            comic_strip.paste(image, (x_offset, y_offset))

        comic_strip.save("comic_strip.png")
        print("Comic generation complete! All panels saved as PNG files and combined into comic_strip.png")

        return story_text, story_panels, panel_images, comic_strip

    except Exception as e:
        print(f"Error in image generation: {e}")
        print("Unable to generate images. Please check if you have enough GPU memory and proper installations.")
        return story_text, story_panels, None, None

# Web UI using Gradio
def create_ui():
    with gr.Blocks() as app:
        gr.Markdown("# Story to Comic Generator with InkoloRA")
        gr.Markdown("Enter a prompt to generate a story and convert it into comic panels with speech bubbles using the InkoloRA model")

        with gr.Row():
            with gr.Column():
                template_dropdown = gr.Dropdown(
                    choices=["None"] + get_available_templates(),
                    label="Choose Template (optional)",
                    value="None"
                )

                story_prompt = gr.Textbox(
                    label="Story Prompt",
                    placeholder="Enter a prompt for the children's story (e.g. 'a little boy who learns to fly')",
                    lines=2
                )

                num_panels = gr.Slider(
                    label="Number of Panels",
                    minimum=2,
                    maximum=8,
                    value=4,
                    step=1
                )

                art_style = gr.Dropdown(
                    choices=["Cartoon Style", "Realistic", "Anime", "Sketch"],
                    label="Art Style",
                    value="Cartoon Style"
                )

                generate_button = gr.Button("Generate Comic")

            with gr.Column():
                output_text = gr.Textbox(
                    label="Generated Story",
                    placeholder="The generated story will appear here...",
                    lines=10,
                    interactive=False
                )

                output_panels = gr.Gallery(
                    label="Comic Panels",
                    show_label=True,
                    elem_id="comic-panels",
                    height=200,
                    columns=2  # Set the number of columns for the gallery
                )

                output_comic = gr.Image(
                    label="Combined Comic Strip",
                    type="pil"
                )

        # Fixed function to handle the generation and properly return outputs
        def generate_comic_action(story_prompt, num_panels, art_style, use_template):
            story_text, story_panels, panel_images, comic_strip = generate_comic(
                story_prompt, num_panels, art_style=art_style, use_template=use_template
            )
            # Return the outputs directly rather than updating them
            return story_text, panel_images if panel_images else [], comic_strip

        generate_button.click(
            generate_comic_action,
            inputs=[story_prompt, num_panels, art_style, template_dropdown],
            outputs=[output_text, output_panels, output_comic]
        )

    app.launch()

# Mount Google Drive if in Colab environment
try:
    drive.mount('/content/drive')
except:
    print("Not running in Colab or drive already mounted")

# Call the function to create the UI
if __name__ == "__main__":
    create_ui()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://44957111b564a93361.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import cv2
from transformers import AutoTokenizer, AutoModelForCausalLM
from diffusers import StableDiffusionPipeline, StableDiffusionXLPipeline
from PIL import Image, ImageDraw, ImageFont
import gradio as gr
from google.colab import drive

# Load the fine-tuned model
def load_fine_tuned_model(model_path=None):
    if not model_path:
        model_path = '/content/drive/MyDrive/fine_tuned_tiny_stories_model'

    try:
        print(f"Loading fine-tuned model from {model_path}...")
        model = AutoModelForCausalLM.from_pretrained(model_path)
        tokenizer = AutoTokenizer.from_pretrained(model_path)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)

        print(f"Model loaded successfully and moved to {device}")
        return model, tokenizer
    except Exception as e:
        print(f"Error loading model: {e}")
        # Try loading a fallback model if the fine-tuned one fails
        try:
            print("Attempting to load fallback model gpt2...")
            model = AutoModelForCausalLM.from_pretrained("gpt2")
            tokenizer = AutoTokenizer.from_pretrained("gpt2")
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model = model.to(device)
            print(f"Fallback model loaded successfully and moved to {device}")
            return model, tokenizer
        except Exception as e2:
            print(f"Error loading fallback model: {e2}")
            return None, None

# Helper functions for story generation
def extract_character_name(prompt):
    character_names = ["Spider-Man", "Peter Parker", "Batman", "Superman", "Wonder Woman",
                       "Iron Man", "Captain America", "Thor", "Hulk", "Black Widow"]

    for name in character_names:
        if name.lower() in prompt.lower():
            return name

    words = prompt.split()
    for word in words:
        if word[0].isupper() and len(word) > 1 and word.lower() not in ["write", "story", "about"]:
            return word

    return None

def extract_topic(prompt):
    cleaned = prompt.lower().replace("write", "").replace("story", "").replace("about", "")
    keywords = ["adventure", "flying", "jumping", "fighting", "learning", "playing", "making friends"]

    for keyword in keywords:
        if keyword in cleaned:
            return keyword

    return cleaned.strip()

def generate_story(model, tokenizer, prompt, max_length=400, retries=3):
    story_prompt = f"Write a simple children's story about {prompt}. Use simple language and short sentences."

    # Increase repetition penalty and adjust parameters for better generation
    for attempt in range(retries):
        try:
            input_ids = tokenizer.encode(story_prompt, return_tensors='pt')
            device = next(model.parameters()).device
            input_ids = input_ids.to(device)

            # Use different generation parameters for each retry attempt
            temperature = 0.7 + (attempt * 0.1)  # Gradually increase temperature with each retry
            repetition_penalty = 1.2 + (attempt * 0.1)  # Gradually increase repetition penalty

            output = model.generate(
                input_ids,
                max_length=max_length,
                num_return_sequences=1,
                no_repeat_ngram_size=3,  # Increased from 2
                top_k=50,  # Increased from 40
                top_p=0.92,  # Adjusted from 0.9
                temperature=temperature,
                do_sample=True,
                repetition_penalty=repetition_penalty
            )

            generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

            # Clean up the generated text to focus on the actual story
            if story_prompt in generated_text:
                generated_text = generated_text.replace(story_prompt, "").strip()

            # Check if we have a reasonable story
            if len(generated_text) > 100 and "." in generated_text:
                print(f"Successfully generated story on attempt {attempt+1}")
                return generated_text
            else:
                print(f"Attempt {attempt+1} produced insufficient content ({len(generated_text)} chars). Retrying...")

        except Exception as e:
            print(f"Error in story generation attempt {attempt+1}: {e}")

    # If all retries fail, generate a more varied story using a different approach
    print("All generation attempts failed. Using alternative generation method...")
    return generate_alternative_story(prompt)

def generate_alternative_story(prompt):
    """
    Generate a more varied story when the model-based generation fails.
    This creates a more dynamic story than the previous structured approach.
    """
    character = extract_character_name(prompt) or "the little hero"
    topic = extract_topic(prompt) or "having an adventure"

    # Create a list of possible story templates
    story_templates = [
        f"""Once upon a time, there was {character} who lived in a small village.
{character} loved {topic} more than anything else.

One day, {character} discovered something magical while playing outside. It was a tiny glowing ball that could float in the air.

"What's this?" wondered {character}. The ball started to move, and {character} followed it into the forest.

The ball led {character} to a hidden garden where other children were playing with similar magical balls. They welcomed {character} and taught them new games.

{character} had so much fun that day! When it was time to go home, one of the children gave {character} a magical ball of their own.

From that day on, {character} visited the hidden garden often and made many new friends. They all loved {topic} together and had wonderful adventures.""",

        f"""{character} woke up early one morning with a big smile. Today was the day for {topic}!

{character} packed a small bag with snacks and a water bottle. "I'm ready for my adventure!" {character} said.

Outside, {character} met a friendly cat. "Hello, cat! Would you like to join me for {topic}?" The cat meowed and followed along.

As they walked through the park, they found a lost teddy bear. "We should find who this belongs to," said {character}.

They asked everyone they met about the teddy bear. Finally, they met a little girl who was crying. "My teddy!" she said happily when she saw it.

"Thank you for helping me find my teddy," the girl smiled. "Would you like to play with me?"

{character}, the cat, and the girl spent the whole day playing and having fun with {topic}. It was the best day ever!""",

        f"""The sun was shining brightly when {character} decided it was the perfect day for {topic}.

"I've never tried {topic} before," {character} thought, "but today I will be brave!"

{character} began slowly, taking small steps. It wasn't easy at first, and {character} almost gave up.

"Don't worry, you can do it!" said a friendly voice. {character} looked up and saw a smiling butterfly.

With the butterfly's encouragement, {character} tried again. This time, {character} did much better!

Soon, {character} was having so much fun with {topic}. "Thank you for believing in me," {character} said to the butterfly.

The butterfly fluttered around happily. "Sometimes we just need a friend to help us be brave," it replied.

{character} went home that evening with a happy heart, excited to try {topic} again tomorrow."""
    ]

    import random
    return random.choice(story_templates)

def split_story_into_panels(story, num_panels=4):
    if len(story) < 100:
        print("Warning: Story is very short. Panel division may not be optimal.")

    paragraphs = story.split('\n')
    paragraphs = [p for p in paragraphs if p.strip()]

    dialogues = []
    panel_texts = []

    if len(paragraphs) >= num_panels:
        if len(paragraphs) > num_panels:
            step = len(paragraphs) / num_panels
            panels = []
            for i in range(num_panels):
                start_idx = int(i * step)
                end_idx = int((i + 1) * step) if i < num_panels - 1 else len(paragraphs)
                panel_text = " ".join(paragraphs[start_idx:end_idx])
                panels.append(panel_text)

                dialogue = extract_dialogue(panel_text)
                dialogues.append(dialogue)

                clean_text = panel_text.replace(dialogue, "") if dialogue else panel_text
                panel_texts.append(clean_text)
            return panels, panel_texts, dialogues
        else:
            panels = paragraphs[:num_panels]
            for panel in panels:
                dialogue = extract_dialogue(panel)
                dialogues.append(dialogue)
                clean_text = panel.replace(dialogue, "") if dialogue else panel
                panel_texts.append(clean_text)
            return panels, panel_texts, dialogues

    sentences = []
    for chunk in story.split('. '):
        sentences.extend([s.strip() + '.' for s in chunk.split('! ')])
    sentences = [s for s in sentences if len(s) > 1]

    if len(sentences) < num_panels:
        while len(sentences) < num_panels:
            longest_idx = max(range(len(sentences)), key=lambda i: len(sentences[i]))
            longest = sentences[longest_idx]

            if ', ' in longest:
                parts = longest.split(', ', 1)
                sentences[longest_idx] = parts[0] + '.'
                sentences.insert(longest_idx + 1, parts[1])
            else:
                sentences.append(f"Meanwhile, {longest.lower()}")

    step = max(1, len(sentences) // num_panels)

    panels = []
    for i in range(num_panels):
        start_idx = i * step
        end_idx = (i + 1) * step if i < num_panels - 1 else len(sentences)
        panel_text = " ".join(sentences[start_idx:end_idx])
        panels.append(panel_text)

        dialogue = extract_dialogue(panel_text)
        dialogues.append(dialogue)

        clean_text = panel_text.replace(dialogue, "") if dialogue else panel_text
        panel_texts.append(clean_text)

    return panels, panel_texts, dialogues

def extract_dialogue(text):
    import re
    dialogue_matches = re.findall(r'"([^"]*)"', text)
    if dialogue_matches:
        return dialogue_matches[0]
    return ""

def load_custom_model():
    print("Loading InkoloRA model - this may take a while...")

    custom_model_path = "/content/drive/MyDrive/InkoloRA.safetensors"

    if not os.path.exists(custom_model_path):
        print(f"Warning: Custom model not found at {custom_model_path}. Falling back to Stable Diffusion XL.")
        return load_sdxl_model()

    try:
        pipe = StableDiffusionPipeline.from_pretrained(
            "runwayml/stable-diffusion-v1-5",
            torch_dtype=torch.float16,
            safety_checker=None
        )

        pipe.unet.load_attn_procs(custom_model_path)

        device = "cuda" if torch.cuda.is_available() else "cpu"
        if device == "cpu":
            print("Warning: CUDA not available, using CPU. This will be very slow.")
        pipe = pipe.to(device)

        print("InkoloRA model loaded successfully!")
        return pipe
    except Exception as e:
        print(f"Error loading custom model: {e}")
        print("Falling back to Stable Diffusion XL...")
        return load_sdxl_model()

def load_sdxl_model():
    print("Loading SDXL model - this may take a while...")
    pipe = StableDiffusionXLPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True
    )

    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device == "cpu":
        print("Warning: CUDA not available, using CPU. This will be very slow.")
    pipe = pipe.to(device)

    return pipe

def add_speech_bubble(image, text, position=None):
    if not text:
        return image

    img = np.array(image.convert("RGBA"))
    height, width, _ = img.shape

    if position is None:
        position = (width // 2, height // 4)

    bubble = np.zeros((height, width, 4), dtype=np.uint8)

    bubble_width = min(300, max(150, len(text) * 10))
    bubble_height = min(130, max(60, (len(text) // 20 + 1) * 30))

    cv2.ellipse(bubble, position, (bubble_width, bubble_height), 0, 0, 360, (255, 255, 255, 255), -1)

    pointer_x = position[0]
    pointer_y = position[1] + bubble_height
    triangle_pts = np.array([
        [pointer_x - 20, pointer_y - 10],
        [pointer_x + 20, pointer_y - 10],
        [pointer_x, pointer_y + 30]
    ])
    cv2.fillPoly(bubble, [triangle_pts], (255, 255, 255, 255))

    bubble_pil = Image.fromarray(bubble)

    draw = ImageDraw.Draw(bubble_pil)

    try:
        font = ImageFont.truetype("arial.ttf", 24)
    except:
        try:
            font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 24)
        except:
            font = ImageFont.load_default()

    wrapped_text = wrap_text(text, font, bubble_width - 40)

    text_x = position[0] - (bubble_width // 2) + 20
    text_y = position[1] - (bubble_height // 2) + 10

    for i, line in enumerate(wrapped_text):
        draw.text((text_x, text_y + i * 30), line, fill="black", font=font)

    return Image.alpha_composite(image.convert("RGBA"), bubble_pil)

def wrap_text(text, font, max_width):
    words = text.split()
    wrapped_lines = []
    current_line = []

    for word in words:
        test_line = ' '.join(current_line + [word])
        try:
            try:
                bbox = font.getbbox(test_line)
                line_width = bbox[2] - bbox[0]
            except AttributeError:
                try:
                    line_width = font.getlength(test_line)
                except AttributeError:
                    line_width = font.getsize(test_line)[0]
        except:
            line_width = len(test_line) * 10

        if line_width <= max_width:
            current_line.append(word)
        else:
            wrapped_lines.append(' '.join(current_line))
            current_line = [word]

    if current_line:
        wrapped_lines.append(' '.join(current_line))

    return wrapped_lines

def get_template_data(template_name):
    templates = {
        "batman": {
            "prompts": [
                "Batman standing on a dark Gotham rooftop, his cape flowing in the wind, the Bat-Signal glowing in the sky behind him.",
                "Close-up shot of Batman's determined face, shadows partially covering his mask, eyes glowing in the darkness.",
                "Batman leaping from a rooftop, his cape spread like wings as he descends towards a group of criminals.",
                "Batman delivering a powerful punch to a thug, the action captured in a comic-style motion blur."
            ],
            "dialogues": [
                "Gotham never sleeps, and neither do I.",
                "Justice isn't a choice. It's a responsibility.",
                "Fear is a tool. I use it against them.",
                "Crime has no place in my city."
            ],
            "positions": [(250, 70), (300, 50), (220, 120), (280, 140)]
        },
        "spiderman": {
            "prompts": [
                "Spider-Man perched on the edge of a skyscraper, overlooking New York City, his iconic red and blue suit vibrant against the skyline.",
                "Spider-Man shooting webs from his wrists, swinging between buildings with great agility.",
                "Spider-Man facing off against a villain, in a dynamic action pose, ready to fight.",
                "Spider-Man helping a civilian, showcasing his friendly neighborhood hero nature."
            ],
            "dialogues": [
                "With great power comes great responsibility.",
                "Just your friendly neighborhood Spider-Man!",
                "My spider-sense is tingling!",
                "I'm not going to let anyone get hurt on my watch."
            ],
            "positions": [(250, 70), (280, 80), (240, 110), (260, 90)]
        },
        "wonderwoman": {
            "prompts": [
                "Wonder Woman standing heroically, her golden lasso and bracelets gleaming, an island paradise visible in the background.",
                "Wonder Woman deflecting bullets with her bracelets, her expression determined and fierce.",
                "Wonder Woman wielding her sword and shield, engaged in an epic battle against mythological creatures.",
                "Wonder Woman using her lasso of truth, the golden rope glowing with magical energy."
            ],
            "dialogues": [
                "I am Diana of Themyscira, daughter of Hippolyta.",
                "I fight for those who cannot fight for themselves.",
                "Peace is a virtue to be shown, not peace enforced.",
                "The truth will set you free."
            ],
            "positions": [(250, 70), (270, 60), (230, 100), (260, 80)]
        }
    }

    return templates.get(template_name.lower(), None)

def get_available_templates():
    return ["Batman", "Spiderman", "WonderWoman"]

def generate_template_comic(template_name, num_panels=4):
    template_data = get_template_data(template_name.lower())

    if not template_data:
        print(f"Template '{template_name}' not found. Available templates: {', '.join(get_available_templates())}")
        return None, None, None, None

    try:
        pipe = load_sdxl_model()

        print(f"\nGenerating {template_name} comic panels...")
        panel_images = []

        print("Generating base character image...")
        base_prompt = f"A detailed illustration of {template_name} in a dynamic pose, full body visible"

        is_sdxl = isinstance(pipe, StableDiffusionXLPipeline)

        if is_sdxl:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                num_inference_steps=30
            ).images[0]
        else:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                num_inference_steps=30,
                guidance_scale=7.5
            ).images[0]

        base_image = base_image.resize((768, 768))

        prompts = template_data["prompts"][:num_panels]
        dialogues = template_data["dialogues"][:num_panels]
        positions = template_data["positions"][:num_panels]

        while len(prompts) < num_panels:
            prompts.append(f"{template_name} in an action scene")
        while len(dialogues) < num_panels:
            dialogues.append("")
        while len(positions) < num_panels:
            positions.append((256, 100))

        for i, prompt in enumerate(prompts):
            print(f"Generating image for panel {i+1}...")

            if is_sdxl:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.65,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                    num_inference_steps=30
                ).images[0]
            else:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.65,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                    num_inference_steps=30,
                    guidance_scale=7.5
                ).images[0]

            image = image.resize((512, 512))

            if dialogues[i]:
                image = add_speech_bubble(image, dialogues[i], positions[i])

            panel_images.append(image)

            image_path = f"panel_{i+1}.png"
            image.save(image_path)

        print("Creating combined comic strip...")
        rows = int(np.ceil(num_panels / 2))
        columns = min(2, num_panels)

        final_width = 512 * columns
        final_height = 512 * rows

        comic_strip = Image.new("RGB", (final_width, final_height), (255, 255, 255))

        for i, image in enumerate(panel_images):
            x_offset = (i % 2) * 512
            y_offset = (i // 2) * 512
            comic_strip.paste(image, (x_offset, y_offset))

        comic_strip.save(f"{template_name}_comic_strip.png")
        print(f"{template_name} comic generation complete!")

        story_text = f"A {template_name} adventure comic generated using a template."
        story_panels = prompts

        return story_text, story_panels, panel_images, comic_strip

    except Exception as e:
        print(f"Error generating template comic: {e}")
        return None, None, None, None

def generate_comic(story_prompt, num_panels=4, model_path=None, art_style="cartoon style", use_template=None):
    if use_template and use_template.lower() != "none":
        return generate_template_comic(use_template, num_panels)

    if len(story_prompt) < 10:
        story_prompt = f"Create an interesting story about {story_prompt}"

    try:
        print("Loading fine-tuned model...")
        model, tokenizer = load_fine_tuned_model(model_path)

        if model is None or tokenizer is None:
            raise Exception("Failed to load the fine-tuned model")

        print(f"Generating story from prompt: '{story_prompt}'")
        story_text = generate_story(model, tokenizer, story_prompt)
        print("Story generated:")
        print(story_text)

        # If story generation still produces insufficient content after multiple attempts
        # and with different parameters, try generating with a completely different approach
        if not story_text or len(story_text) < 50:
            print("Story generation produced insufficient content. Using alternative generation approach.")
            story_text = generate_alternative_story(story_prompt)
            print("Alternative story generated:")
            print(story_text)

    except Exception as e:
        print(f"Error in model-based story generation: {e}")
        print("Using alternative story generation approach...")
        story_text = generate_alternative_story(story_prompt)

    print("\nSplitting story into panels...")
    story_panels, panel_texts, dialogues = split_story_into_panels(story_text, num_panels)
    for i, panel in enumerate(story_panels):
        print(f"Panel {i+1}: {panel}")
        if dialogues[i]:
            print(f"Dialogue for panel {i+1}: \"{dialogues[i]}\"")

    try:
        pipe = load_custom_model()

        print("\nGenerating panel images...")
        panel_images = []

        character_name = extract_character_name(story_prompt) or "the character"

        print("Generating base character image for consistency...")
        base_prompt = f"A friendly {art_style} illustration of {character_name}, whole body visible, simple background, bright colors, child-friendly"

        is_sdxl = isinstance(pipe, StableDiffusionXLPipeline)

        if is_sdxl:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                num_inference_steps=30
            ).images[0]
        else:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                num_inference_steps=30,
                guidance_scale=7.5
            ).images[0]

        base_image = base_image.resize((768, 768))

        for i, panel_text in enumerate(panel_texts):
            print(f"Generating image for panel {i+1}...")
            # Make the image prompts unique for each panel based on the actual story content
            prompt = f"A friendly {art_style} illustration for a children's storybook showing: {panel_text}, featuring {character_name}, bright colors, simple shapes, cute characters, child-friendly"

            if is_sdxl:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.7,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                    num_inference_steps=30
                ).images[0]
            else:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.7,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                    num_inference_steps=30,
                    guidance_scale=7.5
                ).images[0]

            image = image.resize((512, 512))

            if dialogues[i]:
                position = (256, 100 + (i % 2) * 50)
                image = add_speech_bubble(image, dialogues[i], position)

            panel_images.append(image)

            image_path = f"panel_{i+1}.png"
            image.save(image_path)

        print("Creating combined comic strip...")
        rows = int(np.ceil(num_panels / 2))
        columns = min(2, num_panels)

        final_width = 512 * columns
        final_height = 512 * rows

        comic_strip = Image.new("RGB", (final_width, final_height), (255, 255, 255))

        for i, image in enumerate(panel_images):
            x_offset = (i % 2) * 512
            y_offset = (i // 2) * 512
            comic_strip.paste(image, (x_offset, y_offset))

        comic_strip.save("comic_strip.png")
        print("Comic generation complete! All panels saved as PNG files and combined into comic_strip.png")

        return story_text, story_panels, panel_images, comic_strip

    except Exception as e:
        print(f"Error in image generation: {e}")
        print("Unable to generate images. Please check if you have enough GPU memory and proper installations.")
        return story_text, story_panels, None, None

# Web UI using Gradio
def create_ui():
    with gr.Blocks() as app:
        gr.Markdown("# Story to Comic Generator with InkoloRA")
        gr.Markdown("Enter a prompt to generate a story and convert it into comic panels with speech bubbles using the InkoloRA model")

        with gr.Row():
            with gr.Column():
                template_dropdown = gr.Dropdown(
                    choices=["None"] + get_available_templates(),
                    label="Choose Template (optional)",
                    value="None"
                )

                story_prompt = gr.Textbox(
                    label="Story Prompt",
                    placeholder="Enter a prompt for the children's story (e.g. 'a little boy who learns to fly')",
                    lines=2
                )

                num_panels = gr.Slider(
                    label="Number of Panels",
                    minimum=2,
                    maximum=8,
                    value=4,
                    step=1
                )

                art_style = gr.Dropdown(
                    choices=["Cartoon Style", "Realistic", "Anime", "Sketch"],
                    label="Art Style",
                    value="Cartoon Style"
                )

                generate_button = gr.Button("Generate Comic")

            with gr.Column():
                output_text = gr.Textbox(
                    label="Generated Story",
                    placeholder="The generated story will appear here...",
                    lines=10,
                    interactive=False
                )

                output_panels = gr.Gallery(
                    label="Comic Panels",
                    show_label=True,
                    elem_id="comic-panels",
                    height=200,
                    columns=2  # Set the number of columns for the gallery
                )

                output_comic = gr.Image(
                    label="Combined Comic Strip",
                    type="pil"
                )

        # Fixed function to handle the generation and properly return outputs
        def generate_comic_action(story_prompt, num_panels, art_style, use_template):
            story_text, story_panels, panel_images, comic_strip = generate_comic(
                story_prompt, num_panels, art_style=art_style, use_template=use_template
            )
            # Return the outputs directly rather than updating them
            return story_text, panel_images if panel_images else [], comic_strip

        generate_button.click(
            generate_comic_action,
            inputs=[story_prompt, num_panels, art_style, template_dropdown],
            outputs=[output_text, output_panels, output_comic]
        )

    app.launch()

# Mount Google Drive if in Colab environment
try:
    drive.mount('/content/drive')
except:
    print("Not running in Colab or drive already mounted")

# Call the function to create the UI
if __name__ == "__main__":
    create_ui()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://be8fe34d6cbdceef8d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import cv2
from transformers import AutoTokenizer, AutoModelForCausalLM
from diffusers import StableDiffusionPipeline, StableDiffusionXLPipeline
from PIL import Image, ImageDraw, ImageFont
import gradio as gr
from google.colab import drive

# Load the fine-tuned model
def load_fine_tuned_model(model_path=None):
    if not model_path:
        model_path = '/content/drive/MyDrive/fine_tuned_tiny_stories_model'

    try:
        print(f"Loading fine-tuned model from {model_path}...")
        model = AutoModelForCausalLM.from_pretrained(model_path)
        tokenizer = AutoTokenizer.from_pretrained(model_path)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)

        print(f"Model loaded successfully and moved to {device}")
        return model, tokenizer
    except Exception as e:
        print(f"Error loading model: {e}")
        # Try loading a fallback model if the fine-tuned one fails
        try:
            print("Attempting to load fallback model gpt2...")
            model = AutoModelForCausalLM.from_pretrained("gpt2")
            tokenizer = AutoTokenizer.from_pretrained("gpt2")
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model = model.to(device)
            print(f"Fallback model loaded successfully and moved to {device}")
            return model, tokenizer
        except Exception as e2:
            print(f"Error loading fallback model: {e2}")
            return None, None

# Helper functions for story generation
def extract_character_name(prompt):
    character_names = ["Spider-Man", "Peter Parker", "Batman", "Superman", "Wonder Woman",
                       "Iron Man", "Captain America", "Thor", "Hulk", "Black Widow"]

    for name in character_names:
        if name.lower() in prompt.lower():
            return name

    words = prompt.split()
    for word in words:
        if word[0].isupper() and len(word) > 1 and word.lower() not in ["write", "story", "about"]:
            return word

    return None

def extract_topic(prompt):
    cleaned = prompt.lower().replace("write", "").replace("story", "").replace("about", "")
    keywords = ["adventure", "flying", "jumping", "fighting", "learning", "playing", "making friends"]

    for keyword in keywords:
        if keyword in cleaned:
            return keyword

    return cleaned.strip()

def generate_story(model, tokenizer, prompt, max_length=400, retries=3):
    story_prompt = f"Write a simple children's story about {prompt}. Use simple language and short sentences."

    # Increase repetition penalty and adjust parameters for better generation
    for attempt in range(retries):
        try:
            input_ids = tokenizer.encode(story_prompt, return_tensors='pt')
            device = next(model.parameters()).device
            input_ids = input_ids.to(device)

            # Use different generation parameters for each retry attempt
            temperature = 0.7 + (attempt * 0.1)  # Gradually increase temperature with each retry
            repetition_penalty = 1.2 + (attempt * 0.1)  # Gradually increase repetition penalty

            output = model.generate(
                input_ids,
                max_length=max_length,
                num_return_sequences=1,
                no_repeat_ngram_size=3,  # Increased from 2
                top_k=50,  # Increased from 40
                top_p=0.92,  # Adjusted from 0.9
                temperature=temperature,
                do_sample=True,
                repetition_penalty=repetition_penalty
            )

            generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

            # Clean up the generated text to focus on the actual story
            if story_prompt in generated_text:
                generated_text = generated_text.replace(story_prompt, "").strip()

            # Check if we have a reasonable story
            if len(generated_text) > 100 and "." in generated_text:
                print(f"Successfully generated story on attempt {attempt+1}")
                return generated_text
            else:
                print(f"Attempt {attempt+1} produced insufficient content ({len(generated_text)} chars). Retrying...")

        except Exception as e:
            print(f"Error in story generation attempt {attempt+1}: {e}")

    # If all retries fail, generate a more varied story using a different approach
    print("All generation attempts failed. Using alternative generation method...")
    return generate_alternative_story(prompt)

def generate_alternative_story(prompt):
    """
    Generate a more varied story when the model-based generation fails.
    This creates a more dynamic story than the previous structured approach.
    """
    character = extract_character_name(prompt) or "the little hero"
    topic = extract_topic(prompt) or "having an adventure"

    # Create a list of possible story templates
    story_templates = [
        f"""Once upon a time, there was {character} who lived in a small village.
{character} loved {topic} more than anything else.

One day, {character} discovered something magical while playing outside. It was a tiny glowing ball that could float in the air.

"What's this?" wondered {character}. The ball started to move, and {character} followed it into the forest.

The ball led {character} to a hidden garden where other children were playing with similar magical balls. They welcomed {character} and taught them new games.

{character} had so much fun that day! When it was time to go home, one of the children gave {character} a magical ball of their own.

From that day on, {character} visited the hidden garden often and made many new friends. They all loved {topic} together and had wonderful adventures.""",

        f"""{character} woke up early one morning with a big smile. Today was the day for {topic}!

{character} packed a small bag with snacks and a water bottle. "I'm ready for my adventure!" {character} said.

Outside, {character} met a friendly cat. "Hello, cat! Would you like to join me for {topic}?" The cat meowed and followed along.

As they walked through the park, they found a lost teddy bear. "We should find who this belongs to," said {character}.

They asked everyone they met about the teddy bear. Finally, they met a little girl who was crying. "My teddy!" she said happily when she saw it.

"Thank you for helping me find my teddy," the girl smiled. "Would you like to play with me?"

{character}, the cat, and the girl spent the whole day playing and having fun with {topic}. It was the best day ever!""",

        f"""The sun was shining brightly when {character} decided it was the perfect day for {topic}.

"I've never tried {topic} before," {character} thought, "but today I will be brave!"

{character} began slowly, taking small steps. It wasn't easy at first, and {character} almost gave up.

"Don't worry, you can do it!" said a friendly voice. {character} looked up and saw a smiling butterfly.

With the butterfly's encouragement, {character} tried again. This time, {character} did much better!

Soon, {character} was having so much fun with {topic}. "Thank you for believing in me," {character} said to the butterfly.

The butterfly fluttered around happily. "Sometimes we just need a friend to help us be brave," it replied.

{character} went home that evening with a happy heart, excited to try {topic} again tomorrow."""
    ]

    import random
    return random.choice(story_templates)

def split_story_into_panels(story, num_panels=4):
    if len(story) < 100:
        print("Warning: Story is very short. Panel division may not be optimal.")

    paragraphs = story.split('\n')
    paragraphs = [p for p in paragraphs if p.strip()]

    dialogues = []
    panel_texts = []

    if len(paragraphs) >= num_panels:
        if len(paragraphs) > num_panels:
            step = len(paragraphs) / num_panels
            panels = []
            for i in range(num_panels):
                start_idx = int(i * step)
                end_idx = int((i + 1) * step) if i < num_panels - 1 else len(paragraphs)
                panel_text = " ".join(paragraphs[start_idx:end_idx])
                panels.append(panel_text)

                dialogue = extract_dialogue(panel_text)
                dialogues.append(dialogue)

                clean_text = panel_text.replace(dialogue, "") if dialogue else panel_text
                panel_texts.append(clean_text)
            return panels, panel_texts, dialogues
        else:
            panels = paragraphs[:num_panels]
            for panel in panels:
                dialogue = extract_dialogue(panel)
                dialogues.append(dialogue)
                clean_text = panel.replace(dialogue, "") if dialogue else panel
                panel_texts.append(clean_text)
            return panels, panel_texts, dialogues

    sentences = []
    for chunk in story.split('. '):
        sentences.extend([s.strip() + '.' for s in chunk.split('! ')])
    sentences = [s for s in sentences if len(s) > 1]

    if len(sentences) < num_panels:
        while len(sentences) < num_panels:
            longest_idx = max(range(len(sentences)), key=lambda i: len(sentences[i]))
            longest = sentences[longest_idx]

            if ', ' in longest:
                parts = longest.split(', ', 1)
                sentences[longest_idx] = parts[0] + '.'
                sentences.insert(longest_idx + 1, parts[1])
            else:
                sentences.append(f"Meanwhile, {longest.lower()}")

    step = max(1, len(sentences) // num_panels)

    panels = []
    for i in range(num_panels):
        start_idx = i * step
        # For the last panel, include any remaining sentences
        end_idx = (i + 1) * step if i < num_panels - 1 else len(sentences)
        panel_text = " ".join(sentences[start_idx:end_idx])
        panels.append(panel_text)

        dialogue = extract_dialogue(panel_text)
        dialogues.append(dialogue)

        clean_text = panel_text.replace(dialogue, "") if dialogue else panel_text
        panel_texts.append(clean_text)

    return panels, panel_texts, dialogues

def extract_dialogue(text):
    import re
    dialogue_matches = re.findall(r'"([^"]*)"', text)
    if dialogue_matches:
        # Return the first dialogue found
        return dialogue_matches[0]
    return ""

def load_custom_model():
    print("Loading InkoloRA model - this may take a while...")

    custom_model_path = "/content/drive/MyDrive/InkoloRA.safetensors"

    if not os.path.exists(custom_model_path):
        print(f"Warning: Custom model not found at {custom_model_path}. Falling back to Stable Diffusion XL.")
        return load_sdxl_model()

    try:
        # Load the model - adjust the model_id based on what base model your safetensors file is trained on
        # Assuming it's SD 1.5 based; modify if it's based on a different architecture
        pipe = StableDiffusionPipeline.from_pretrained(
            "runwayml/stable-diffusion-v1-5",
            torch_dtype=torch.float16,
            safety_checker=None
        )

        # Load the custom weights
        pipe.unet.load_attn_procs(custom_model_path)

        # Check if CUDA is available, otherwise use CPU
        device = "cuda" if torch.cuda.is_available() else "cpu"
        if device == "cpu":
            print("Warning: CUDA not available, using CPU. This will be very slow.")
        pipe = pipe.to(device)

        print("InkoloRA model loaded successfully!")
        return pipe
    except Exception as e:
        print(f"Error loading custom model: {e}")
        print("Falling back to Stable Diffusion XL...")
        return load_sdxl_model()

# Original SDXL model load function as fallback
def load_sdxl_model():
    print("Loading SDXL model - this may take a while...")
    pipe = StableDiffusionXLPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True
    )

    # Check if CUDA is available, otherwise use CPU
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device == "cpu":
        print("Warning: CUDA not available, using CPU. This will be very slow.")
    pipe = pipe.to(device)

    return pipe

# Function to add speech bubbles to images
def add_speech_bubble(image, text, position=None):
    if not text:
        return image

    # Convert PIL image to numpy array for OpenCV operations
    img = np.array(image.convert("RGBA"))
    height, width, _ = img.shape

    # If position not provided, place the bubble at a suitable position
    if position is None:
        position = (width // 2, height // 4)

    # Create a transparent layer for the bubble
    bubble = np.zeros((height, width, 4), dtype=np.uint8)

    # Calculate bubble size based on text length
    bubble_width = min(300, max(150, len(text) * 10))
    bubble_height = min(130, max(60, (len(text) // 20 + 1) * 30))

    # Draw speech bubble
    cv2.ellipse(bubble, position, (bubble_width, bubble_height), 0, 0, 360, (255, 255, 255, 255), -1)

    # Add a pointer to make it look like a speech bubble
    pointer_x = position[0]
    pointer_y = position[1] + bubble_height
    triangle_pts = np.array([
        [pointer_x - 20, pointer_y - 10],
        [pointer_x + 20, pointer_y - 10],
        [pointer_x, pointer_y + 30]
    ])
    cv2.fillPoly(bubble, [triangle_pts], (255, 255, 255, 255))

    bubble_pil = Image.fromarray(bubble)

    # Draw text
    draw = ImageDraw.Draw(bubble_pil)

    # Try to load a font, fall back to default if not available
    try:
        font = ImageFont.truetype("arial.ttf", 24)
    except:
        try:
            font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 24)
        except:
            font = ImageFont.load_default()

    # Wrap text to fit in the bubble
    wrapped_text = wrap_text(text, font, bubble_width - 40)

    # Calculate text position for centering
    text_x = position[0] - (bubble_width // 2) + 20
    text_y = position[1] - (bubble_height // 2) + 10

    # Draw the wrapped text
    for i, line in enumerate(wrapped_text):
        draw.text((text_x, text_y + i * 30), line, fill="black", font=font)

    # Combine the original image with the bubble
    return Image.alpha_composite(image.convert("RGBA"), bubble_pil)

# Function to wrap text to fit within a certain width
def wrap_text(text, font, max_width):
    words = text.split()
    wrapped_lines = []
    current_line = []

    for word in words:
        # Test if adding this word would exceed the width
        test_line = ' '.join(current_line + [word])
        try:
            # Use getbbox or getlength instead of deprecated getsize
            try:
                bbox = font.getbbox(test_line)
                line_width = bbox[2] - bbox[0]  # right - left
            except AttributeError:
                try:
                    line_width = font.getlength(test_line)
                except AttributeError:
                    line_width = font.getsize(test_line)[0]  # Fallback to deprecated method
        except:
            # If all methods fail, estimate based on character count
            line_width = len(test_line) * 10

        if line_width <= max_width:
            current_line.append(word)
        else:
            wrapped_lines.append(' '.join(current_line))
            current_line = [word]

    # Add the last line
    if current_line:
        wrapped_lines.append(' '.join(current_line))

    return wrapped_lines

# NEW FEATURE: Predefined comic templates
def get_template_data(template_name):
    """Return predefined prompts and dialogue for various comic templates"""
    templates = {
        "batman": {
            "prompts": [
                "Batman standing on a dark Gotham rooftop, his cape flowing in the wind, the Bat-Signal glowing in the sky behind him.",
                "Close-up shot of Batman's determined face, shadows partially covering his mask, eyes glowing in the darkness.",
                "Batman leaping from a rooftop, his cape spread like wings as he descends towards a group of criminals.",
                "Batman delivering a powerful punch to a thug, the action captured in a comic-style motion blur."
            ],
            "dialogues": [
                "Gotham never sleeps, and neither do I.",
                "Justice isn't a choice. It's a responsibility.",
                "Fear is a tool. I use it against them.",
                "Crime has no place in my city."
            ],
            "positions": [(250, 70), (300, 50), (220, 120), (280, 140)],
            "story": "The night was cold in Gotham City. Crime was at an all-time high, and the police were overwhelmed. Standing on the tallest building in the city, Batman observed the dark streets below. The Bat-Signal illuminated the night sky, calling for his help.\n\nBatman knew his duty to the city. With unwavering resolve, he prepared himself for another night of fighting crime. His years of training had made him not just a vigilante, but a symbol of justice.\n\nSpotting a group of armed criminals attempting to rob a bank, Batman leapt from the rooftop. His cape spread like wings as he descended through the night air, becoming the fear that criminals dreaded.\n\nWithin moments, Batman had subdued the criminals. Another night in Gotham, another battle won. But Batman knew his work would never be done as long as there was crime in his city. He was the Dark Knight, Gotham's silent guardian."
        },
        "spiderman": {
            "prompts": [
                "Spider-Man perched on the edge of a skyscraper, overlooking New York City, his iconic red and blue suit vibrant against the skyline.",
                "Spider-Man shooting webs from his wrists, swinging between buildings with great agility.",
                "Spider-Man facing off against a villain, in a dynamic action pose, ready to fight.",
                "Spider-Man helping a civilian, showcasing his friendly neighborhood hero nature."
            ],
            "dialogues": [
                "With great power comes great responsibility.",
                "Just your friendly neighborhood Spider-Man!",
                "My spider-sense is tingling!",
                "I'm not going to let anyone get hurt on my watch."
            ],
            "positions": [(250, 70), (280, 80), (240, 110), (260, 90)],
            "story": "Peter Parker sat perched on the edge of a skyscraper, his Spider-Man mask pulled up just enough to enjoy a quick sandwich. The skyline of New York City stretched out before him, beautiful even in the chaos of the busy afternoon. He reflected on his Uncle Ben's words that had become his guiding principle.\n\nFinishing his lunch, Spider-Man pulled down his mask and leapt off the building, shooting a web that attached to a nearby tower. He swung through the concrete canyons of Manhattan with incredible agility, waving to surprised office workers as he passed their windows.\n\nSuddenly, his spider-sense warned him of danger. Several blocks away, the Rhino was tearing through cars and causing chaos. Spider-Man swung towards the trouble, landing in a fighting stance before the villain, ready to protect the innocent people fleeing the scene.\n\nAfter subduing Rhino with his webs, Spider-Man helped a scared young child find his mother among the crowd. The boy smiled up at his hero, and Spider-Man remembered why he did this every day. Being a hero wasn't about the glory—it was about helping people, one person at a time."
        },
        "wonderwoman": {
            "prompts": [
                "Wonder Woman standing heroically, her golden lasso and bracelets gleaming, an island paradise visible in the background.",
                "Wonder Woman deflecting bullets with her bracelets, her expression determined and fierce.",
                "Wonder Woman wielding her sword and shield, engaged in an epic battle against mythological creatures.",
                "Wonder Woman using her lasso of truth, the golden rope glowing with magical energy."
            ],
            "dialogues": [
                "I am Diana of Themyscira, daughter of Hippolyta.",
                "I fight for those who cannot fight for themselves.",
                "Peace is a virtue to be shown, not peace enforced.",
                "The truth will set you free."
            ],
            "positions": [(250, 70), (270, 60), (230, 100), (260, 80)],
            "story": "The sun rose over Themyscira, illuminating the peaceful island paradise of the Amazons. Diana, princess of the Amazons, stood on a cliff overlooking the sea, her armor gleaming in the morning light. She had been trained as a warrior since birth, but her mission now extended beyond her home.\n\nThat mission brought her to the world of mankind, where conflict raged. In the midst of a battle, Diana stood firm against a hail of bullets, deflecting them with her enchanted bracelets. Her years of training had prepared her for this moment—to protect those who could not protect themselves.\n\nLater, Diana found herself facing ancient monsters awakened from myth, creatures that threatened both gods and humans alike. With sword and shield in hand, she engaged in combat, her movements a blur of strength and grace, demonstrating that true power comes from fighting for what is right.\n\nAfter the battle, Diana used her lasso of truth to extract critical information from an enemy agent. The golden rope glowed with divine energy as she wielded it with compassion and wisdom. Diana knew that while force might win battles, truth and love would ultimately win the war."
        }
    }

    return templates.get(template_name.lower(), None)

def get_available_templates():
    return ["Batman", "Spiderman", "WonderWoman"]

def generate_template_comic(template_name, num_panels=4):
    template_data = get_template_data(template_name.lower())

    if not template_data:
        print(f"Template '{template_name}' not found. Available templates: {', '.join(get_available_templates())}")
        return None, None, None, None

    try:
        pipe = load_sdxl_model()

        print(f"\nGenerating {template_name} comic panels...")
        panel_images = []

        print("Generating base character image...")
        base_prompt = f"A detailed illustration of {template_name} in a dynamic pose, full body visible"

        is_sdxl = isinstance(pipe, StableDiffusionXLPipeline)

        if is_sdxl:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                num_inference_steps=30
            ).images[0]
        else:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                num_inference_steps=30,
                guidance_scale=7.5
            ).images[0]

        base_image = base_image.resize((768, 768))

        prompts = template_data["prompts"][:num_panels]
        dialogues = template_data["dialogues"][:num_panels]
        positions = template_data["positions"][:num_panels]
        story_text = template_data["story"]

        while len(prompts) < num_panels:
            prompts.append(f"{template_name} in an action scene")
        while len(dialogues) < num_panels:
            dialogues.append("")
        while len(positions) < num_panels:
            positions.append((256, 100))

        # If the story needs to be trimmed based on the number of panels requested
        if num_panels < 4:
            story_paragraphs = story_text.split('\n\n')
            story_text = '\n\n'.join(story_paragraphs[:num_panels])

        for i, prompt in enumerate(prompts):
            print(f"Generating image for panel {i+1}...")

            if is_sdxl:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.65,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                    num_inference_steps=30
                ).images[0]
            else:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.65,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                    num_inference_steps=30,
                    guidance_scale=7.5
                ).images[0]

            image = image.resize((512, 512))

            if dialogues[i]:
                image = add_speech_bubble(image, dialogues[i], positions[i])

            panel_images.append(image)

            image_path = f"panel_{i+1}.png"
            image.save(image_path)

        print("Creating combined comic strip...")
        rows = int(np.ceil(num_panels / 2))
        columns = min(2, num_panels)

        final_width = 512 * columns
        final_height = 512 * rows

        comic_strip = Image.new("RGB", (final_width, final_height), (255, 255, 255))

        for i, image in enumerate(panel_images):
            x_offset = (i % 2) * 512
            y_offset = (i // 2) * 512
            comic_strip.paste(image, (x_offset, y_offset))

        comic_strip.save(f"{template_name}_comic_strip.png")
        print(f"{template_name} comic generation complete!")

        # Split the story into panels based on paragraphs
        story_panels = story_text.split('\n\n')[:num_panels]

        # If we have fewer paragraphs than panels, duplicate the last paragraph
        while len(story_panels) < num_panels:
            story_panels.append(story_panels[-1])

        return story_text, story_panels, panel_images, comic_strip

    except Exception as e:
        print(f"Error generating template comic: {e}")
        return None, None, None, None

def generate_comic(story_prompt, num_panels=4, model_path=None, art_style="cartoon style", use_template=None):
    if use_template and use_template.lower() != "none":
        return generate_template_comic(use_template, num_panels)

    if len(story_prompt) < 10:
        story_prompt = f"Create an interesting story about {story_prompt}"

    try:
        print("Loading fine-tuned model...")
        model, tokenizer = load_fine_tuned_model(model_path)

        if model is None or tokenizer is None:
            raise Exception("Failed to load the fine-tuned model")

        print(f"Generating story from prompt: '{story_prompt}'")
        story_text = generate_story(model, tokenizer, story_prompt)
        print("Story generated:")
        print(story_text)

        # If story generation still produces insufficient content after multiple attempts
        # and with different parameters, try generating with a completely different approach
        if not story_text or len(story_text) < 50:
            print("Story generation produced insufficient content. Using alternative generation approach.")
            story_text = generate_alternative_story(story_prompt)
            print("Alternative story generated:")
            print(story_text)

    except Exception as e:
        print(f"Error in model-based story generation: {e}")
        print("Using alternative story generation approach...")
        story_text = generate_alternative_story(story_prompt)

    print("\nSplitting story into panels...")
    story_panels, panel_texts, dialogues = split_story_into_panels(story_text, num_panels)
    for i, panel in enumerate(story_panels):
        print(f"Panel {i+1}: {panel}")
        if dialogues[i]:
            print(f"Dialogue for panel {i+1}: \"{dialogues[i]}\"")

    try:
        pipe = load_custom_model()

        print("\nGenerating panel images...")
        panel_images = []

        character_name = extract_character_name(story_prompt) or "the character"

        print("Generating base character image for consistency...")
        base_prompt = f"A friendly {art_style} illustration of {character_name}, whole body visible, simple background, bright colors, child-friendly"

        is_sdxl = isinstance(pipe, StableDiffusionXLPipeline)

        if is_sdxl:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                num_inference_steps=30
            ).images[0]
        else:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                num_inference_steps=30,
                guidance_scale=7.5
            ).images[0]

        base_image = base_image.resize((768, 768))

        for i, panel_text in enumerate(panel_texts):
            print(f"Generating image for panel {i+1}...")
            # Make the image prompts unique for each panel based on the actual story content
            prompt = f"A friendly {art_style} illustration for a children's storybook showing: {panel_text}, featuring {character_name}, bright colors, simple shapes, cute characters, child-friendly"

            if is_sdxl:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.7,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                    num_inference_steps=30
                ).images[0]
            else:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.7,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                    num_inference_steps=30,
                    guidance_scale=7.5
                ).images[0]

            image = image.resize((512, 512))

            if dialogues[i]:
                position = (256, 100 + (i % 2) * 50)
                image = add_speech_bubble(image, dialogues[i], position)

            panel_images.append(image)

            image_path = f"panel_{i+1}.png"
            image.save(image_path)

        print("Creating combined comic strip...")
        rows = int(np.ceil(num_panels / 2))
        columns = min(2, num_panels)

        final_width = 512 * columns
        final_height = 512 * rows

        comic_strip = Image.new("RGB", (final_width, final_height), (255, 255, 255))

        for i, image in enumerate(panel_images):
            x_offset = (i % 2) * 512
            y_offset = (i // 2) * 512
            comic_strip.paste(image, (x_offset, y_offset))

        comic_strip.save("comic_strip.png")
        print("Comic generation complete! All panels saved as PNG files and combined into comic_strip.png")

        return story_text, story_panels, panel_images, comic_strip

    except Exception as e:
        print(f"Error in image generation: {e}")
        print("Unable to generate images. Please check if you have enough GPU memory and proper installations.")
        return story_text, story_panels, None, None

# Web UI using Gradio
def create_ui():
    with gr.Blocks() as app:
        gr.Markdown("# Story to Comic Generator with InkoloRA")
        gr.Markdown("Enter a prompt to generate a story and convert it into comic panels with speech bubbles using the InkoloRA model")

        with gr.Row():
            with gr.Column():
                # Add template selection dropdown
                template_dropdown = gr.Dropdown(
                    choices=["None"] + get_available_templates(),
                    label="Choose Template (optional)",
                    value="None"
                )

                story_prompt = gr.Textbox(
                    label="Story Prompt",
                    placeholder="Enter a prompt for the children's story (e.g. 'a little boy who learns to fly')",
                    lines=2
                )

                num_panels = gr.Slider(
                    label="Number of Panels",
                    minimum=2,
                    maximum=8,
                    value=4

SyntaxError: incomplete input (<ipython-input-9-db059e05f110>, line 719)

In [ ]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import cv2
from transformers import AutoTokenizer, AutoModelForCausalLM
from diffusers import StableDiffusionPipeline, StableDiffusionXLPipeline
from PIL import Image, ImageDraw, ImageFont
import gradio as gr
from google.colab import drive

# Load the fine-tuned model
def load_fine_tuned_model(model_path=None):
    if not model_path:
        model_path = '/content/drive/MyDrive/fine_tuned_tiny_stories_model'

    try:
        print(f"Loading fine-tuned model from {model_path}...")
        model = AutoModelForCausalLM.from_pretrained(model_path)
        tokenizer = AutoTokenizer.from_pretrained(model_path)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)

        print(f"Model loaded successfully and moved to {device}")
        return model, tokenizer
    except Exception as e:
        print(f"Error loading model: {e}")
        # Try loading a fallback model if the fine-tuned one fails
        try:
            print("Attempting to load fallback model gpt2...")
            model = AutoModelForCausalLM.from_pretrained("gpt2")
            tokenizer = AutoTokenizer.from_pretrained("gpt2")
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model = model.to(device)
            print(f"Fallback model loaded successfully and moved to {device}")
            return model, tokenizer
        except Exception as e2:
            print(f"Error loading fallback model: {e2}")
            return None, None

# Helper functions for story generation
def extract_character_name(prompt):
    character_names = ["Spider-Man", "Peter Parker", "Batman", "Superman", "Wonder Woman",
                       "Iron Man", "Captain America", "Thor", "Hulk", "Black Widow"]

    for name in character_names:
        if name.lower() in prompt.lower():
            return name

    words = prompt.split()
    for word in words:
        if word[0].isupper() and len(word) > 1 and word.lower() not in ["write", "story", "about"]:
            return word

    return None

def extract_topic(prompt):
    cleaned = prompt.lower().replace("write", "").replace("story", "").replace("about", "")
    keywords = ["adventure", "flying", "jumping", "fighting", "learning", "playing", "making friends"]

    for keyword in keywords:
        if keyword in cleaned:
            return keyword

    return cleaned.strip()

def generate_story(model, tokenizer, prompt, max_length=400, retries=3):
    story_prompt = f"Write a simple children's story about {prompt}. Use simple language and short sentences."

    # Increase repetition penalty and adjust parameters for better generation
    for attempt in range(retries):
        try:
            input_ids = tokenizer.encode(story_prompt, return_tensors='pt')
            device = next(model.parameters()).device
            input_ids = input_ids.to(device)

            # Use different generation parameters for each retry attempt
            temperature = 0.7 + (attempt * 0.1)  # Gradually increase temperature with each retry
            repetition_penalty = 1.2 + (attempt * 0.1)  # Gradually increase repetition penalty

            output = model.generate(
                input_ids,
                max_length=max_length,
                num_return_sequences=1,
                no_repeat_ngram_size=3,  # Increased from 2
                top_k=50,  # Increased from 40
                top_p=0.92,  # Adjusted from 0.9
                temperature=temperature,
                do_sample=True,
                repetition_penalty=repetition_penalty
            )

            generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

            # Clean up the generated text to focus on the actual story
            if story_prompt in generated_text:
                generated_text = generated_text.replace(story_prompt, "").strip()

            # Check if we have a reasonable story
            if len(generated_text) > 100 and "." in generated_text:
                print(f"Successfully generated story on attempt {attempt+1}")
                return generated_text
            else:
                print(f"Attempt {attempt+1} produced insufficient content ({len(generated_text)} chars). Retrying...")

        except Exception as e:
            print(f"Error in story generation attempt {attempt+1}: {e}")

    # If all retries fail, generate a more varied story using a different approach
    print("All generation attempts failed. Using alternative generation method...")
    return generate_alternative_story(prompt)

def generate_alternative_story(prompt):
    """
    Generate a more varied story when the model-based generation fails.
    This creates a more dynamic story than the previous structured approach.
    """
    character = extract_character_name(prompt) or "the little hero"
    topic = extract_topic(prompt) or "having an adventure"

    # Create a list of possible story templates
    story_templates = [
        f"""Once upon a time, there was {character} who lived in a small village.
{character} loved {topic} more than anything else.

One day, {character} discovered something magical while playing outside. It was a tiny glowing ball that could float in the air.

"What's this?" wondered {character}. The ball started to move, and {character} followed it into the forest.

The ball led {character} to a hidden garden where other children were playing with similar magical balls. They welcomed {character} and taught them new games.

{character} had so much fun that day! When it was time to go home, one of the children gave {character} a magical ball of their own.

From that day on, {character} visited the hidden garden often and made many new friends. They all loved {topic} together and had wonderful adventures.""",

        f"""{character} woke up early one morning with a big smile. Today was the day for {topic}!

{character} packed a small bag with snacks and a water bottle. "I'm ready for my adventure!" {character} said.

Outside, {character} met a friendly cat. "Hello, cat! Would you like to join me for {topic}?" The cat meowed and followed along.

As they walked through the park, they found a lost teddy bear. "We should find who this belongs to," said {character}.

They asked everyone they met about the teddy bear. Finally, they met a little girl who was crying. "My teddy!" she said happily when she saw it.

"Thank you for helping me find my teddy," the girl smiled. "Would you like to play with me?"

{character}, the cat, and the girl spent the whole day playing and having fun with {topic}. It was the best day ever!""",

        f"""The sun was shining brightly when {character} decided it was the perfect day for {topic}.

"I've never tried {topic} before," {character} thought, "but today I will be brave!"

{character} began slowly, taking small steps. It wasn't easy at first, and {character} almost gave up.

"Don't worry, you can do it!" said a friendly voice. {character} looked up and saw a smiling butterfly.

With the butterfly's encouragement, {character} tried again. This time, {character} did much better!

Soon, {character} was having so much fun with {topic}. "Thank you for believing in me," {character} said to the butterfly.

The butterfly fluttered around happily. "Sometimes we just need a friend to help us be brave," it replied.

{character} went home that evening with a happy heart, excited to try {topic} again tomorrow."""
    ]

    import random
    return random.choice(story_templates)

def split_story_into_panels(story, num_panels=4):
    if len(story) < 100:
        print("Warning: Story is very short. Panel division may not be optimal.")

    paragraphs = story.split('\n')
    paragraphs = [p for p in paragraphs if p.strip()]

    dialogues = []
    panel_texts = []

    if len(paragraphs) >= num_panels:
        if len(paragraphs) > num_panels:
            step = len(paragraphs) / num_panels
            panels = []
            for i in range(num_panels):
                start_idx = int(i * step)
                end_idx = int((i + 1) * step) if i < num_panels - 1 else len(paragraphs)
                panel_text = " ".join(paragraphs[start_idx:end_idx])
                panels.append(panel_text)

                dialogue = extract_dialogue(panel_text)
                dialogues.append(dialogue)

                clean_text = panel_text.replace(dialogue, "") if dialogue else panel_text
                panel_texts.append(clean_text)
            return panels, panel_texts, dialogues
        else:
            panels = paragraphs[:num_panels]
            for panel in panels:
                dialogue = extract_dialogue(panel)
                dialogues.append(dialogue)
                clean_text = panel.replace(dialogue, "") if dialogue else panel
                panel_texts.append(clean_text)
            return panels, panel_texts, dialogues

    sentences = []
    for chunk in story.split('. '):
        sentences.extend([s.strip() + '.' for s in chunk.split('! ')])
    sentences = [s for s in sentences if len(s) > 1]

    if len(sentences) < num_panels:
        while len(sentences) < num_panels:
            longest_idx = max(range(len(sentences)), key=lambda i: len(sentences[i]))
            longest = sentences[longest_idx]

            if ', ' in longest:
                parts = longest.split(', ', 1)
                sentences[longest_idx] = parts[0] + '.'
                sentences.insert(longest_idx + 1, parts[1])
            else:
                sentences.append(f"Meanwhile, {longest.lower()}")

    step = max(1, len(sentences) // num_panels)

    panels = []
    for i in range(num_panels):
        start_idx = i * step
        # For the last panel, include any remaining sentences
        end_idx = (i + 1) * step if i < num_panels - 1 else len(sentences)
        panel_text = " ".join(sentences[start_idx:end_idx])
        panels.append(panel_text)

        dialogue = extract_dialogue(panel_text)
        dialogues.append(dialogue)

        clean_text = panel_text.replace(dialogue, "") if dialogue else panel_text
        panel_texts.append(clean_text)

    return panels, panel_texts, dialogues

def extract_dialogue(text):
    import re
    dialogue_matches = re.findall(r'"([^"]*)"', text)
    if dialogue_matches:
        # Return the first dialogue found
        return dialogue_matches[0]
    return ""

def load_custom_model():
    print("Loading InkoloRA model - this may take a while...")

    custom_model_path = "/content/drive/MyDrive/InkoloRA.safetensors"

    if not os.path.exists(custom_model_path):
        print(f"Warning: Custom model not found at {custom_model_path}. Falling back to Stable Diffusion XL.")
        return load_sdxl_model()

    try:
        # Load the model - adjust the model_id based on what base model your safetensors file is trained on
        # Assuming it's SD 1.5 based; modify if it's based on a different architecture
        pipe = StableDiffusionPipeline.from_pretrained(
            "runwayml/stable-diffusion-v1-5",
            torch_dtype=torch.float16,
            safety_checker=None
        )

        # Load the custom weights
        pipe.unet.load_attn_procs(custom_model_path)

        # Check if CUDA is available, otherwise use CPU
        device = "cuda" if torch.cuda.is_available() else "cpu"
        if device == "cpu":
            print("Warning: CUDA not available, using CPU. This will be very slow.")
        pipe = pipe.to(device)

        print("InkoloRA model loaded successfully!")
        return pipe
    except Exception as e:
        print(f"Error loading custom model: {e}")
        print("Falling back to Stable Diffusion XL...")
        return load_sdxl_model()

# Original SDXL model load function as fallback
def load_sdxl_model():
    print("Loading SDXL model - this may take a while...")
    pipe = StableDiffusionXLPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True
    )

    # Check if CUDA is available, otherwise use CPU
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device == "cpu":
        print("Warning: CUDA not available, using CPU. This will be very slow.")
    pipe = pipe.to(device)

    return pipe

# Function to add speech bubbles to images
def add_speech_bubble(image, text, position=None):
    if not text:
        return image

    # Convert PIL image to numpy array for OpenCV operations
    img = np.array(image.convert("RGBA"))
    height, width, _ = img.shape

    # If position not provided, place the bubble at a suitable position
    if position is None:
        position = (width // 2, height // 4)

    # Create a transparent layer for the bubble
    bubble = np.zeros((height, width, 4), dtype=np.uint8)

    # Calculate bubble size based on text length
    bubble_width = min(300, max(150, len(text) * 10))
    bubble_height = min(130, max(60, (len(text) // 20 + 1) * 30))

    # Draw speech bubble
    cv2.ellipse(bubble, position, (bubble_width, bubble_height), 0, 0, 360, (255, 255, 255, 255), -1)

    # Add a pointer to make it look like a speech bubble
    pointer_x = position[0]
    pointer_y = position[1] + bubble_height
    triangle_pts = np.array([
        [pointer_x - 20, pointer_y - 10],
        [pointer_x + 20, pointer_y - 10],
        [pointer_x, pointer_y + 30]
    ])
    cv2.fillPoly(bubble, [triangle_pts], (255, 255, 255, 255))

    bubble_pil = Image.fromarray(bubble)

    # Draw text
    draw = ImageDraw.Draw(bubble_pil)

    # Try to load a font, fall back to default if not available
    try:
        font = ImageFont.truetype("arial.ttf", 24)
    except:
        try:
            font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 24)
        except:
            font = ImageFont.load_default()

    # Wrap text to fit in the bubble
    wrapped_text = wrap_text(text, font, bubble_width - 40)

    # Calculate text position for centering
    text_x = position[0] - (bubble_width // 2) + 20
    text_y = position[1] - (bubble_height // 2) + 10

    # Draw the wrapped text
    for i, line in enumerate(wrapped_text):
        draw.text((text_x, text_y + i * 30), line, fill="black", font=font)

    # Combine the original image with the bubble
    return Image.alpha_composite(image.convert("RGBA"), bubble_pil)

# Function to wrap text to fit within a certain width
def wrap_text(text, font, max_width):
    words = text.split()
    wrapped_lines = []
    current_line = []

    for word in words:
        # Test if adding this word would exceed the width
        test_line = ' '.join(current_line + [word])
        try:
            # Use getbbox or getlength instead of deprecated getsize
            try:
                bbox = font.getbbox(test_line)
                line_width = bbox[2] - bbox[0]  # right - left
            except AttributeError:
                try:
                    line_width = font.getlength(test_line)
                except AttributeError:
                    line_width = font.getsize(test_line)[0]  # Fallback to deprecated method
        except:
            # If all methods fail, estimate based on character count
            line_width = len(test_line) * 10

        if line_width <= max_width:
            current_line.append(word)
        else:
            wrapped_lines.append(' '.join(current_line))
            current_line = [word]

    # Add the last line
    if current_line:
        wrapped_lines.append(' '.join(current_line))

    return wrapped_lines

# NEW FEATURE: Predefined comic templates
def get_template_data(template_name):
    """Return predefined prompts and dialogue for various comic templates"""
    templates = {
        "batman": {
            "prompts": [
                "Batman standing on a dark Gotham rooftop, his cape flowing in the wind, the Bat-Signal glowing in the sky behind him.",
                "Close-up shot of Batman's determined face, shadows partially covering his mask, eyes glowing in the darkness.",
                "Batman leaping from a rooftop, his cape spread like wings as he descends towards a group of criminals.",
                "Batman delivering a powerful punch to a thug, the action captured in a comic-style motion blur."
            ],
            "dialogues": [
                "Gotham never sleeps, and neither do I.",
                "Justice isn't a choice. It's a responsibility.",
                "Fear is a tool. I use it against them.",
                "Crime has no place in my city."
            ],
            "positions": [(250, 70), (300, 50), (220, 120), (280, 140)],
            "story": "The night was cold in Gotham City. Crime was at an all-time high, and the police were overwhelmed. Standing on the tallest building in the city, Batman observed the dark streets below. The Bat-Signal illuminated the night sky, calling for his help.\n\nBatman knew his duty to the city. With unwavering resolve, he prepared himself for another night of fighting crime. His years of training had made him not just a vigilante, but a symbol of justice.\n\nSpotting a group of armed criminals attempting to rob a bank, Batman leapt from the rooftop. His cape spread like wings as he descended through the night air, becoming the fear that criminals dreaded.\n\nWithin moments, Batman had subdued the criminals. Another night in Gotham, another battle won. But Batman knew his work would never be done as long as there was crime in his city. He was the Dark Knight, Gotham's silent guardian."
        },
        "spiderman": {
            "prompts": [
                "Spider-Man perched on the edge of a skyscraper, overlooking New York City, his iconic red and blue suit vibrant against the skyline.",
                "Spider-Man shooting webs from his wrists, swinging between buildings with great agility.",
                "Spider-Man facing off against a villain, in a dynamic action pose, ready to fight.",
                "Spider-Man helping a civilian, showcasing his friendly neighborhood hero nature."
            ],
            "dialogues": [
                "With great power comes great responsibility.",
                "Just your friendly neighborhood Spider-Man!",
                "My spider-sense is tingling!",
                "I'm not going to let anyone get hurt on my watch."
            ],
            "positions": [(250, 70), (280, 80), (240, 110), (260, 90)],
            "story": "Peter Parker sat perched on the edge of a skyscraper, his Spider-Man mask pulled up just enough to enjoy a quick sandwich. The skyline of New York City stretched out before him, beautiful even in the chaos of the busy afternoon. He reflected on his Uncle Ben's words that had become his guiding principle.\n\nFinishing his lunch, Spider-Man pulled down his mask and leapt off the building, shooting a web that attached to a nearby tower. He swung through the concrete canyons of Manhattan with incredible agility, waving to surprised office workers as he passed their windows.\n\nSuddenly, his spider-sense warned him of danger. Several blocks away, the Rhino was tearing through cars and causing chaos. Spider-Man swung towards the trouble, landing in a fighting stance before the villain, ready to protect the innocent people fleeing the scene.\n\nAfter subduing Rhino with his webs, Spider-Man helped a scared young child find his mother among the crowd. The boy smiled up at his hero, and Spider-Man remembered why he did this every day. Being a hero wasn't about the glory—it was about helping people, one person at a time."
        },
        "wonderwoman": {
            "prompts": [
                "Wonder Woman standing heroically, her golden lasso and bracelets gleaming, an island paradise visible in the background.",
                "Wonder Woman deflecting bullets with her bracelets, her expression determined and fierce.",
                "Wonder Woman wielding her sword and shield, engaged in an epic battle against mythological creatures.",
                "Wonder Woman using her lasso of truth, the golden rope glowing with magical energy."
            ],
            "dialogues": [
                "I am Diana of Themyscira, daughter of Hippolyta.",
                "I fight for those who cannot fight for themselves.",
                "Peace is a virtue to be shown, not peace enforced.",
                "The truth will set you free."
            ],
            "positions": [(250, 70), (270, 60), (230, 100), (260, 80)],
            "story": "The sun rose over Themyscira, illuminating the peaceful island paradise of the Amazons. Diana, princess of the Amazons, stood on a cliff overlooking the sea, her armor gleaming in the morning light. She had been trained as a warrior since birth, but her mission now extended beyond her home.\n\nThat mission brought her to the world of mankind, where conflict raged. In the midst of a battle, Diana stood firm against a hail of bullets, deflecting them with her enchanted bracelets. Her years of training had prepared her for this moment—to protect those who could not protect themselves.\n\nLater, Diana found herself facing ancient monsters awakened from myth, creatures that threatened both gods and humans alike. With sword and shield in hand, she engaged in combat, her movements a blur of strength and grace, demonstrating that true power comes from fighting for what is right.\n\nAfter the battle, Diana used her lasso of truth to extract critical information from an enemy agent. The golden rope glowed with divine energy as she wielded it with compassion and wisdom. Diana knew that while force might win battles, truth and love would ultimately win the war."
        }
    }

    return templates.get(template_name.lower(), None)

def get_available_templates():
    return ["Batman", "Spiderman", "WonderWoman"]

def generate_template_comic(template_name, num_panels=4):
    template_data = get_template_data(template_name.lower())

    if not template_data:
        print(f"Template '{template_name}' not found. Available templates: {', '.join(get_available_templates())}")
        return None, None, None, None

    try:
        pipe = load_sdxl_model()

        print(f"\nGenerating {template_name} comic panels...")
        panel_images = []

        print("Generating base character image...")
        base_prompt = f"A detailed illustration of {template_name} in a dynamic pose, full body visible"

        is_sdxl = isinstance(pipe, StableDiffusionXLPipeline)

        if is_sdxl:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                num_inference_steps=30
            ).images[0]
        else:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                num_inference_steps=30,
                guidance_scale=7.5
            ).images[0]

        base_image = base_image.resize((768, 768))

        prompts = template_data["prompts"][:num_panels]
        dialogues = template_data["dialogues"][:num_panels]
        positions = template_data["positions"][:num_panels]
        story_text = template_data["story"]

        while len(prompts) < num_panels:
            prompts.append(f"{template_name} in an action scene")
        while len(dialogues) < num_panels:
            dialogues.append("")
        while len(positions) < num_panels:
            positions.append((256, 100))

        # If the story needs to be trimmed based on the number of panels requested
        if num_panels < 4:
            story_paragraphs = story_text.split('\n\n')
            story_text = '\n\n'.join(story_paragraphs[:num_panels])

        for i, prompt in enumerate(prompts):
            print(f"Generating image for panel {i+1}...")

            if is_sdxl:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.65,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                    num_inference_steps=30
                ).images[0]
            else:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.65,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                    num_inference_steps=30,
                    guidance_scale=7.5
                ).images[0]

            image = image.resize((512, 512))

            if dialogues[i]:
                image = add_speech_bubble(image, dialogues[i], positions[i])

            panel_images.append(image)

            image_path = f"panel_{i+1}.png"
            image.save(image_path)

        print("Creating combined comic strip...")
        rows = int(np.ceil(num_panels / 2))
        columns = min(2, num_panels)

        final_width = 512 * columns
        final_height = 512 * rows

        comic_strip = Image.new("RGB", (final_width, final_height), (255, 255, 255))

        for i, image in enumerate(panel_images):
            x_offset = (i % 2) * 512
            y_offset = (i // 2) * 512
            comic_strip.paste(image, (x_offset, y_offset))

        comic_strip.save(f"{template_name}_comic_strip.png")
        print(f"{template_name} comic generation complete!")

        # Split the story into panels based on paragraphs
        story_panels = story_text.split('\n\n')[:num_panels]

        # If we have fewer paragraphs than panels, duplicate the last paragraph
        while len(story_panels) < num_panels:
            story_panels.append(story_panels[-1])

        return story_text, story_panels, panel_images, comic_strip

    except Exception as e:
        print(f"Error generating template comic: {e}")
        return None, None, None, None

def generate_comic(story_prompt, num_panels=4, model_path=None, art_style="cartoon style", use_template=None):
    if use_template and use_template.lower() != "none":
        return generate_template_comic(use_template, num_panels)

    if len(story_prompt) < 10:
        story_prompt = f"Create an interesting story about {story_prompt}"

    try:
        print("Loading fine-tuned model...")
        model, tokenizer = load_fine_tuned_model(model_path)

        if model is None or tokenizer is None:
            raise Exception("Failed to load the fine-tuned model")

        print(f"Generating story from prompt: '{story_prompt}'")
        story_text = generate_story(model, tokenizer, story_prompt)
        print("Story generated:")
        print(story_text)

        # If story generation still produces insufficient content after multiple attempts
        # and with different parameters, try generating with a completely different approach
        if not story_text or len(story_text) < 50:
            print("Story generation produced insufficient content. Using alternative generation approach.")
            story_text = generate_alternative_story(story_prompt)
            print("Alternative story generated:")
            print(story_text)

    except Exception as e:
        print(f"Error in model-based story generation: {e}")
        print("Using alternative story generation approach...")
        story_text = generate_alternative_story(story_prompt)

    print("\nSplitting story into panels...")
    story_panels, panel_texts, dialogues = split_story_into_panels(story_text, num_panels)
    for i, panel in enumerate(story_panels):
        print(f"Panel {i+1}: {panel}")
        if dialogues[i]:
            print(f"Dialogue for panel {i+1}: \"{dialogues[i]}\"")

    try:
        pipe = load_custom_model()

        print("\nGenerating panel images...")
        panel_images = []

        character_name = extract_character_name(story_prompt) or "the character"

        print("Generating base character image for consistency...")
        base_prompt = f"A friendly {art_style} illustration of {character_name}, whole body visible, simple background, bright colors, child-friendly"

        is_sdxl = isinstance(pipe, StableDiffusionXLPipeline)

        if is_sdxl:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                num_inference_steps=30
            ).images[0]
        else:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                num_inference_steps=30,
                guidance_scale=7.5
            ).images[0]

        base_image = base_image.resize((768, 768))

        for i, panel_text in enumerate(panel_texts):
            print(f"Generating image for panel {i+1}...")
            # Make the image prompts unique for each panel based on the actual story content
            prompt = f"A friendly {art_style} illustration for a children's storybook showing: {panel_text}, featuring {character_name}, bright colors, simple shapes, cute characters, child-friendly"

            if is_sdxl:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.7,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                    num_inference_steps=30
                ).images[0]
            else:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.7,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                    num_inference_steps=30,
                    guidance_scale=7.5
                ).images[0]

            image = image.resize((512, 512))

            if dialogues[i]:
                position = (256, 100 + (i % 2) * 50)
                image = add_speech_bubble(image, dialogues[i], position)

            panel_images.append(image)

            image_path = f"panel_{i+1}.png"
            image.save(image_path)

        print("Creating combined comic strip...")
        rows = int(np.ceil(num_panels / 2))
        columns = min(2, num_panels)

        final_width = 512 * columns
        final_height = 512 * rows

        comic_strip = Image.new("RGB", (final_width, final_height), (255, 255, 255))

        for i, image in enumerate(panel_images):
            x_offset = (i % 2) * 512
            y_offset = (i // 2) * 512
            comic_strip.paste(image, (x_offset, y_offset))

        comic_strip.save("comic_strip.png")
        print("Comic generation complete! All panels saved as PNG files and combined into comic_strip.png")

        return story_text, story_panels, panel_images, comic_strip

    except Exception as e:
        print(f"Error in image generation: {e}")
        print("Unable to generate images. Please check if you have enough GPU memory and proper installations.")
        return story_text, story_panels, None, None

# Web UI using Gradio
def create_ui():
    with gr.Blocks() as app:
        gr.Markdown("# Story to Comic Generator with InkoloRA")
        gr.Markdown("Enter a prompt to generate a story and convert it into comic panels with speech bubbles using the InkoloRA model")

        with gr.Row():
            with gr.Column():
                # Add template selection dropdown
                template_dropdown = gr.Dropdown(
                    choices=["None"] + get_available_templates(),
                    label="Choose Template (optional)",
                    value="None"
                )

                story_prompt = gr.Textbox(
                    label="Story Prompt",
                    placeholder="Enter a prompt for the children's story (e.g. 'a little boy who learns to

SyntaxError: unterminated string literal (detected at line 711) (<ipython-input-10-a99ce5ee10c1>, line 711)

In [ ]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import cv2
from transformers import AutoTokenizer, AutoModelForCausalLM
from diffusers import StableDiffusionPipeline, StableDiffusionXLPipeline
from PIL import Image, ImageDraw, ImageFont
import gradio as gr
from google.colab import drive

# Load the fine-tuned model
def load_fine_tuned_model(model_path=None):
    if not model_path:
        model_path = '/content/drive/MyDrive/fine_tuned_tiny_stories_model'

    try:
        print(f"Loading fine-tuned model from {model_path}...")
        model = AutoModelForCausalLM.from_pretrained(model_path)
        tokenizer = AutoTokenizer.from_pretrained(model_path)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)

        print(f"Model loaded successfully and moved to {device}")
        return model, tokenizer
    except Exception as e:
        print(f"Error loading model: {e}")
        # Try loading a fallback model if the fine-tuned one fails
        try:
            print("Attempting to load fallback model gpt2...")
            model = AutoModelForCausalLM.from_pretrained("gpt2")
            tokenizer = AutoTokenizer.from_pretrained("gpt2")
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model = model.to(device)
            print(f"Fallback model loaded successfully and moved to {device}")
            return model, tokenizer
        except Exception as e2:
            print(f"Error loading fallback model: {e2}")
            return None, None

# Helper functions for story generation
def extract_character_name(prompt):
    character_names = ["Spider-Man", "Peter Parker", "Batman", "Superman", "Wonder Woman",
                       "Iron Man", "Captain America", "Thor", "Hulk", "Black Widow"]

    for name in character_names:
        if name.lower() in prompt.lower():
            return name

    words = prompt.split()
    for word in words:
        if word[0].isupper() and len(word) > 1 and word.lower() not in ["write", "story", "about"]:
            return word

    return None

def extract_topic(prompt):
    cleaned = prompt.lower().replace("write", "").replace("story", "").replace("about", "")
    keywords = ["adventure", "flying", "jumping", "fighting", "learning", "playing", "making friends"]

    for keyword in keywords:
        if keyword in cleaned:
            return keyword

    return cleaned.strip()

def generate_story(model, tokenizer, prompt, max_length=400, retries=3):
    story_prompt = f"Write a simple children's story about {prompt}. Use simple language and short sentences."

    # Increase repetition penalty and adjust parameters for better generation
    for attempt in range(retries):
        try:
            input_ids = tokenizer.encode(story_prompt, return_tensors='pt')
            device = next(model.parameters()).device
            input_ids = input_ids.to(device)

            # Use different generation parameters for each retry attempt
            temperature = 0.7 + (attempt * 0.1)  # Gradually increase temperature with each retry
            repetition_penalty = 1.2 + (attempt * 0.1)  # Gradually increase repetition penalty

            output = model.generate(
                input_ids,
                max_length=max_length,
                num_return_sequences=1,
                no_repeat_ngram_size=3,  # Increased from 2
                top_k=50,  # Increased from 40
                top_p=0.92,  # Adjusted from 0.9
                temperature=temperature,
                do_sample=True,
                repetition_penalty=repetition_penalty
            )

            generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

            # Clean up the generated text to focus on the actual story
            if story_prompt in generated_text:
                generated_text = generated_text.replace(story_prompt, "").strip()

            # Check if we have a reasonable story
            if len(generated_text) > 100 and "." in generated_text:
                print(f"Successfully generated story on attempt {attempt+1}")
                return generated_text
            else:
                print(f"Attempt {attempt+1} produced insufficient content ({len(generated_text)} chars). Retrying...")

        except Exception as e:
            print(f"Error in story generation attempt {attempt+1}: {e}")

    # If all retries fail, generate a more varied story using a different approach
    print("All generation attempts failed. Using alternative generation method...")
    return generate_alternative_story(prompt)

def generate_alternative_story(prompt):
    """
    Generate a more varied story when the model-based generation fails.
    This creates a more dynamic story than the previous structured approach.
    """
    character = extract_character_name(prompt) or "the little hero"
    topic = extract_topic(prompt) or "having an adventure"

    # Create a list of possible story templates
    story_templates = [
        f"""Once upon a time, there was {character} who lived in a small village.
{character} loved {topic} more than anything else.

One day, {character} discovered something magical while playing outside. It was a tiny glowing ball that could float in the air.

"What's this?" wondered {character}. The ball started to move, and {character} followed it into the forest.

The ball led {character} to a hidden garden where other children were playing with similar magical balls. They welcomed {character} and taught them new games.

{character} had so much fun that day! When it was time to go home, one of the children gave {character} a magical ball of their own.

From that day on, {character} visited the hidden garden often and made many new friends. They all loved {topic} together and had wonderful adventures.""",

        f"""{character} woke up early one morning with a big smile. Today was the day for {topic}!

{character} packed a small bag with snacks and a water bottle. "I'm ready for my adventure!" {character} said.

Outside, {character} met a friendly cat. "Hello, cat! Would you like to join me for {topic}?" The cat meowed and followed along.

As they walked through the park, they found a lost teddy bear. "We should find who this belongs to," said {character}.

They asked everyone they met about the teddy bear. Finally, they met a little girl who was crying. "My teddy!" she said happily when she saw it.

"Thank you for helping me find my teddy," the girl smiled. "Would you like to play with me?"

{character}, the cat, and the girl spent the whole day playing and having fun with {topic}. It was the best day ever!""",

        f"""The sun was shining brightly when {character} decided it was the perfect day for {topic}.

"I've never tried {topic} before," {character} thought, "but today I will be brave!"

{character} began slowly, taking small steps. It wasn't easy at first, and {character} almost gave up.

"Don't worry, you can do it!" said a friendly voice. {character} looked up and saw a smiling butterfly.

With the butterfly's encouragement, {character} tried again. This time, {character} did much better!

Soon, {character} was having so much fun with {topic}. "Thank you for believing in me," {character} said to the butterfly.

The butterfly fluttered around happily. "Sometimes we just need a friend to help us be brave," it replied.

{character} went home that evening with a happy heart, excited to try {topic} again tomorrow."""
    ]

    import random
    return random.choice(story_templates)

def split_story_into_panels(story, num_panels=4):
    if len(story) < 100:
        print("Warning: Story is very short. Panel division may not be optimal.")

    paragraphs = story.split('\n')
    paragraphs = [p for p in paragraphs if p.strip()]

    dialogues = []
    panel_texts = []

    if len(paragraphs) >= num_panels:
        if len(paragraphs) > num_panels:
            step = len(paragraphs) / num_panels
            panels = []
            for i in range(num_panels):
                start_idx = int(i * step)
                end_idx = int((i + 1) * step) if i < num_panels - 1 else len(paragraphs)
                panel_text = " ".join(paragraphs[start_idx:end_idx])
                panels.append(panel_text)

                dialogue = extract_dialogue(panel_text)
                dialogues.append(dialogue)

                clean_text = panel_text.replace(dialogue, "") if dialogue else panel_text
                panel_texts.append(clean_text)
            return panels, panel_texts, dialogues
        else:
            panels = paragraphs[:num_panels]
            for panel in panels:
                dialogue = extract_dialogue(panel)
                dialogues.append(dialogue)
                clean_text = panel.replace(dialogue, "") if dialogue else panel
                panel_texts.append(clean_text)
            return panels, panel_texts, dialogues

    sentences = []
    for chunk in story.split('. '):
        sentences.extend([s.strip() + '.' for s in chunk.split('! ')])
    sentences = [s for s in sentences if len(s) > 1]

    if len(sentences) < num_panels:
        while len(sentences) < num_panels:
            longest_idx = max(range(len(sentences)), key=lambda i: len(sentences[i]))
            longest = sentences[longest_idx]

            if ', ' in longest:
                parts = longest.split(', ', 1)
                sentences[longest_idx] = parts[0] + '.'
                sentences.insert(longest_idx + 1, parts[1])
            else:
                sentences.append(f"Meanwhile, {longest.lower()}")

    step = max(1, len(sentences) // num_panels)

    panels = []
    for i in range(num_panels):
        start_idx = i * step
        end_idx = (i + 1) * step if i < num_panels - 1 else len(sentences)
        panel_text = " ".join(sentences[start_idx:end_idx])
        panels.append(panel_text)

        dialogue = extract_dialogue(panel_text)
        dialogues.append(dialogue)

        clean_text = panel_text.replace(dialogue, "") if dialogue else panel_text
        panel_texts.append(clean_text)

    return panels, panel_texts, dialogues

def extract_dialogue(text):
    import re
    dialogue_matches = re.findall(r'"([^"]*)"', text)
    if dialogue_matches:
        return dialogue_matches[0]
    return ""

def load_custom_model():
    print("Loading InkoloRA model - this may take a while...")

    custom_model_path = "/content/drive/MyDrive/InkoloRA.safetensors"

    if not os.path.exists(custom_model_path):
        print(f"Warning: Custom model not found at {custom_model_path}. Falling back to Stable Diffusion XL.")
        return load_sdxl_model()

    try:
        pipe = StableDiffusionPipeline.from_pretrained(
            "runwayml/stable-diffusion-v1-5",
            torch_dtype=torch.float16,
            safety_checker=None
        )

        pipe.unet.load_attn_procs(custom_model_path)

        device = "cuda" if torch.cuda.is_available() else "cpu"
        if device == "cpu":
            print("Warning: CUDA not available, using CPU. This will be very slow.")
        pipe = pipe.to(device)

        print("InkoloRA model loaded successfully!")
        return pipe
    except Exception as e:
        print(f"Error loading custom model: {e}")
        print("Falling back to Stable Diffusion XL...")
        return load_sdxl_model()

def load_sdxl_model():
    print("Loading SDXL model - this may take a while...")
    pipe = StableDiffusionXLPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True
    )

    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device == "cpu":
        print("Warning: CUDA not available, using CPU. This will be very slow.")
    pipe = pipe.to(device)

    return pipe

def add_speech_bubble(image, text, position=None):
    if not text:
        return image

    img = np.array(image.convert("RGBA"))
    height, width, _ = img.shape

    if position is None:
        position = (width // 2, height // 4)

    bubble = np.zeros((height, width, 4), dtype=np.uint8)

    bubble_width = min(300, max(150, len(text) * 10))
    bubble_height = min(130, max(60, (len(text) // 20 + 1) * 30))

    cv2.ellipse(bubble, position, (bubble_width, bubble_height), 0, 0, 360, (255, 255, 255, 255), -1)

    pointer_x = position[0]
    pointer_y = position[1] + bubble_height
    triangle_pts = np.array([
        [pointer_x - 20, pointer_y - 10],
        [pointer_x + 20, pointer_y - 10],
        [pointer_x, pointer_y + 30]
    ])
    cv2.fillPoly(bubble, [triangle_pts], (255, 255, 255, 255))

    bubble_pil = Image.fromarray(bubble)

    draw = ImageDraw.Draw(bubble_pil)

    try:
        font = ImageFont.truetype("arial.ttf", 24)
    except:
        try:
            font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 24)
        except:
            font = ImageFont.load_default()

    wrapped_text = wrap_text(text, font, bubble_width - 40)

    text_x = position[0] - (bubble_width // 2) + 20
    text_y = position[1] - (bubble_height // 2) + 10

    for i, line in enumerate(wrapped_text):
        draw.text((text_x, text_y + i * 30), line, fill="black", font=font)

    return Image.alpha_composite(image.convert("RGBA"), bubble_pil)

def wrap_text(text, font, max_width):
    words = text.split()
    wrapped_lines = []
    current_line = []

    for word in words:
        test_line = ' '.join(current_line + [word])
        try:
            try:
                bbox = font.getbbox(test_line)
                line_width = bbox[2] - bbox[0]
            except AttributeError:
                try:
                    line_width = font.getlength(test_line)
                except AttributeError:
                    line_width = font.getsize(test_line)[0]
        except:
            line_width = len(test_line) * 10

        if line_width <= max_width:
            current_line.append(word)
        else:
            wrapped_lines.append(' '.join(current_line))
            current_line = [word]

    if current_line:
        wrapped_lines.append(' '.join(current_line))

    return wrapped_lines

def get_template_data(template_name):
    templates = {
        "batman": {
            "prompts": [
                "Batman standing on a dark Gotham rooftop, his cape flowing in the wind, the Bat-Signal glowing in the sky behind him.",
                "Close-up shot of Batman's determined face, shadows partially covering his mask, eyes glowing in the darkness.",
                "Batman leaping from a rooftop, his cape spread like wings as he descends towards a group of criminals.",
                "Batman delivering a powerful punch to a thug, the action captured in a comic-style motion blur."
            ],
            "dialogues": [
                "Gotham never sleeps, and neither do I.",
                "Justice isn't a choice. It's a responsibility.",
                "Fear is a tool. I use it against them.",
                "Crime has no place in my city."
            ],
            "positions": [(250, 70), (300, 50), (220, 120), (280, 140)],
            "story": """The Dark Knight watches over Gotham City from high above. Tonight is like any other night for Batman - a city full of shadows that need his protection.

As the Bat-Signal illuminates the cloudy sky, Batman's eyes narrow with determination. The signal isn't just a call for help - it's a reminder of the oath he took to protect the innocent.

Batman spots a group of criminals attempting to rob a local shop. He leaps from the rooftop, his cape billowing around him like massive wings. The criminals look up in terror as his silhouette blots out the moonlight.

With swift, calculated movements, Batman takes down the criminals one by one. Another night in Gotham, another battle won. But Batman knows his work is never truly done. As long as there is crime in Gotham, he will be there to fight it."""
        },
        "spiderman": {
            "prompts": [
                "Spider-Man perched on the edge of a skyscraper, overlooking New York City, his iconic red and blue suit vibrant against the skyline.",
                "Spider-Man shooting webs from his wrists, swinging between buildings with great agility.",
                "Spider-Man facing off against a villain, in a dynamic action pose, ready to fight.",
                "Spider-Man helping a civilian, showcasing his friendly neighborhood hero nature."
            ],
            "dialogues": [
                "With great power comes great responsibility.",
                "Just your friendly neighborhood Spider-Man!",
                "My spider-sense is tingling!",
                "I'm not going to let anyone get hurt on my watch."
            ],
            "positions": [(250, 70), (280, 80), (240, 110), (260, 90)],
            "story": """Peter Parker sits perched on the edge of a towering skyscraper, overlooking the bustling streets of New York City. His uncle Ben's words echo in his mind as he surveys the city he's sworn to protect.

With a flick of his wrist, Spider-Man launches himself off the building, shooting a web that attaches to a nearby structure. He swings gracefully through the concrete canyons of Manhattan, the wind rushing past his mask as pedestrians below point and cheer.

Suddenly, his spider-sense alerts him to danger. A bank robbery is in progress just a few blocks away. Spider-Man changes direction mid-swing, heading straight for the trouble. As he arrives, he sees masked robbers threatening innocent civilians.

With swift movements and well-placed webs, Spider-Man subdues the criminals and ensures everyone is safe. A young child approaches him with wide eyes of admiration. Spider-Man kneels down to their level, reminding them that anyone can be a hero if they choose to help others."""
        },
        "wonderwoman": {
            "prompts": [
                "Wonder Woman standing heroically, her golden lasso and bracelets gleaming, an island paradise visible in the background.",
                "Wonder Woman deflecting bullets with her bracelets, her expression determined and fierce.",
                "Wonder Woman wielding her sword and shield, engaged in an epic battle against mythological creatures.",
                "Wonder Woman using her lasso of truth, the golden rope glowing with magical energy."
            ],
            "dialogues": [
                "I am Diana of Themyscira, daughter of Hippolyta.",
                "I fight for those who cannot fight for themselves.",
                "Peace is a virtue to be shown, not peace enforced.",
                "The truth will set you free."
            ],
            "positions": [(250, 70), (270, 60), (230, 100), (260, 80)],
            "story": """The sun rises over Themyscira, casting a golden glow on the island paradise. Diana stands proudly on a cliff overlooking the ocean, her armor gleaming in the morning light, ready to face whatever challenges the day may bring.

News reaches the Amazon princess of danger in the world of men. Without hesitation, Diana leaves her homeland behind. In the midst of chaos, Wonder Woman stands firm, her bracelets deflecting a barrage of bullets meant to harm innocent bystanders.

Ancient mythological creatures, released by a power-hungry sorcerer, terrorize a small village. Wonder Woman meets them in battle, her sword and shield moving with incredible speed and precision. Her training as an Amazon warrior guides each strike.

With the creatures defeated, Wonder Woman confronts the sorcerer. She wraps her golden Lasso of Truth around him, the mystical rope glowing with divine energy. As the magic of the lasso compels him to speak honestly, the truth behind his actions is revealed, and justice can finally be served."""
        }
    }

    return templates.get(template_name.lower(), None)

def get_available_templates():
    return ["Batman", "Spiderman", "WonderWoman"]

def generate_template_comic(template_name, num_panels=4):
    template_data = get_template_data(template_name.lower())

    if not template_data:
        print(f"Template '{template_name}' not found. Available templates: {', '.join(get_available_templates())}")
        return None, None, None, None

    try:
        pipe = load_sdxl_model()

        print(f"\nGenerating {template_name} comic panels...")
        panel_images = []

        print("Generating base character image...")
        base_prompt = f"A detailed illustration of {template_name} in a dynamic pose, full body visible"

        is_sdxl = isinstance(pipe, StableDiffusionXLPipeline)

        if is_sdxl:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                num_inference_steps=30
            ).images[0]
        else:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                num_inference_steps=30,
                guidance_scale=7.5
            ).images[0]

        base_image = base_image.resize((768, 768))

        prompts = template_data["prompts"][:num_panels]
        dialogues = template_data["dialogues"][:num_panels]
        positions = template_data["positions"][:num_panels]

        while len(prompts) < num_panels:
            prompts.append(f"{template_name} in an action scene")
        while len(dialogues) < num_panels:
            dialogues.append("")
        while len(positions) < num_panels:
            positions.append((256, 100))

        for i, prompt in enumerate(prompts):
            print(f"Generating image for panel {i+1}...")

            if is_sdxl:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.65,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                    num_inference_steps=30
                ).images[0]
            else:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.65,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                    num_inference_steps=30,
                    guidance_scale=7.5
                ).images[0]

            image = image.resize((512, 512))

            if dialogues[i]:
                image = add_speech_bubble(image, dialogues[i], positions[i])

            panel_images.append(image)

            image_path = f"panel_{i+1}.png"
            image.save(image_path)

        print("Creating combined comic strip...")
        rows = int(np.ceil(num_panels / 2))
        columns = min(2, num_panels)

        final_width = 512 * columns
        final_height = 512 * rows

        comic_strip = Image.new("RGB", (final_width, final_height), (255, 255, 255))

        for i, image in enumerate(panel_images):
            x_offset = (i % 2) * 512
            y_offset = (i // 2) * 512
            comic_strip.paste(image, (x_offset, y_offset))

        comic_strip.save(f"{template_name}_comic_strip.png")
        print(f"{template_name} comic generation complete!")

        # Use the pre-written story from the template data
        story_text = template_data["story"]
        story_panels = prompts

        return story_text, story_panels, panel_images, comic_strip

    except Exception as e:
        print(f"Error generating template comic: {e}")
        return None, None, None, None

def generate_comic(story_prompt, num_panels=4, model_path=None, art_style="cartoon style", use_template=None):
    if use_template and use_template.lower() != "none":
        return generate_template_comic(use_template, num_panels)

    if len(story_prompt) < 10:
        story_prompt = f"Create an interesting story about {story_prompt}"

    try:
        print("Loading fine-tuned model...")
        model, tokenizer = load_fine_tuned_model(model_path)

        if model is None or tokenizer is None:
            raise Exception("Failed to load the fine-tuned model")

        print(f"Generating story from prompt: '{story_prompt}'")
        story_text = generate_story(model, tokenizer, story_prompt)
        print("Story generated:")
        print(story_text)

        # If story generation still produces insufficient content after multiple attempts
        # and with different parameters, try generating with a completely different approach
        if not story_text or len(story_text) < 50:
            print("Story generation produced insufficient content. Using alternative generation approach.")
            story_text = generate_alternative_story(story_prompt)
            print("Alternative story generated:")
            print(story_text)

    except Exception as e:
        print(f"Error in model-based story generation: {e}")
        print("Using alternative story generation approach...")
        story_text = generate_alternative_story(story_prompt)

    print("\nSplitting story into panels...")
    story_panels, panel_texts, dialogues = split_story_into_panels(story_text, num_panels)
    for i, panel in enumerate(story_panels):
        print(f"Panel {i+1}: {panel}")
        if dialogues[i]:
            print(f"Dialogue for panel {i+1}: \"{dialogues[i]}\"")

    try:
        pipe = load_custom_model()

        print("\nGenerating panel images...")
        panel_images = []

        character_name = extract_character_name(story_prompt) or "the character"

        print("Generating base character image for consistency...")
        base_prompt = f"A friendly {art_style} illustration of {character_name}, whole body visible, simple background, bright colors, child-friendly"

        is_sdxl = isinstance(pipe, StableDiffusionXLPipeline)

        if is_sdxl:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                num_inference_steps=30
            ).images[0]
        else:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                num_inference_steps=30,
                guidance_scale=7.5
            ).images[0]

        base_image = base_image.resize((768, 768))

        for i, panel_text in enumerate(panel_texts):
            print(f"Generating image for panel {i+1}...")
            # Make the image prompts unique for each panel based on the actual story content
            prompt = f"A friendly {art_style} illustration for a children's storybook showing: {panel_text}, featuring {character_name}, bright colors, simple shapes, cute characters, child-friendly"

            if is_sdxl:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.7,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                    num_inference_steps=30
                ).images[0]
            else:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.7,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                    num_inference_steps=30,
                    guidance_scale=7.5
                ).images[0]

            image = image.resize((512, 512))

            if dialogues[i]:
                position = (256, 100 + (i % 2) * 50)
                image = add_speech_bubble(image, dialogues[i], position)

            panel_images.append(image)

            image_path = f"panel_{i+1}.png"
            image.save(image_path)

        print("Creating combined comic strip...")
        rows = int(np.ceil(num_panels / 2))
        columns = min(2, num_panels)

        final_width = 512 * columns
        final_height = 512 * rows

        comic_strip = Image.new("RGB", (final_width, final_height), (255, 255, 255))

        for i, image in enumerate(panel_images):
            x_offset = (i % 2) * 512
            y_offset = (i // 2) * 512
            comic_strip.paste(image, (x_offset, y_offset))

        comic_strip.save("comic_strip.png")
        print("Comic generation complete! All panels saved as PNG files and combined into comic_strip.png")

        return story_text, story_panels, panel_images, comic_strip

    except Exception as e:
        print(f"Error in image generation: {e}")
        print("Unable to generate images. Please check if you have enough GPU memory and proper installations.")
        return story_text, story_panels, None, None

# Web UI using Gradio
def create_ui():
    with gr.Blocks() as app:
        gr.Markdown("# Story to Comic Generator with InkoloRA")
        gr.Markdown("Enter a prompt to generate a story and convert it into comic panels with speech bubbles using the InkoloRA model")

        with gr.Row():
            with gr.Column():
                template_dropdown = gr.Dropdown(
                    choices=["None"] + get_available_templates(),
                    label="Choose Template (optional)",
                    value="None"
                )

                story_prompt = gr.Textbox(
                    label="Story Prompt",
                    placeholder="Enter a prompt for the children's story (e.g. 'a little boy who learns to fly')",
                    lines=2
                )

                num_panels = gr.Slider(
                    label="Number of Panels",
                    minimum=2,
                    maximum=8,
                    value=4,
                    step=1
                )
art_style = gr.Dropdown(
                    choices=["Cartoon Style", "Realistic", "Anime", "Sketch"],
                    label="Art Style",
                    value="Cartoon Style"
                )

                generate_button = gr.Button("Generate Comic")

            with gr.Column():
                output_text = gr.Textbox(
                    label="Generated Story",
                    placeholder="The generated story will appear here...",
                    lines=10,
                    interactive=False
                )
                 output_panels = gr.Gallery(
                    label="Comic Panels",
                    show_label=True,
                    elem_id="comic-panels",
                    height=200,
                    columns=2  # Set the number of columns for the gallery
                )

                output_comic = gr.Image(
                    label="Combined Comic Strip",
                    type="pil"
                )

        # Fixed function to handle the generation and properly return outputs
        def generate_comic_action(story_prompt, num_panels, art_style, use_template):
            story_text, story_panels, panel_images, comic_strip = generate_comic(
                story_prompt, num_panels, art_style=art_style, use_template=use_template
            )
# Return the outputs directly rather than updating them
            return story_text, panel_images if panel_images else [], comic_strip

        generate_button.click(
            generate_comic_action,
            inputs=[story_prompt, num_panels, art_style, template_dropdown],
            outputs=[output_text, output_panels, output_comic]
        )

    app.launch()

# Mount Google Drive if in Colab environment
try:
    drive.mount('/content/drive')
except:
    print("Not running in Colab or drive already mounted")

# Call the function to create the UI
if name == "main":
    create_ui()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 710)

In [ ]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import cv2
from transformers import AutoTokenizer, AutoModelForCausalLM
from diffusers import StableDiffusionPipeline, StableDiffusionXLPipeline
from PIL import Image, ImageDraw, ImageFont
import gradio as gr
from google.colab import drive

# Load the fine-tuned model
def load_fine_tuned_model(model_path=None):
    if not model_path:
        model_path = '/content/drive/MyDrive/fine_tuned_tiny_stories_model'

    try:
        print(f"Loading fine-tuned model from {model_path}...")
        model = AutoModelForCausalLM.from_pretrained(model_path)
        tokenizer = AutoTokenizer.from_pretrained(model_path)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)

        print(f"Model loaded successfully and moved to {device}")
        return model, tokenizer
    except Exception as e:
        print(f"Error loading model: {e}")
        # Try loading a fallback model if the fine-tuned one fails
        try:
            print("Attempting to load fallback model gpt2...")
            model = AutoModelForCausalLM.from_pretrained("gpt2")
            tokenizer = AutoTokenizer.from_pretrained("gpt2")
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model = model.to(device)
            print(f"Fallback model loaded successfully and moved to {device}")
            return model, tokenizer
        except Exception as e2:
            print(f"Error loading fallback model: {e2}")
            return None, None

# Helper functions for story generation
def extract_character_name(prompt):
    character_names = ["Spider-Man", "Peter Parker", "Batman", "Superman", "Wonder Woman",
                       "Iron Man", "Captain America", "Thor", "Hulk", "Black Widow"]

    for name in character_names:
        if name.lower() in prompt.lower():
            return name

    words = prompt.split()
    for word in words:
        if word[0].isupper() and len(word) > 1 and word.lower() not in ["write", "story", "about"]:
            return word

    return None

def extract_topic(prompt):
    cleaned = prompt.lower().replace("write", "").replace("story", "").replace("about", "")
    keywords = ["adventure", "flying", "jumping", "fighting", "learning", "playing", "making friends"]

    for keyword in keywords:
        if keyword in cleaned:
            return keyword

    return cleaned.strip()

def generate_story(model, tokenizer, prompt, max_length=400, retries=3):
    story_prompt = f"Write a simple children's story about {prompt}. Use simple language and short sentences."

    # Increase repetition penalty and adjust parameters for better generation
    for attempt in range(retries):
        try:
            input_ids = tokenizer.encode(story_prompt, return_tensors='pt')
            device = next(model.parameters()).device
            input_ids = input_ids.to(device)

            # Use different generation parameters for each retry attempt
            temperature = 0.7 + (attempt * 0.1)  # Gradually increase temperature with each retry
            repetition_penalty = 1.2 + (attempt * 0.1)  # Gradually increase repetition penalty

            output = model.generate(
                input_ids,
                max_length=max_length,
                num_return_sequences=1,
                no_repeat_ngram_size=3,  # Increased from 2
                top_k=50,  # Increased from 40
                top_p=0.92,  # Adjusted from 0.9
                temperature=temperature,
                do_sample=True,
                repetition_penalty=repetition_penalty
            )

            generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

            # Clean up the generated text to focus on the actual story
            if story_prompt in generated_text:
                generated_text = generated_text.replace(story_prompt, "").strip()

            # Check if we have a reasonable story
            if len(generated_text) > 100 and "." in generated_text:
                print(f"Successfully generated story on attempt {attempt+1}")
                return generated_text
            else:
                print(f"Attempt {attempt+1} produced insufficient content ({len(generated_text)} chars). Retrying...")

        except Exception as e:
            print(f"Error in story generation attempt {attempt+1}: {e}")

    # If all retries fail, generate a more varied story using a different approach
    print("All generation attempts failed. Using alternative generation method...")
    return generate_alternative_story(prompt)

def generate_alternative_story(prompt):
    """
    Generate a more varied story when the model-based generation fails.
    This creates a more dynamic story than the previous structured approach.
    """
    character = extract_character_name(prompt) or "the little hero"
    topic = extract_topic(prompt) or "having an adventure"

    # Create a list of possible story templates
    story_templates = [
        f"""Once upon a time, there was {character} who lived in a small village.
{character} loved {topic} more than anything else.

One day, {character} discovered something magical while playing outside. It was a tiny glowing ball that could float in the air.

"What's this?" wondered {character}. The ball started to move, and {character} followed it into the forest.

The ball led {character} to a hidden garden where other children were playing with similar magical balls. They welcomed {character} and taught them new games.

{character} had so much fun that day! When it was time to go home, one of the children gave {character} a magical ball of their own.

From that day on, {character} visited the hidden garden often and made many new friends. They all loved {topic} together and had wonderful adventures.""",

        f"""{character} woke up early one morning with a big smile. Today was the day for {topic}!

{character} packed a small bag with snacks and a water bottle. "I'm ready for my adventure!" {character} said.

Outside, {character} met a friendly cat. "Hello, cat! Would you like to join me for {topic}?" The cat meowed and followed along.

As they walked through the park, they found a lost teddy bear. "We should find who this belongs to," said {character}.

They asked everyone they met about the teddy bear. Finally, they met a little girl who was crying. "My teddy!" she said happily when she saw it.

"Thank you for helping me find my teddy," the girl smiled. "Would you like to play with me?"

{character}, the cat, and the girl spent the whole day playing and having fun with {topic}. It was the best day ever!""",

        f"""The sun was shining brightly when {character} decided it was the perfect day for {topic}.

"I've never tried {topic} before," {character} thought, "but today I will be brave!"

{character} began slowly, taking small steps. It wasn't easy at first, and {character} almost gave up.

"Don't worry, you can do it!" said a friendly voice. {character} looked up and saw a smiling butterfly.

With the butterfly's encouragement, {character} tried again. This time, {character} did much better!

Soon, {character} was having so much fun with {topic}. "Thank you for believing in me," {character} said to the butterfly.

The butterfly fluttered around happily. "Sometimes we just need a friend to help us be brave," it replied.

{character} went home that evening with a happy heart, excited to try {topic} again tomorrow."""
    ]

    import random
    return random.choice(story_templates)

def split_story_into_panels(story, num_panels=4):
    if len(story) < 100:
        print("Warning: Story is very short. Panel division may not be optimal.")

    paragraphs = story.split('\n')
    paragraphs = [p for p in paragraphs if p.strip()]

    dialogues = []
    panel_texts = []

    if len(paragraphs) >= num_panels:
        if len(paragraphs) > num_panels:
            step = len(paragraphs) / num_panels
            panels = []
            for i in range(num_panels):
                start_idx = int(i * step)
                end_idx = int((i + 1) * step) if i < num_panels - 1 else len(paragraphs)
                panel_text = " ".join(paragraphs[start_idx:end_idx])
                panels.append(panel_text)

                dialogue = extract_dialogue(panel_text)
                dialogues.append(dialogue)

                clean_text = panel_text.replace(dialogue, "") if dialogue else panel_text
                panel_texts.append(clean_text)
            return panels, panel_texts, dialogues
        else:
            panels = paragraphs[:num_panels]
            for panel in panels:
                dialogue = extract_dialogue(panel)
                dialogues.append(dialogue)
                clean_text = panel.replace(dialogue, "") if dialogue else panel
                panel_texts.append(clean_text)
            return panels, panel_texts, dialogues

    sentences = []
    for chunk in story.split('. '):
        sentences.extend([s.strip() + '.' for s in chunk.split('! ')])
    sentences = [s for s in sentences if len(s) > 1]

    if len(sentences) < num_panels:
        while len(sentences) < num_panels:
            longest_idx = max(range(len(sentences)), key=lambda i: len(sentences[i]))
            longest = sentences[longest_idx]

            if ', ' in longest:
                parts = longest.split(', ', 1)
                sentences[longest_idx] = parts[0] + '.'
                sentences.insert(longest_idx + 1, parts[1])
            else:
                sentences.append(f"Meanwhile, {longest.lower()}")

    step = max(1, len(sentences) // num_panels)

    panels = []
    for i in range(num_panels):
        start_idx = i * step
        end_idx = (i + 1) * step if i < num_panels - 1 else len(sentences)
        panel_text = " ".join(sentences[start_idx:end_idx])
        panels.append(panel_text)

        dialogue = extract_dialogue(panel_text)
        dialogues.append(dialogue)

        clean_text = panel_text.replace(dialogue, "") if dialogue else panel_text
        panel_texts.append(clean_text)

    return panels, panel_texts, dialogues

def extract_dialogue(text):
    import re
    dialogue_matches = re.findall(r'"([^"]*)"', text)
    if dialogue_matches:
        return dialogue_matches[0]
    return ""

def load_custom_model():
    print("Loading InkoloRA model - this may take a while...")

    custom_model_path = "/content/drive/MyDrive/InkoloRA.safetensors"

    if not os.path.exists(custom_model_path):
        print(f"Warning: Custom model not found at {custom_model_path}. Falling back to Stable Diffusion XL.")
        return load_sdxl_model()

    try:
        pipe = StableDiffusionPipeline.from_pretrained(
            "runwayml/stable-diffusion-v1-5",
            torch_dtype=torch.float16,
            safety_checker=None
        )

        pipe.unet.load_attn_procs(custom_model_path)

        device = "cuda" if torch.cuda.is_available() else "cpu"
        if device == "cpu":
            print("Warning: CUDA not available, using CPU. This will be very slow.")
        pipe = pipe.to(device)

        print("InkoloRA model loaded successfully!")
        return pipe
    except Exception as e:
        print(f"Error loading custom model: {e}")
        print("Falling back to Stable Diffusion XL...")
        return load_sdxl_model()

def load_sdxl_model():
    print("Loading SDXL model - this may take a while...")
    pipe = StableDiffusionXLPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True
    )

    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device == "cpu":
        print("Warning: CUDA not available, using CPU. This will be very slow.")
    pipe = pipe.to(device)

    return pipe

def add_speech_bubble(image, text, position=None):
    if not text:
        return image

    img = np.array(image.convert("RGBA"))
    height, width, _ = img.shape

    if position is None:
        position = (width // 2, height // 4)

    bubble = np.zeros((height, width, 4), dtype=np.uint8)

    bubble_width = min(300, max(150, len(text) * 10))
    bubble_height = min(130, max(60, (len(text) // 20 + 1) * 30))

    cv2.ellipse(bubble, position, (bubble_width, bubble_height), 0, 0, 360, (255, 255, 255, 255), -1)

    pointer_x = position[0]
    pointer_y = position[1] + bubble_height
    triangle_pts = np.array([
        [pointer_x - 20, pointer_y - 10],
        [pointer_x + 20, pointer_y - 10],
        [pointer_x, pointer_y + 30]
    ])
    cv2.fillPoly(bubble, [triangle_pts], (255, 255, 255, 255))

    bubble_pil = Image.fromarray(bubble)

    draw = ImageDraw.Draw(bubble_pil)

    try:
        font = ImageFont.truetype("arial.ttf", 24)
    except:
        try:
            font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 24)
        except:
            font = ImageFont.load_default()

    wrapped_text = wrap_text(text, font, bubble_width - 40)

    text_x = position[0] - (bubble_width // 2) + 20
    text_y = position[1] - (bubble_height // 2) + 10

    for i, line in enumerate(wrapped_text):
        draw.text((text_x, text_y + i * 30), line, fill="black", font=font)

    return Image.alpha_composite(image.convert("RGBA"), bubble_pil)

def wrap_text(text, font, max_width):
    words = text.split()
    wrapped_lines = []
    current_line = []

    for word in words:
        test_line = ' '.join(current_line + [word])
        try:
            try:
                bbox = font.getbbox(test_line)
                line_width = bbox[2] - bbox[0]
            except AttributeError:
                try:
                    line_width = font.getlength(test_line)
                except AttributeError:
                    line_width = font.getsize(test_line)[0]
        except:
            line_width = len(test_line) * 10

        if line_width <= max_width:
            current_line.append(word)
        else:
            wrapped_lines.append(' '.join(current_line))
            current_line = [word]

    if current_line:
        wrapped_lines.append(' '.join(current_line))

    return wrapped_lines

def get_template_data(template_name):
    templates = {
        "batman": {
            "prompts": [
                "Batman standing on a dark Gotham rooftop, his cape flowing in the wind, the Bat-Signal glowing in the sky behind him.",
                "Close-up shot of Batman's determined face, shadows partially covering his mask, eyes glowing in the darkness.",
                "Batman leaping from a rooftop, his cape spread like wings as he descends towards a group of criminals.",
                "Batman delivering a powerful punch to a thug, the action captured in a comic-style motion blur."
            ],
            "dialogues": [
                "Gotham never sleeps, and neither do I.",
                "Justice isn't a choice. It's a responsibility.",
                "Fear is a tool. I use it against them.",
                "Crime has no place in my city."
            ],
            "positions": [(250, 70), (300, 50), (220, 120), (280, 140)],
            "story": """The Dark Knight watches over Gotham City from high above. Tonight is like any other night for Batman - a city full of shadows that need his protection.

As the Bat-Signal illuminates the cloudy sky, Batman's eyes narrow with determination. The signal isn't just a call for help - it's a reminder of the oath he took to protect the innocent.

Batman spots a group of criminals attempting to rob a local shop. He leaps from the rooftop, his cape billowing around him like massive wings. The criminals look up in terror as his silhouette blots out the moonlight.

With swift, calculated movements, Batman takes down the criminals one by one. Another night in Gotham, another battle won. But Batman knows his work is never truly done. As long as there is crime in Gotham, he will be there to fight it."""
        },
        "spiderman": {
            "prompts": [
                "Spider-Man perched on the edge of a skyscraper, overlooking New York City, his iconic red and blue suit vibrant against the skyline.",
                "Spider-Man shooting webs from his wrists, swinging between buildings with great agility.",
                "Spider-Man facing off against a villain, in a dynamic action pose, ready to fight.",
                "Spider-Man helping a civilian, showcasing his friendly neighborhood hero nature."
            ],
            "dialogues": [
                "With great power comes great responsibility.",
                "Just your friendly neighborhood Spider-Man!",
                "My spider-sense is tingling!",
                "I'm not going to let anyone get hurt on my watch."
            ],
            "positions": [(250, 70), (280, 80), (240, 110), (260, 90)],
            "story": """Peter Parker sits perched on the edge of a towering skyscraper, overlooking the bustling streets of New York City. His uncle Ben's words echo in his mind as he surveys the city he's sworn to protect.

With a flick of his wrist, Spider-Man launches himself off the building, shooting a web that attaches to a nearby structure. He swings gracefully through the concrete canyons of Manhattan, the wind rushing past his mask as pedestrians below point and cheer.

Suddenly, his spider-sense alerts him to danger. A bank robbery is in progress just a few blocks away. Spider-Man changes direction mid-swing, heading straight for the trouble. As he arrives, he sees masked robbers threatening innocent civilians.

With swift movements and well-placed webs, Spider-Man subdues the criminals and ensures everyone is safe. A young child approaches him with wide eyes of admiration. Spider-Man kneels down to their level, reminding them that anyone can be a hero if they choose to help others."""
        },
        "wonderwoman": {
            "prompts": [
                "Wonder Woman standing heroically, her golden lasso and bracelets gleaming, an island paradise visible in the background.",
                "Wonder Woman deflecting bullets with her bracelets, her expression determined and fierce.",
                "Wonder Woman wielding her sword and shield, engaged in an epic battle against mythological creatures.",
                "Wonder Woman using her lasso of truth, the golden rope glowing with magical energy."
            ],
            "dialogues": [
                "I am Diana of Themyscira, daughter of Hippolyta.",
                "I fight for those who cannot fight for themselves.",
                "Peace is a virtue to be shown, not peace enforced.",
                "The truth will set you free."
            ],
            "positions": [(250, 70), (270, 60), (230, 100), (260, 80)],
            "story": """The sun rises over Themyscira, casting a golden glow on the island paradise. Diana stands proudly on a cliff overlooking the ocean, her armor gleaming in the morning light, ready to face whatever challenges the day may bring.

News reaches the Amazon princess of danger in the world of men. Without hesitation, Diana leaves her homeland behind. In the midst of chaos, Wonder Woman stands firm, her bracelets deflecting a barrage of bullets meant to harm innocent bystanders.

Ancient mythological creatures, released by a power-hungry sorcerer, terrorize a small village. Wonder Woman meets them in battle, her sword and shield moving with incredible speed and precision. Her training as an Amazon warrior guides each strike.

With the creatures defeated, Wonder Woman confronts the sorcerer. She wraps her golden Lasso of Truth around him, the mystical rope glowing with divine energy. As the magic of the lasso compels him to speak honestly, the truth behind his actions is revealed, and justice can finally be served."""
        }
    }

    return templates.get(template_name.lower(), None)

def get_available_templates():
    return ["Batman", "Spiderman", "WonderWoman"]

def generate_template_comic(template_name, num_panels=4):
    template_data = get_template_data(template_name.lower())

    if not template_data:
        print(f"Template '{template_name}' not found. Available templates: {', '.join(get_available_templates())}")
        return None, None, None, None

    try:
        pipe = load_sdxl_model()

        print(f"\nGenerating {template_name} comic panels...")
        panel_images = []

        print("Generating base character image...")
        base_prompt = f"A detailed illustration of {template_name} in a dynamic pose, full body visible"

        is_sdxl = isinstance(pipe, StableDiffusionXLPipeline)

        if is_sdxl:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                num_inference_steps=30
            ).images[0]
        else:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                num_inference_steps=30,
                guidance_scale=7.5
            ).images[0]

        base_image = base_image.resize((768, 768))

        prompts = template_data["prompts"][:num_panels]
        dialogues = template_data["dialogues"][:num_panels]
        positions = template_data["positions"][:num_panels]

        while len(prompts) < num_panels:
            prompts.append(f"{template_name} in an action scene")
        while len(dialogues) < num_panels:
            dialogues.append("")
        while len(positions) < num_panels:
            positions.append((256, 100))

        for i, prompt in enumerate(prompts):
            print(f"Generating image for panel {i+1}...")

            if is_sdxl:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.65,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                    num_inference_steps=30
                ).images[0]
            else:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.65,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                    num_inference_steps=30,
                    guidance_scale=7.5
                ).images[0]

            image = image.resize((512, 512))

            if dialogues[i]:
                image = add_speech_bubble(image, dialogues[i], positions[i])

            panel_images.append(image)

            image_path = f"panel_{i+1}.png"
            image.save(image_path)

        print("Creating combined comic strip...")
        rows = int(np.ceil(num_panels / 2))
        columns = min(2, num_panels)

        final_width = 512 * columns
        final_height = 512 * rows

        comic_strip = Image.new("RGB", (final_width, final_height), (255, 255, 255))

        for i, image in enumerate(panel_images):
            x_offset = (i % 2) * 512
            y_offset = (i // 2) * 512
            comic_strip.paste(image, (x_offset, y_offset))

        comic_strip.save(f"{template_name}_comic_strip.png")
        print(f"{template_name} comic generation complete!")

        # Use the pre-written story from the template data
        story_text = template_data["story"]
        story_panels = prompts

        return story_text, story_panels, panel_images, comic_strip

    except Exception as e:
        print(f"Error generating template comic: {e}")
        return None, None, None, None

def generate_comic(story_prompt, num_panels=4, model_path=None, art_style="cartoon style", use_template=None):
    if use_template and use_template.lower() != "none":
        return generate_template_comic(use_template, num_panels)

    if len(story_prompt) < 10:
        story_prompt = f"Create an interesting story about {story_prompt}"

    try:
        print("Loading fine-tuned model...")
        model, tokenizer = load_fine_tuned_model(model_path)

        if model is None or tokenizer is None:
            raise Exception("Failed to load the fine-tuned model")

        print(f"Generating story from prompt: '{story_prompt}'")
        story_text = generate_story(model, tokenizer, story_prompt)
        print("Story generated:")
        print(story_text)

        # If story generation still produces insufficient content after multiple attempts
        # and with different parameters, try generating with a completely different approach
        if not story_text or len(story_text) < 50:
            print("Story generation produced insufficient content. Using alternative generation approach.")
            story_text = generate_alternative_story(story_prompt)
            print("Alternative story generated:")
            print(story_text)

    except Exception as e:
        print(f"Error in model-based story generation: {e}")
        print("Using alternative story generation approach...")
        story_text = generate_alternative_story(story_prompt)

    print("\nSplitting story into panels...")
    story_panels, panel_texts, dialogues = split_story_into_panels(story_text, num_panels)
    for i, panel in enumerate(story_panels):
        print(f"Panel {i+1}: {panel}")
        if dialogues[i]:
            print(f"Dialogue for panel {i+1}: \"{dialogues[i]}\"")

    try:
        pipe = load_custom_model()

        print("\nGenerating panel images...")
        panel_images = []

        character_name = extract_character_name(story_prompt) or "the character"

        print("Generating base character image for consistency...")
        base_prompt = f"A friendly {art_style} illustration of {character_name}, whole body visible, simple background, bright colors, child-friendly"

        is_sdxl = isinstance(pipe, StableDiffusionXLPipeline)

        if is_sdxl:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                num_inference_steps=30
            ).images[0]
        else:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                num_inference_steps=30,
                guidance_scale=7.5
            ).images[0]

        base_image = base_image.resize((768, 768))

        for i, panel_text in enumerate(panel_texts):
            print(f"Generating image for panel {i+1}...")
            # Make the image prompts unique for each panel based on the actual story content
            prompt = f"A friendly {art_style} illustration for a children's storybook showing: {panel_text}, featuring {character_name}, bright colors, simple shapes, cute characters, child-friendly"

            if is_sdxl:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.7,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                    num_inference_steps=30
                ).images[0]
            else:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.7,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                    num_inference_steps=30,
                    guidance_scale=7.5
                ).images[0]

            image = image.resize((512, 512))

            if dialogues[i]:
                position = (256, 100 + (i % 2) * 50)
                image = add_speech_bubble(image, dialogues[i], position)

            panel_images.append(image)

            image_path = f"panel_{i+1}.png"
            image.save(image_path)

        print("Creating combined comic strip...")
        rows = int(np.ceil(num_panels / 2))
        columns = min(2, num_panels)

        final_width = 512 * columns
        final_height = 512 * rows

        comic_strip = Image.new("RGB", (final_width, final_height), (255, 255, 255))

        for i, image in enumerate(panel_images):
            x_offset = (i % 2) * 512
            y_offset = (i // 2) * 512
            comic_strip.paste(image, (x_offset, y_offset))

        comic_strip.save("comic_strip.png")
        print("Comic generation complete! All panels saved as PNG files and combined into comic_strip.png")

        return story_text, story_panels, panel_images, comic_strip

    except Exception as e:
        print(f"Error in image generation: {e}")
        print("Unable to generate images. Please check if you have enough GPU memory and proper installations.")
        return story_text, story_panels, None, None

# Web UI using Gradio
def create_ui():
    with gr.Blocks() as app:
        gr.Markdown("# Story to Comic Generator with InkoloRA")
        gr.Markdown("Enter a prompt to generate a story and convert it into comic panels with speech bubbles using the InkoloRA model")

        with gr.Row():
            with gr.Column():
                template_dropdown = gr.Dropdown(
                    choices=["None"] + get_available_templates(),
                    label="Choose Template (optional)",
                    value="None"
                )

                story_prompt = gr.Textbox(
                    label="Story Prompt",
                    placeholder="Enter a prompt for the children's story (e.g. 'a little boy who learns to fly')",
                    lines=2
                )

                num_panels = gr.Slider(
                    label="Number of Panels",
                    minimum=2,
                    maximum=8,
                    value=4,
                    step=1
                )

                art_style = gr.Dropdown(
                    choices=["Cartoon Style", "Realistic", "Anime", "Sketch"],
                    label="Art Style",
                    value="Cartoon Style"
                )

                generate_button = gr.Button("Generate Comic")

            with gr.Column():
                output_text = gr.Textbox(
                    label="Generated Story",
                    placeholder="The generated story will appear here...",
                    lines=10,
                    interactive=False
                )

                output_panels = gr.Gallery(
                    label="Comic Panels",
                    show_label=True,
                    elem_id="comic-panels",
                    height=200,
                    columns=2
                )

                output_comic = gr.Image(
                    label="Combined Comic Strip",
                    type="pil"
                )

        # Fixed function to handle the generation and properly return outputs
        def generate_comic_action(story_prompt, num_panels, art_style, use_template):
            story_text, story_panels, panel_images, comic_strip = generate_comic(
                story_prompt, num_panels, art_style=art_style, use_template=use_template
            )
            # Return the outputs directly rather than updating them
            return story_text, panel_images if panel_images else [], comic_strip

        generate_button.click(
            generate_comic_action,
            inputs=[story_prompt, num_panels, art_style, template_dropdown],
            outputs=[output_text, output_panels, output_comic]
        )

    app.launch()

# Mount Google Drive if in Colab environment
try:
    drive.mount('/content/drive')
except:
    print("Not running in Colab or drive already mounted")

# Call the function to create the UI
if __name__ == "__main__":  # Fixed from 'name' to 'name_'
    create_ui()

KeyboardInterrupt: 

In [ ]:
import torch
print(torch.cuda.is_available())  # Should print True if GPU is available

True


In [ ]:
!pip install llama-cpp-python diffusers torch torchvision transformers accelerate numpy pillow opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 8.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.2 MB/s eta 0:00:00


In [ ]:
!pip install llama-cpp-python torch diffusers transformers accelerate safetensors matplotlib Pillow gradio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import cv2
from transformers import AutoTokenizer, AutoModelForCausalLM
from diffusers import StableDiffusionPipeline, StableDiffusionXLPipeline
from PIL import Image, ImageDraw, ImageFont
import gradio as gr
from google.colab import drive

# Load the fine-tuned model
def load_fine_tuned_model(model_path=None):
    if not model_path:
        model_path = '/content/drive/MyDrive/fine_tuned_tiny_stories_model'

    try:
        print(f"Loading fine-tuned model from {model_path}...")
        model = AutoModelForCausalLM.from_pretrained(model_path)
        tokenizer = AutoTokenizer.from_pretrained(model_path)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)

        print(f"Model loaded successfully and moved to {device}")
        return model, tokenizer
    except Exception as e:
        print(f"Error loading model: {e}")
        # Try loading a fallback model if the fine-tuned one fails
        try:
            print("Attempting to load fallback model gpt2...")
            model = AutoModelForCausalLM.from_pretrained("gpt2")
            tokenizer = AutoTokenizer.from_pretrained("gpt2")
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model = model.to(device)
            print(f"Fallback model loaded successfully and moved to {device}")
            return model, tokenizer
        except Exception as e2:
            print(f"Error loading fallback model: {e2}")
            return None, None

# Helper functions for story generation
def extract_character_name(prompt):
    character_names = ["Spider-Man", "Peter Parker", "Batman", "Superman", "Wonder Woman",
                       "Iron Man", "Captain America", "Thor", "Hulk", "Black Widow"]

    for name in character_names:
        if name.lower() in prompt.lower():
            return name

    words = prompt.split()
    for word in words:
        if word[0].isupper() and len(word) > 1 and word.lower() not in ["write", "story", "about"]:
            return word

    return None

def extract_topic(prompt):
    cleaned = prompt.lower().replace("write", "").replace("story", "").replace("about", "")
    keywords = ["adventure", "flying", "jumping", "fighting", "learning", "playing", "making friends"]

    for keyword in keywords:
        if keyword in cleaned:
            return keyword

    return cleaned.strip()

def generate_story(model, tokenizer, prompt, max_length=400, retries=3):
    story_prompt = f"Write a simple children's story about {prompt}. Use simple language and short sentences."

    # Increase repetition penalty and adjust parameters for better generation
    for attempt in range(retries):
        try:
            input_ids = tokenizer.encode(story_prompt, return_tensors='pt')
            device = next(model.parameters()).device
            input_ids = input_ids.to(device)

            # Use different generation parameters for each retry attempt
            temperature = 0.7 + (attempt * 0.1)  # Gradually increase temperature with each retry
            repetition_penalty = 1.2 + (attempt * 0.1)  # Gradually increase repetition penalty

            output = model.generate(
                input_ids,
                max_length=max_length,
                num_return_sequences=1,
                no_repeat_ngram_size=3,  # Increased from 2
                top_k=50,  # Increased from 40
                top_p=0.92,  # Adjusted from 0.9
                temperature=temperature,
                do_sample=True,
                repetition_penalty=repetition_penalty
            )

            generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

            # Clean up the generated text to focus on the actual story
            if story_prompt in generated_text:
                generated_text = generated_text.replace(story_prompt, "").strip()

            # Check if we have a reasonable story
            if len(generated_text) > 100 and "." in generated_text:
                print(f"Successfully generated story on attempt {attempt+1}")
                return generated_text
            else:
                print(f"Attempt {attempt+1} produced insufficient content ({len(generated_text)} chars). Retrying...")

        except Exception as e:
            print(f"Error in story generation attempt {attempt+1}: {e}")

    # If all retries fail, generate a more varied story using a different approach
    print("All generation attempts failed. Using alternative generation method...")
    return generate_alternative_story(prompt)

def generate_alternative_story(prompt):
    """
    Generate a more varied story when the model-based generation fails.
    This creates a more dynamic story than the previous structured approach.
    """
    character = extract_character_name(prompt) or "the little hero"
    topic = extract_topic(prompt) or "having an adventure"

    # Create a list of possible story templates
    story_templates = [
        f"""Once upon a time, there was {character} who lived in a small village.
{character} loved {topic} more than anything else.

One day, {character} discovered something magical while playing outside. It was a tiny glowing ball that could float in the air.

"What's this?" wondered {character}. The ball started to move, and {character} followed it into the forest.

The ball led {character} to a hidden garden where other children were playing with similar magical balls. They welcomed {character} and taught them new games.

{character} had so much fun that day! When it was time to go home, one of the children gave {character} a magical ball of their own.

From that day on, {character} visited the hidden garden often and made many new friends. They all loved {topic} together and had wonderful adventures.""",

        f"""{character} woke up early one morning with a big smile. Today was the day for {topic}!

{character} packed a small bag with snacks and a water bottle. "I'm ready for my adventure!" {character} said.

Outside, {character} met a friendly cat. "Hello, cat! Would you like to join me for {topic}?" The cat meowed and followed along.

As they walked through the park, they found a lost teddy bear. "We should find who this belongs to," said {character}.

They asked everyone they met about the teddy bear. Finally, they met a little girl who was crying. "My teddy!" she said happily when she saw it.

"Thank you for helping me find my teddy," the girl smiled. "Would you like to play with me?"

{character}, the cat, and the girl spent the whole day playing and having fun with {topic}. It was the best day ever!""",

        f"""The sun was shining brightly when {character} decided it was the perfect day for {topic}.

"I've never tried {topic} before," {character} thought, "but today I will be brave!"

{character} began slowly, taking small steps. It wasn't easy at first, and {character} almost gave up.

"Don't worry, you can do it!" said a friendly voice. {character} looked up and saw a smiling butterfly.

With the butterfly's encouragement, {character} tried again. This time, {character} did much better!

Soon, {character} was having so much fun with {topic}. "Thank you for believing in me," {character} said to the butterfly.

The butterfly fluttered around happily. "Sometimes we just need a friend to help us be brave," it replied.

{character} went home that evening with a happy heart, excited to try {topic} again tomorrow."""
    ]

    import random
    return random.choice(story_templates)

def split_story_into_panels(story, num_panels=4):
    if len(story) < 100:
        print("Warning: Story is very short. Panel division may not be optimal.")

    paragraphs = story.split('\n')
    paragraphs = [p for p in paragraphs if p.strip()]

    dialogues = []
    panel_texts = []

    if len(paragraphs) >= num_panels:
        if len(paragraphs) > num_panels:
            step = len(paragraphs) / num_panels
            panels = []
            for i in range(num_panels):
                start_idx = int(i * step)
                end_idx = int((i + 1) * step) if i < num_panels - 1 else len(paragraphs)
                panel_text = " ".join(paragraphs[start_idx:end_idx])
                panels.append(panel_text)

                dialogue = extract_dialogue(panel_text)
                dialogues.append(dialogue)

                clean_text = panel_text.replace(dialogue, "") if dialogue else panel_text
                panel_texts.append(clean_text)
            return panels, panel_texts, dialogues
        else:
            panels = paragraphs[:num_panels]
            for panel in panels:
                dialogue = extract_dialogue(panel)
                dialogues.append(dialogue)
                clean_text = panel.replace(dialogue, "") if dialogue else panel
                panel_texts.append(clean_text)
            return panels, panel_texts, dialogues

    sentences = []
    for chunk in story.split('. '):
        sentences.extend([s.strip() + '.' for s in chunk.split('! ')])
    sentences = [s for s in sentences if len(s) > 1]

    if len(sentences) < num_panels:
        while len(sentences) < num_panels:
            longest_idx = max(range(len(sentences)), key=lambda i: len(sentences[i]))
            longest = sentences[longest_idx]

            if ', ' in longest:
                parts = longest.split(', ', 1)
                sentences[longest_idx] = parts[0] + '.'
                sentences.insert(longest_idx + 1, parts[1])
            else:
                sentences.append(f"Meanwhile, {longest.lower()}")

    step = max(1, len(sentences) // num_panels)

    panels = []
    for i in range(num_panels):
        start_idx = i * step
        end_idx = (i + 1) * step if i < num_panels - 1 else len(sentences)
        panel_text = " ".join(sentences[start_idx:end_idx])
        panels.append(panel_text)

        dialogue = extract_dialogue(panel_text)
        dialogues.append(dialogue)

        clean_text = panel_text.replace(dialogue, "") if dialogue else panel_text
        panel_texts.append(clean_text)

    return panels, panel_texts, dialogues

def extract_dialogue(text):
    import re
    dialogue_matches = re.findall(r'"([^"]*)"', text)
    if dialogue_matches:
        return dialogue_matches[0]
    return ""

def load_custom_model():
    print("Loading InkoloRA model - this may take a while...")

    custom_model_path = "/content/drive/MyDrive/InkoloRA.safetensors"

    if not os.path.exists(custom_model_path):
        print(f"Warning: Custom model not found at {custom_model_path}. Falling back to Stable Diffusion XL.")
        return load_sdxl_model()

    try:
        pipe = StableDiffusionPipeline.from_pretrained(
            "runwayml/stable-diffusion-v1-5",
            torch_dtype=torch.float16,
            safety_checker=None
        )

        pipe.unet.load_attn_procs(custom_model_path)

        device = "cuda" if torch.cuda.is_available() else "cpu"
        if device == "cpu":
            print("Warning: CUDA not available, using CPU. This will be very slow.")
        pipe = pipe.to(device)

        print("InkoloRA model loaded successfully!")
        return pipe
    except Exception as e:
        print(f"Error loading custom model: {e}")
        print("Falling back to Stable Diffusion XL...")
        return load_sdxl_model()

def load_sdxl_model():
    print("Loading SDXL model - this may take a while...")
    pipe = StableDiffusionXLPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True
    )

    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device == "cpu":
        print("Warning: CUDA not available, using CPU. This will be very slow.")
    pipe = pipe.to(device)

    return pipe

def add_speech_bubble(image, text, position=None):
    if not text:
        return image

    img = np.array(image.convert("RGBA"))
    height, width, _ = img.shape

    if position is None:
        position = (width // 2, height // 4)

    bubble = np.zeros((height, width, 4), dtype=np.uint8)

    bubble_width = min(300, max(150, len(text) * 10))
    bubble_height = min(130, max(60, (len(text) // 20 + 1) * 30))

    cv2.ellipse(bubble, position, (bubble_width, bubble_height), 0, 0, 360, (255, 255, 255, 255), -1)

    pointer_x = position[0]
    pointer_y = position[1] + bubble_height
    triangle_pts = np.array([
        [pointer_x - 20, pointer_y - 10],
        [pointer_x + 20, pointer_y - 10],
        [pointer_x, pointer_y + 30]
    ])
    cv2.fillPoly(bubble, [triangle_pts], (255, 255, 255, 255))

    bubble_pil = Image.fromarray(bubble)

    draw = ImageDraw.Draw(bubble_pil)

    try:
        font = ImageFont.truetype("arial.ttf", 24)
    except:
        try:
            font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 24)
        except:
            font = ImageFont.load_default()

    wrapped_text = wrap_text(text, font, bubble_width - 40)

    text_x = position[0] - (bubble_width // 2) + 20
    text_y = position[1] - (bubble_height // 2) + 10

    for i, line in enumerate(wrapped_text):
        draw.text((text_x, text_y + i * 30), line, fill="black", font=font)

    return Image.alpha_composite(image.convert("RGBA"), bubble_pil)

def wrap_text(text, font, max_width):
    words = text.split()
    wrapped_lines = []
    current_line = []

    for word in words:
        test_line = ' '.join(current_line + [word])
        try:
            try:
                bbox = font.getbbox(test_line)
                line_width = bbox[2] - bbox[0]
            except AttributeError:
                try:
                    line_width = font.getlength(test_line)
                except AttributeError:
                    line_width = font.getsize(test_line)[0]
        except:
            line_width = len(test_line) * 10

        if line_width <= max_width:
            current_line.append(word)
        else:
            wrapped_lines.append(' '.join(current_line))
            current_line = [word]

    if current_line:
        wrapped_lines.append(' '.join(current_line))

    return wrapped_lines

def get_template_data(template_name):
    templates = {
        "batman": {
            "prompts": [
                "Batman standing on a dark Gotham rooftop, his cape flowing in the wind, the Bat-Signal glowing in the sky behind him.",
                "Close-up shot of Batman's determined face, shadows partially covering his mask, eyes glowing in the darkness.",
                "Batman leaping from a rooftop, his cape spread like wings as he descends towards a group of criminals.",
                "Batman delivering a powerful punch to a thug, the action captured in a comic-style motion blur."
            ],
            "dialogues": [
                "Gotham never sleeps, and neither do I.",
                "Justice isn't a choice. It's a responsibility.",
                "Fear is a tool. I use it against them.",
                "Crime has no place in my city."
            ],
            "positions": [(250, 70), (300, 50), (220, 120), (280, 140)],
            "story": """The Dark Knight watches over Gotham City from high above. Tonight is like any other night for Batman - a city full of shadows that need his protection.

As the Bat-Signal illuminates the cloudy sky, Batman's eyes narrow with determination. The signal isn't just a call for help - it's a reminder of the oath he took to protect the innocent.

Batman spots a group of criminals attempting to rob a local shop. He leaps from the rooftop, his cape billowing around him like massive wings. The criminals look up in terror as his silhouette blots out the moonlight.

With swift, calculated movements, Batman takes down the criminals one by one. Another night in Gotham, another battle won. But Batman knows his work is never truly done. As long as there is crime in Gotham, he will be there to fight it."""
        },
        "spiderman": {
            "prompts": [
                "Spider-Man perched on the edge of a skyscraper, overlooking New York City, his iconic red and blue suit vibrant against the skyline.",
                "Spider-Man shooting webs from his wrists, swinging between buildings with great agility.",
                "Spider-Man facing off against a villain, in a dynamic action pose, ready to fight.",
                "Spider-Man helping a civilian, showcasing his friendly neighborhood hero nature."
            ],
            "dialogues": [
                "With great power comes great responsibility.",
                "Just your friendly neighborhood Spider-Man!",
                "My spider-sense is tingling!",
                "I'm not going to let anyone get hurt on my watch."
            ],
            "positions": [(250, 70), (280, 80), (240, 110), (260, 90)],
            "story": """Peter Parker sits perched on the edge of a towering skyscraper, overlooking the bustling streets of New York City. His uncle Ben's words echo in his mind as he surveys the city he's sworn to protect.

With a flick of his wrist, Spider-Man launches himself off the building, shooting a web that attaches to a nearby structure. He swings gracefully through the concrete canyons of Manhattan, the wind rushing past his mask as pedestrians below point and cheer.

Suddenly, his spider-sense alerts him to danger. A bank robbery is in progress just a few blocks away. Spider-Man changes direction mid-swing, heading straight for the trouble. As he arrives, he sees masked robbers threatening innocent civilians.

With swift movements and well-placed webs, Spider-Man subdues the criminals and ensures everyone is safe. A young child approaches him with wide eyes of admiration. Spider-Man kneels down to their level, reminding them that anyone can be a hero if they choose to help others."""
        },
        "wonderwoman": {
            "prompts": [
                "Wonder Woman standing heroically, her golden lasso and bracelets gleaming, an island paradise visible in the background.",
                "Wonder Woman deflecting bullets with her bracelets, her expression determined and fierce.",
                "Wonder Woman wielding her sword and shield, engaged in an epic battle against mythological creatures.",
                "Wonder Woman using her lasso of truth, the golden rope glowing with magical energy."
            ],
            "dialogues": [
                "I am Diana of Themyscira, daughter of Hippolyta.",
                "I fight for those who cannot fight for themselves.",
                "Peace is a virtue to be shown, not peace enforced.",
                "The truth will set you free."
            ],
            "positions": [(250, 70), (270, 60), (230, 100), (260, 80)],
            "story": """The sun rises over Themyscira, casting a golden glow on the island paradise. Diana stands proudly on a cliff overlooking the ocean, her armor gleaming in the morning light, ready to face whatever challenges the day may bring.

News reaches the Amazon princess of danger in the world of men. Without hesitation, Diana leaves her homeland behind. In the midst of chaos, Wonder Woman stands firm, her bracelets deflecting a barrage of bullets meant to harm innocent bystanders.

Ancient mythological creatures, released by a power-hungry sorcerer, terrorize a small village. Wonder Woman meets them in battle, her sword and shield moving with incredible speed and precision. Her training as an Amazon warrior guides each strike.

With the creatures defeated, Wonder Woman confronts the sorcerer. She wraps her golden Lasso of Truth around him, the mystical rope glowing with divine energy. As the magic of the lasso compels him to speak honestly, the truth behind his actions is revealed, and justice can finally be served."""
        }
    }

    return templates.get(template_name.lower(), None)

def get_available_templates():
    return ["Batman", "Spiderman", "WonderWoman"]

def generate_template_comic(template_name, num_panels=4):
    template_data = get_template_data(template_name.lower())

    if not template_data:
        print(f"Template '{template_name}' not found. Available templates: {', '.join(get_available_templates())}")
        return None, None, None, None

    try:
        pipe = load_sdxl_model()

        print(f"\nGenerating {template_name} comic panels...")
        panel_images = []

        print("Generating base character image...")
        base_prompt = f"A detailed illustration of {template_name} in a dynamic pose, full body visible"

        is_sdxl = isinstance(pipe, StableDiffusionXLPipeline)

        if is_sdxl:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                num_inference_steps=30
            ).images[0]
        else:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                num_inference_steps=30,
                guidance_scale=7.5
            ).images[0]

        base_image = base_image.resize((768, 768))

        prompts = template_data["prompts"][:num_panels]
        dialogues = template_data["dialogues"][:num_panels]
        positions = template_data["positions"][:num_panels]

        while len(prompts) < num_panels:
            prompts.append(f"{template_name} in an action scene")
        while len(dialogues) < num_panels:
            dialogues.append("")
        while len(positions) < num_panels:
            positions.append((256, 100))

        for i, prompt in enumerate(prompts):
            print(f"Generating image for panel {i+1}...")

            if is_sdxl:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.65,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                    num_inference_steps=30
                ).images[0]
            else:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.65,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                    num_inference_steps=30,
                    guidance_scale=7.5
                ).images[0]

            image = image.resize((512, 512))

            if dialogues[i]:
                image = add_speech_bubble(image, dialogues[i], positions[i])

            panel_images.append(image)

            image_path = f"panel_{i+1}.png"
            image.save(image_path)

        print("Creating combined comic strip...")
        rows = int(np.ceil(num_panels / 2))
        columns = min(2, num_panels)

        final_width = 512 * columns
        final_height = 512 * rows

        comic_strip = Image.new("RGB", (final_width, final_height), (255, 255, 255))

        for i, image in enumerate(panel_images):
            x_offset = (i % 2) * 512
            y_offset = (i // 2) * 512
            comic_strip.paste(image, (x_offset, y_offset))

        comic_strip.save(f"{template_name}_comic_strip.png")
        print(f"{template_name} comic generation complete!")

        # Use the pre-written story from the template data
        story_text = template_data["story"]
        story_panels = prompts

        return story_text, story_panels, panel_images, comic_strip

    except Exception as e:
        print(f"Error generating template comic: {e}")
        return None, None, None, None

def generate_comic(story_prompt, num_panels=4, model_path=None, art_style="cartoon style", use_template=None):
    if use_template and use_template.lower() != "none":
        return generate_template_comic(use_template, num_panels)

    if len(story_prompt) < 10:
        story_prompt = f"Create an interesting story about {story_prompt}"

    try:
        print("Loading fine-tuned model...")
        model, tokenizer = load_fine_tuned_model(model_path)

        if model is None or tokenizer is None:
            raise Exception("Failed to load the fine-tuned model")

        print(f"Generating story from prompt: '{story_prompt}'")
        story_text = generate_story(model, tokenizer, story_prompt)
        print("Story generated:")
        print(story_text)

        # If story generation still produces insufficient content after multiple attempts
        # and with different parameters, try generating with a completely different approach
        if not story_text or len(story_text) < 50:
            print("Story generation produced insufficient content. Using alternative generation approach.")
            story_text = generate_alternative_story(story_prompt)
            print("Alternative story generated:")
            print(story_text)

    except Exception as e:
        print(f"Error in model-based story generation: {e}")
        print("Using alternative story generation approach...")
        story_text = generate_alternative_story(story_prompt)

    print("\nSplitting story into panels...")
    story_panels, panel_texts, dialogues = split_story_into_panels(story_text, num_panels)
    for i, panel in enumerate(story_panels):
        print(f"Panel {i+1}: {panel}")
        if dialogues[i]:
            print(f"Dialogue for panel {i+1}: \"{dialogues[i]}\"")

    try:
        pipe = load_custom_model()

        print("\nGenerating panel images...")
        panel_images = []

        character_name = extract_character_name(story_prompt) or "the character"

        print("Generating base character image for consistency...")
        base_prompt = f"A friendly {art_style} illustration of {character_name}, whole body visible, simple background, bright colors, child-friendly"

        is_sdxl = isinstance(pipe, StableDiffusionXLPipeline)

        if is_sdxl:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                num_inference_steps=30
            ).images[0]
        else:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                num_inference_steps=30,
                guidance_scale=7.5
            ).images[0]

        base_image = base_image.resize((768, 768))

        for i, panel_text in enumerate(panel_texts):
            print(f"Generating image for panel {i+1}...")
            # Make the image prompts unique for each panel based on the actual story content
            prompt = f"A friendly {art_style} illustration for a children's storybook showing: {panel_text}, featuring {character_name}, bright colors, simple shapes, cute characters, child-friendly"

            if is_sdxl:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.7,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                    num_inference_steps=30
                ).images[0]
            else:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.7,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                    num_inference_steps=30,
                    guidance_scale=7.5
                ).images[0]

            image = image.resize((512, 512))

            if dialogues[i]:
                position = (256, 100 + (i % 2) * 50)
                image = add_speech_bubble(image, dialogues[i], position)

            panel_images.append(image)

            image_path = f"panel_{i+1}.png"
            image.save(image_path)

        print("Creating combined comic strip...")
        rows = int(np.ceil(num_panels / 2))
        columns = min(2, num_panels)

        final_width = 512 * columns
        final_height = 512 * rows

        comic_strip = Image.new("RGB", (final_width, final_height), (255, 255, 255))

        for i, image in enumerate(panel_images):
            x_offset = (i % 2) * 512
            y_offset = (i // 2) * 512
            comic_strip.paste(image, (x_offset, y_offset))

        comic_strip.save("comic_strip.png")
        print("Comic generation complete! All panels saved as PNG files and combined into comic_strip.png")

        return story_text, story_panels, panel_images, comic_strip

    except Exception as e:
        print(f"Error in image generation: {e}")
        print("Unable to generate images. Please check if you have enough GPU memory and proper installations.")
        return story_text, story_panels, None, None

# Web UI using Gradio
def create_ui():
    with gr.Blocks() as app:
        gr.Markdown("# Story to Comic Generator with InkoloRA")
        gr.Markdown("Enter a prompt to generate a story and convert it into comic panels with speech bubbles using the InkoloRA model")

        with gr.Row():
            with gr.Column():
                template_dropdown = gr.Dropdown(
                    choices=["None"] + get_available_templates(),
                    label="Choose Template (optional)",
                    value="None"
                )

                story_prompt = gr.Textbox(
                    label="Story Prompt",
                    placeholder="Enter a prompt for the children's story (e.g. 'a little boy who learns to fly')",
                    lines=2
                )

                num_panels = gr.Slider(
                    label="Number of Panels",
                    minimum=2,
                    maximum=8,
                    value=4,
                    step=1
                )

                art_style = gr.Dropdown(
                    choices=["Cartoon Style", "Realistic", "Anime", "Sketch"],
                    label="Art Style",
                    value="Cartoon Style"
                )

                generate_button = gr.Button("Generate Comic")

            with gr.Column():
                output_text = gr.Textbox(
                    label="Generated Story",
                    placeholder="The generated story will appear here...",
                    lines=10,
                    interactive=False
                )

                output_panels = gr.Gallery(
                    label="Comic Panels",
                    show_label=True,
                    elem_id="comic-panels",
                    height=200,
                    columns=2
                )

                output_comic = gr.Image(
                    label="Combined Comic Strip",
                    type="pil"
                )

        # Fixed function to handle the generation and properly return outputs
        def generate_comic_action(story_prompt, num_panels, art_style, use_template):
            story_text, story_panels, panel_images, comic_strip = generate_comic(
                story_prompt, num_panels, art_style=art_style, use_template=use_template
            )
            # Return the outputs directly rather than updating them
            return story_text, panel_images if panel_images else [], comic_strip

        generate_button.click(
            generate_comic_action,
            inputs=[story_prompt, num_panels, art_style, template_dropdown],
            outputs=[output_text, output_panels, output_comic]
        )

    app.launch()

# Mount Google Drive if in Colab environment
try:
    drive.mount('/content/drive')
except:
    print("Not running in Colab or drive already mounted")

# Call the function to create the UI
if __name__ == "__main__":  # Fixed from 'name' to 'name_'
    create_ui()

ModuleNotFoundError: No module named 'gradio'

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 125.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.4 MB/s eta 0:00:00


In [ ]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import cv2
from transformers import AutoTokenizer, AutoModelForCausalLM
from diffusers import StableDiffusionPipeline, StableDiffusionXLPipeline
from PIL import Image, ImageDraw, ImageFont
import gradio as gr
from google.colab import drive

# Load the fine-tuned model
def load_fine_tuned_model(model_path=None):
    if not model_path:
        model_path = '/content/drive/MyDrive/fine_tuned_tiny_stories_model'

    try:
        print(f"Loading fine-tuned model from {model_path}...")
        model = AutoModelForCausalLM.from_pretrained(model_path)
        tokenizer = AutoTokenizer.from_pretrained(model_path)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)

        print(f"Model loaded successfully and moved to {device}")
        return model, tokenizer
    except Exception as e:
        print(f"Error loading model: {e}")
        # Try loading a fallback model if the fine-tuned one fails
        try:
            print("Attempting to load fallback model gpt2...")
            model = AutoModelForCausalLM.from_pretrained("gpt2")
            tokenizer = AutoTokenizer.from_pretrained("gpt2")
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model = model.to(device)
            print(f"Fallback model loaded successfully and moved to {device}")
            return model, tokenizer
        except Exception as e2:
            print(f"Error loading fallback model: {e2}")
            return None, None

# Helper functions for story generation
def extract_character_name(prompt):
    character_names = ["Spider-Man", "Peter Parker", "Batman", "Superman", "Wonder Woman",
                       "Iron Man", "Captain America", "Thor", "Hulk", "Black Widow"]

    for name in character_names:
        if name.lower() in prompt.lower():
            return name

    words = prompt.split()
    for word in words:
        if word[0].isupper() and len(word) > 1 and word.lower() not in ["write", "story", "about"]:
            return word

    return None

def extract_topic(prompt):
    cleaned = prompt.lower().replace("write", "").replace("story", "").replace("about", "")
    keywords = ["adventure", "flying", "jumping", "fighting", "learning", "playing", "making friends"]

    for keyword in keywords:
        if keyword in cleaned:
            return keyword

    return cleaned.strip()

def generate_story(model, tokenizer, prompt, max_length=400, retries=3):
    story_prompt = f"Write a simple children's story about {prompt}. Use simple language and short sentences."

    # Increase repetition penalty and adjust parameters for better generation
    for attempt in range(retries):
        try:
            input_ids = tokenizer.encode(story_prompt, return_tensors='pt')
            device = next(model.parameters()).device
            input_ids = input_ids.to(device)

            # Use different generation parameters for each retry attempt
            temperature = 0.7 + (attempt * 0.1)  # Gradually increase temperature with each retry
            repetition_penalty = 1.2 + (attempt * 0.1)  # Gradually increase repetition penalty

            output = model.generate(
                input_ids,
                max_length=max_length,
                num_return_sequences=1,
                no_repeat_ngram_size=3,  # Increased from 2
                top_k=50,  # Increased from 40
                top_p=0.92,  # Adjusted from 0.9
                temperature=temperature,
                do_sample=True,
                repetition_penalty=repetition_penalty
            )

            generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

            # Clean up the generated text to focus on the actual story
            if story_prompt in generated_text:
                generated_text = generated_text.replace(story_prompt, "").strip()

            # Check if we have a reasonable story
            if len(generated_text) > 100 and "." in generated_text:
                print(f"Successfully generated story on attempt {attempt+1}")
                return generated_text
            else:
                print(f"Attempt {attempt+1} produced insufficient content ({len(generated_text)} chars). Retrying...")

        except Exception as e:
            print(f"Error in story generation attempt {attempt+1}: {e}")

    # If all retries fail, generate a more varied story using a different approach
    print("All generation attempts failed. Using alternative generation method...")
    return generate_alternative_story(prompt)

def generate_alternative_story(prompt):
    """
    Generate a more varied story when the model-based generation fails.
    This creates a more dynamic story than the previous structured approach.
    """
    character = extract_character_name(prompt) or "the little hero"
    topic = extract_topic(prompt) or "having an adventure"

    # Create a list of possible story templates
    story_templates = [
        f"""Once upon a time, there was {character} who lived in a small village.
{character} loved {topic} more than anything else.

One day, {character} discovered something magical while playing outside. It was a tiny glowing ball that could float in the air.

"What's this?" wondered {character}. The ball started to move, and {character} followed it into the forest.

The ball led {character} to a hidden garden where other children were playing with similar magical balls. They welcomed {character} and taught them new games.

{character} had so much fun that day! When it was time to go home, one of the children gave {character} a magical ball of their own.

From that day on, {character} visited the hidden garden often and made many new friends. They all loved {topic} together and had wonderful adventures.""",

        f"""{character} woke up early one morning with a big smile. Today was the day for {topic}!

{character} packed a small bag with snacks and a water bottle. "I'm ready for my adventure!" {character} said.

Outside, {character} met a friendly cat. "Hello, cat! Would you like to join me for {topic}?" The cat meowed and followed along.

As they walked through the park, they found a lost teddy bear. "We should find who this belongs to," said {character}.

They asked everyone they met about the teddy bear. Finally, they met a little girl who was crying. "My teddy!" she said happily when she saw it.

"Thank you for helping me find my teddy," the girl smiled. "Would you like to play with me?"

{character}, the cat, and the girl spent the whole day playing and having fun with {topic}. It was the best day ever!""",

        f"""The sun was shining brightly when {character} decided it was the perfect day for {topic}.

"I've never tried {topic} before," {character} thought, "but today I will be brave!"

{character} began slowly, taking small steps. It wasn't easy at first, and {character} almost gave up.

"Don't worry, you can do it!" said a friendly voice. {character} looked up and saw a smiling butterfly.

With the butterfly's encouragement, {character} tried again. This time, {character} did much better!

Soon, {character} was having so much fun with {topic}. "Thank you for believing in me," {character} said to the butterfly.

The butterfly fluttered around happily. "Sometimes we just need a friend to help us be brave," it replied.

{character} went home that evening with a happy heart, excited to try {topic} again tomorrow."""
    ]

    import random
    return random.choice(story_templates)

def split_story_into_panels(story, num_panels=4):
    if len(story) < 100:
        print("Warning: Story is very short. Panel division may not be optimal.")

    paragraphs = story.split('\n')
    paragraphs = [p for p in paragraphs if p.strip()]

    dialogues = []
    panel_texts = []

    if len(paragraphs) >= num_panels:
        if len(paragraphs) > num_panels:
            step = len(paragraphs) / num_panels
            panels = []
            for i in range(num_panels):
                start_idx = int(i * step)
                end_idx = int((i + 1) * step) if i < num_panels - 1 else len(paragraphs)
                panel_text = " ".join(paragraphs[start_idx:end_idx])
                panels.append(panel_text)

                dialogue = extract_dialogue(panel_text)
                dialogues.append(dialogue)

                clean_text = panel_text.replace(dialogue, "") if dialogue else panel_text
                panel_texts.append(clean_text)
            return panels, panel_texts, dialogues
        else:
            panels = paragraphs[:num_panels]
            for panel in panels:
                dialogue = extract_dialogue(panel)
                dialogues.append(dialogue)
                clean_text = panel.replace(dialogue, "") if dialogue else panel
                panel_texts.append(clean_text)
            return panels, panel_texts, dialogues

    sentences = []
    for chunk in story.split('. '):
        sentences.extend([s.strip() + '.' for s in chunk.split('! ')])
    sentences = [s for s in sentences if len(s) > 1]

    if len(sentences) < num_panels:
        while len(sentences) < num_panels:
            longest_idx = max(range(len(sentences)), key=lambda i: len(sentences[i]))
            longest = sentences[longest_idx]

            if ', ' in longest:
                parts = longest.split(', ', 1)
                sentences[longest_idx] = parts[0] + '.'
                sentences.insert(longest_idx + 1, parts[1])
            else:
                sentences.append(f"Meanwhile, {longest.lower()}")

    step = max(1, len(sentences) // num_panels)

    panels = []
    for i in range(num_panels):
        start_idx = i * step
        end_idx = (i + 1) * step if i < num_panels - 1 else len(sentences)
        panel_text = " ".join(sentences[start_idx:end_idx])
        panels.append(panel_text)

        dialogue = extract_dialogue(panel_text)
        dialogues.append(dialogue)

        clean_text = panel_text.replace(dialogue, "") if dialogue else panel_text
        panel_texts.append(clean_text)

    return panels, panel_texts, dialogues

def extract_dialogue(text):
    import re
    dialogue_matches = re.findall(r'"([^"]*)"', text)
    if dialogue_matches:
        return dialogue_matches[0]
    return ""

def load_custom_model():
    print("Loading InkoloRA model - this may take a while...")

    custom_model_path = "/content/drive/MyDrive/InkoloRA.safetensors"

    if not os.path.exists(custom_model_path):
        print(f"Warning: Custom model not found at {custom_model_path}. Falling back to Stable Diffusion XL.")
        return load_sdxl_model()

    try:
        pipe = StableDiffusionPipeline.from_pretrained(
            "runwayml/stable-diffusion-v1-5",
            torch_dtype=torch.float16,
            safety_checker=None
        )

        pipe.unet.load_attn_procs(custom_model_path)

        device = "cuda" if torch.cuda.is_available() else "cpu"
        if device == "cpu":
            print("Warning: CUDA not available, using CPU. This will be very slow.")
        pipe = pipe.to(device)

        print("InkoloRA model loaded successfully!")
        return pipe
    except Exception as e:
        print(f"Error loading custom model: {e}")
        print("Falling back to Stable Diffusion XL...")
        return load_sdxl_model()

def load_sdxl_model():
    print("Loading SDXL model - this may take a while...")
    pipe = StableDiffusionXLPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True
    )

    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device == "cpu":
        print("Warning: CUDA not available, using CPU. This will be very slow.")
    pipe = pipe.to(device)

    return pipe

def add_speech_bubble(image, text, position=None):
    if not text:
        return image

    img = np.array(image.convert("RGBA"))
    height, width, _ = img.shape

    if position is None:
        position = (width // 2, height // 4)

    bubble = np.zeros((height, width, 4), dtype=np.uint8)

    bubble_width = min(300, max(150, len(text) * 10))
    bubble_height = min(130, max(60, (len(text) // 20 + 1) * 30))

    cv2.ellipse(bubble, position, (bubble_width, bubble_height), 0, 0, 360, (255, 255, 255, 255), -1)

    pointer_x = position[0]
    pointer_y = position[1] + bubble_height
    triangle_pts = np.array([
        [pointer_x - 20, pointer_y - 10],
        [pointer_x + 20, pointer_y - 10],
        [pointer_x, pointer_y + 30]
    ])
    cv2.fillPoly(bubble, [triangle_pts], (255, 255, 255, 255))

    bubble_pil = Image.fromarray(bubble)

    draw = ImageDraw.Draw(bubble_pil)

    try:
        font = ImageFont.truetype("arial.ttf", 24)
    except:
        try:
            font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 24)
        except:
            font = ImageFont.load_default()

    wrapped_text = wrap_text(text, font, bubble_width - 40)

    text_x = position[0] - (bubble_width // 2) + 20
    text_y = position[1] - (bubble_height // 2) + 10

    for i, line in enumerate(wrapped_text):
        draw.text((text_x, text_y + i * 30), line, fill="black", font=font)

    return Image.alpha_composite(image.convert("RGBA"), bubble_pil)

def wrap_text(text, font, max_width):
    words = text.split()
    wrapped_lines = []
    current_line = []

    for word in words:
        test_line = ' '.join(current_line + [word])
        try:
            try:
                bbox = font.getbbox(test_line)
                line_width = bbox[2] - bbox[0]
            except AttributeError:
                try:
                    line_width = font.getlength(test_line)
                except AttributeError:
                    line_width = font.getsize(test_line)[0]
        except:
            line_width = len(test_line) * 10

        if line_width <= max_width:
            current_line.append(word)
        else:
            wrapped_lines.append(' '.join(current_line))
            current_line = [word]

    if current_line:
        wrapped_lines.append(' '.join(current_line))

    return wrapped_lines

def get_template_data(template_name):
    templates = {
        "batman": {
            "prompts": [
                "Batman standing on a dark Gotham rooftop, his cape flowing in the wind, the Bat-Signal glowing in the sky behind him.",
                "Close-up shot of Batman's determined face, shadows partially covering his mask, eyes glowing in the darkness.",
                "Batman leaping from a rooftop, his cape spread like wings as he descends towards a group of criminals.",
                "Batman delivering a powerful punch to a thug, the action captured in a comic-style motion blur."
            ],
            "dialogues": [
                "Gotham never sleeps, and neither do I.",
                "Justice isn't a choice. It's a responsibility.",
                "Fear is a tool. I use it against them.",
                "Crime has no place in my city."
            ],
            "positions": [(250, 70), (300, 50), (220, 120), (280, 140)],
            "story": """The Dark Knight watches over Gotham City from high above. Tonight is like any other night for Batman - a city full of shadows that need his protection.

As the Bat-Signal illuminates the cloudy sky, Batman's eyes narrow with determination. The signal isn't just a call for help - it's a reminder of the oath he took to protect the innocent.

Batman spots a group of criminals attempting to rob a local shop. He leaps from the rooftop, his cape billowing around him like massive wings. The criminals look up in terror as his silhouette blots out the moonlight.

With swift, calculated movements, Batman takes down the criminals one by one. Another night in Gotham, another battle won. But Batman knows his work is never truly done. As long as there is crime in Gotham, he will be there to fight it."""
        },
        "spiderman": {
            "prompts": [
                "Spider-Man perched on the edge of a skyscraper, overlooking New York City, his iconic red and blue suit vibrant against the skyline.",
                "Spider-Man shooting webs from his wrists, swinging between buildings with great agility.",
                "Spider-Man facing off against a villain, in a dynamic action pose, ready to fight.",
                "Spider-Man helping a civilian, showcasing his friendly neighborhood hero nature."
            ],
            "dialogues": [
                "With great power comes great responsibility.",
                "Just your friendly neighborhood Spider-Man!",
                "My spider-sense is tingling!",
                "I'm not going to let anyone get hurt on my watch."
            ],
            "positions": [(250, 70), (280, 80), (240, 110), (260, 90)],
            "story": """Peter Parker sits perched on the edge of a towering skyscraper, overlooking the bustling streets of New York City. His uncle Ben's words echo in his mind as he surveys the city he's sworn to protect.

With a flick of his wrist, Spider-Man launches himself off the building, shooting a web that attaches to a nearby structure. He swings gracefully through the concrete canyons of Manhattan, the wind rushing past his mask as pedestrians below point and cheer.

Suddenly, his spider-sense alerts him to danger. A bank robbery is in progress just a few blocks away. Spider-Man changes direction mid-swing, heading straight for the trouble. As he arrives, he sees masked robbers threatening innocent civilians.

With swift movements and well-placed webs, Spider-Man subdues the criminals and ensures everyone is safe. A young child approaches him with wide eyes of admiration. Spider-Man kneels down to their level, reminding them that anyone can be a hero if they choose to help others."""
        },
        "wonderwoman": {
            "prompts": [
                "Wonder Woman standing heroically, her golden lasso and bracelets gleaming, an island paradise visible in the background.",
                "Wonder Woman deflecting bullets with her bracelets, her expression determined and fierce.",
                "Wonder Woman wielding her sword and shield, engaged in an epic battle against mythological creatures.",
                "Wonder Woman using her lasso of truth, the golden rope glowing with magical energy."
            ],
            "dialogues": [
                "I am Diana of Themyscira, daughter of Hippolyta.",
                "I fight for those who cannot fight for themselves.",
                "Peace is a virtue to be shown, not peace enforced.",
                "The truth will set you free."
            ],
            "positions": [(250, 70), (270, 60), (230, 100), (260, 80)],
            "story": """The sun rises over Themyscira, casting a golden glow on the island paradise. Diana stands proudly on a cliff overlooking the ocean, her armor gleaming in the morning light, ready to face whatever challenges the day may bring.

News reaches the Amazon princess of danger in the world of men. Without hesitation, Diana leaves her homeland behind. In the midst of chaos, Wonder Woman stands firm, her bracelets deflecting a barrage of bullets meant to harm innocent bystanders.

Ancient mythological creatures, released by a power-hungry sorcerer, terrorize a small village. Wonder Woman meets them in battle, her sword and shield moving with incredible speed and precision. Her training as an Amazon warrior guides each strike.

With the creatures defeated, Wonder Woman confronts the sorcerer. She wraps her golden Lasso of Truth around him, the mystical rope glowing with divine energy. As the magic of the lasso compels him to speak honestly, the truth behind his actions is revealed, and justice can finally be served."""
        }
    }

    return templates.get(template_name.lower(), None)

def get_available_templates():
    return ["Batman", "Spiderman", "WonderWoman"]

def generate_template_comic(template_name, num_panels=4):
    template_data = get_template_data(template_name.lower())

    if not template_data:
        print(f"Template '{template_name}' not found. Available templates: {', '.join(get_available_templates())}")
        return None, None, None, None

    try:
        pipe = load_sdxl_model()

        print(f"\nGenerating {template_name} comic panels...")
        panel_images = []

        print("Generating base character image...")
        base_prompt = f"A detailed illustration of {template_name} in a dynamic pose, full body visible"

        is_sdxl = isinstance(pipe, StableDiffusionXLPipeline)

        if is_sdxl:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                num_inference_steps=30
            ).images[0]
        else:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                num_inference_steps=30,
                guidance_scale=7.5
            ).images[0]

        base_image = base_image.resize((768, 768))

        prompts = template_data["prompts"][:num_panels]
        dialogues = template_data["dialogues"][:num_panels]
        positions = template_data["positions"][:num_panels]

        while len(prompts) < num_panels:
            prompts.append(f"{template_name} in an action scene")
        while len(dialogues) < num_panels:
            dialogues.append("")
        while len(positions) < num_panels:
            positions.append((256, 100))

        for i, prompt in enumerate(prompts):
            print(f"Generating image for panel {i+1}...")

            if is_sdxl:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.65,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                    num_inference_steps=30
                ).images[0]
            else:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.65,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                    num_inference_steps=30,
                    guidance_scale=7.5
                ).images[0]

            image = image.resize((512, 512))

            if dialogues[i]:
                image = add_speech_bubble(image, dialogues[i], positions[i])

            panel_images.append(image)

            image_path = f"panel_{i+1}.png"
            image.save(image_path)

        print("Creating combined comic strip...")
        rows = int(np.ceil(num_panels / 2))
        columns = min(2, num_panels)

        final_width = 512 * columns
        final_height = 512 * rows

        comic_strip = Image.new("RGB", (final_width, final_height), (255, 255, 255))

        for i, image in enumerate(panel_images):
            x_offset = (i % 2) * 512
            y_offset = (i // 2) * 512
            comic_strip.paste(image, (x_offset, y_offset))

        comic_strip.save(f"{template_name}_comic_strip.png")
        print(f"{template_name} comic generation complete!")

        # Use the pre-written story from the template data
        story_text = template_data["story"]
        story_panels = prompts

        return story_text, story_panels, panel_images, comic_strip

    except Exception as e:
        print(f"Error generating template comic: {e}")
        return None, None, None, None

def generate_comic(story_prompt, num_panels=4, model_path=None, art_style="cartoon style", use_template=None):
    if use_template and use_template.lower() != "none":
        return generate_template_comic(use_template, num_panels)

    if len(story_prompt) < 10:
        story_prompt = f"Create an interesting story about {story_prompt}"

    try:
        print("Loading fine-tuned model...")
        model, tokenizer = load_fine_tuned_model(model_path)

        if model is None or tokenizer is None:
            raise Exception("Failed to load the fine-tuned model")

        print(f"Generating story from prompt: '{story_prompt}'")
        story_text = generate_story(model, tokenizer, story_prompt)
        print("Story generated:")
        print(story_text)

        # If story generation still produces insufficient content after multiple attempts
        # and with different parameters, try generating with a completely different approach
        if not story_text or len(story_text) < 50:
            print("Story generation produced insufficient content. Using alternative generation approach.")
            story_text = generate_alternative_story(story_prompt)
            print("Alternative story generated:")
            print(story_text)

    except Exception as e:
        print(f"Error in model-based story generation: {e}")
        print("Using alternative story generation approach...")
        story_text = generate_alternative_story(story_prompt)

    print("\nSplitting story into panels...")
    story_panels, panel_texts, dialogues = split_story_into_panels(story_text, num_panels)
    for i, panel in enumerate(story_panels):
        print(f"Panel {i+1}: {panel}")
        if dialogues[i]:
            print(f"Dialogue for panel {i+1}: \"{dialogues[i]}\"")

    try:
        pipe = load_custom_model()

        print("\nGenerating panel images...")
        panel_images = []

        character_name = extract_character_name(story_prompt) or "the character"

        print("Generating base character image for consistency...")
        base_prompt = f"A friendly {art_style} illustration of {character_name}, whole body visible, simple background, bright colors, child-friendly"

        is_sdxl = isinstance(pipe, StableDiffusionXLPipeline)

        if is_sdxl:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                num_inference_steps=30
            ).images[0]
        else:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                num_inference_steps=30,
                guidance_scale=7.5
            ).images[0]

        base_image = base_image.resize((768, 768))

        for i, panel_text in enumerate(panel_texts):
            print(f"Generating image for panel {i+1}...")
            # Make the image prompts unique for each panel based on the actual story content
            prompt = f"A friendly {art_style} illustration for a children's storybook showing: {panel_text}, featuring {character_name}, bright colors, simple shapes, cute characters, child-friendly"

            if is_sdxl:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.7,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                    num_inference_steps=30
                ).images[0]
            else:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.7,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                    num_inference_steps=30,
                    guidance_scale=7.5
                ).images[0]

            image = image.resize((512, 512))

            if dialogues[i]:
                position = (256, 100 + (i % 2) * 50)
                image = add_speech_bubble(image, dialogues[i], position)

            panel_images.append(image)

            image_path = f"panel_{i+1}.png"
            image.save(image_path)

        print("Creating combined comic strip...")
        rows = int(np.ceil(num_panels / 2))
        columns = min(2, num_panels)

        final_width = 512 * columns
        final_height = 512 * rows

        comic_strip = Image.new("RGB", (final_width, final_height), (255, 255, 255))

        for i, image in enumerate(panel_images):
            x_offset = (i % 2) * 512
            y_offset = (i // 2) * 512
            comic_strip.paste(image, (x_offset, y_offset))

        comic_strip.save("comic_strip.png")
        print("Comic generation complete! All panels saved as PNG files and combined into comic_strip.png")

        return story_text, story_panels, panel_images, comic_strip

    except Exception as e:
        print(f"Error in image generation: {e}")
        print("Unable to generate images. Please check if you have enough GPU memory and proper installations.")
        return story_text, story_panels, None, None

# Web UI using Gradio
def create_ui():
    with gr.Blocks() as app:
        gr.Markdown("# Story to Comic Generator with InkoloRA")
        gr.Markdown("Enter a prompt to generate a story and convert it into comic panels with speech bubbles using the InkoloRA model")

        with gr.Row():
            with gr.Column():
                template_dropdown = gr.Dropdown(
                    choices=["None"] + get_available_templates(),
                    label="Choose Template (optional)",
                    value="None"
                )

                story_prompt = gr.Textbox(
                    label="Story Prompt",
                    placeholder="Enter a prompt for the children's story (e.g. 'a little boy who learns to fly')",
                    lines=2
                )

                num_panels = gr.Slider(
                    label="Number of Panels",
                    minimum=2,
                    maximum=8,
                    value=4,
                    step=1
                )

                art_style = gr.Dropdown(
                    choices=["Cartoon Style", "Realistic", "Anime", "Sketch"],
                    label="Art Style",
                    value="Cartoon Style"
                )

                generate_button = gr.Button("Generate Comic")

            with gr.Column():
                output_text = gr.Textbox(
                    label="Generated Story",
                    placeholder="The generated story will appear here...",
                    lines=10,
                    interactive=False
                )

                output_panels = gr.Gallery(
                    label="Comic Panels",
                    show_label=True,
                    elem_id="comic-panels",
                    height=200,
                    columns=2
                )

                output_comic = gr.Image(
                    label="Combined Comic Strip",
                    type="pil"
                )

        # Fixed function to handle the generation and properly return outputs
        def generate_comic_action(story_prompt, num_panels, art_style, use_template):
            story_text, story_panels, panel_images, comic_strip = generate_comic(
                story_prompt, num_panels, art_style=art_style, use_template=use_template
            )
            # Return the outputs directly rather than updating them
            return story_text, panel_images if panel_images else [], comic_strip

        generate_button.click(
            generate_comic_action,
            inputs=[story_prompt, num_panels, art_style, template_dropdown],
            outputs=[output_text, output_panels, output_comic]
        )

    app.launch()

# Mount Google Drive if in Colab environment
try:
    drive.mount('/content/drive')
except:
    print("Not running in Colab or drive already mounted")

# Call the function to create the UI
if __name__ == "__main__":  # Fixed from 'name' to 'name_'
    create_ui()

Mounted at /content/drive
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7d3227aabc390bc96c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
